In [0]:
period_value_ld_df = spark.sql("SELECT SUBSTRING(legacyperiod, 1, 6) as period_value_ld from qtdf_assu_faas_frh_uc.frheng1.temp_legacysddates")
period_value_ld_list = [row.period_value_ld for row in period_value_ld_df.collect()]
period_value_ld = ",".join(["'" + value + "'" for value in period_value_ld_list])
print(period_value_ld)

In [0]:

a = spark.sql("SELECT count(*) FROM qtdf_assu_faas_frh_uc.frheng1.trialbalance WHERE EntityID_fk=6709 and AccountingStdID_fk=1 and IsActive = true  AND Createddate <= '2024-05-20 13:56:10' and SUBSTRING(Period,1,6) IN ("+period_value_ld+")")

a.display()

#### Notebook Tag
 | Type  | Value | 
 | -----------  | ----------- | 
 | **Description** | Notebook takes data from TB, Adjustment , Disclosure and SCOA Table and writes data for multiperiod which is input data for staging calculations|
 | **Input Data**| Raw tables from catalog and sql database |
 | **Output Data**| Combined Data of Dates, Adjustments and Disclosures |  

#### 1. Import Libraries

In [0]:
from pyspark.sql.functions import col,expr,explode,when,lit,concat,current_timestamp,months_between,date_diff,coalesce,substring,instr, abs,window,row_number,date_add, to_date, left, right, max,split,concat_ws,length,datediff,month, monotonically_increasing_id, regexp_extract, first, date_format
from pyspark.sql import functions as F
from pyspark.sql.types import (StructType,StructField,StringType,IntegerType,TimestampType,DoubleType,DateType,DecimalType,BooleanType,FloatType)
from pyspark.sql.window import Window
from functools import reduce
from collections import defaultdict
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta
import json
from pyspark.sql.functions import from_json
import logging
from pyspark.sql import Row

In [0]:
#dbutils.widgets.removeAll()

#### 2. Parameters & Variables
##### Widgets to pass as parameters from ADF pipeline

In [0]:
# dbutils.widgets.text("EntityID", "50")
# dbutils.widgets.text("Bunit", "0")
# dbutils.widgets.text("CategoryID", "1488")
# dbutils.widgets.text("FinancialYear", "2025")
# dbutils.widgets.text("AccountingStandardID","5")  
# # dbutils.widgets.text("PeriodValue","Year-202204-202303")  # to test extended period less than 12 months Dimension entity id : 3052, catgeoryid: 487, acc startard id : 1, finacial year : 2023
# # dbutils.widgets.text("PeriodValue","Year-202204-202303")  # to test standard period Dimension entity id : 3050, catgeoryid: 395, acc startard id : 41, finacial year : 2023
# # dbutils.widgets.text("PeriodValue","Year-202110-202301")  # to test extended period more than 12 months Dimension entityid: 3051, acc stadtrd id :41 , category id : 437, year : 2023
# # dbutils.widgets.text("PeriodValue","Year-202302-202401")  # to test not an extended period Dimension entityid: 3051, acc stadtrd id :41 , category id : 438, year : 2024
# dbutils.widgets.text("PeriodValue","Year-202404-202503") # to test reporting control table, entity id : 50 , categoryid : 1488, financial year : 2025, Acc id : 5,Amount : Million,RoundTypeLevel: Child level,
# dbutils.widgets.text("ComparitivePeriodValue",'')
# dbutils.widgets.text("AmountsIn","FCU")
# # dbutils.widgets.text("AmountsIn","Million") 
# dbutils.widgets.text("Roundoff","0")
# # dbutils.widgets.text("RoundTypeLevel","GL code level")
# dbutils.widgets.text("RoundTypeLevel","Child level")
# dbutils.widgets.text("IsDrillDown","1")
# # dbutils.widgets.text("Flag", "1")

# # dbutils.widgets.text("Dim", '{"BusinessUnit": "BU1,BU2"}') # not an extended
# # dbutils.widgets.text("Dim", '{"BusinessUnit": "BU1,BU2", "Segmentupdate": "Segm1"}')
# # dbutils.widgets.text("Dim", '{"ServiceLinechange": "SL1", "Department": "Dept2"}') # standard period for 105 and 103
# # dbutils.widgets.text("Dim", '{"Geography": "India,Delhi"}') # less than 12 months
# # dbutils.widgets.text("Dim", '{"Geography": "India,Delhi", "ProfitCentre": "PF1"}') # more than 12 months

# EntityID = dbutils.widgets.get("EntityID")
# Bunit = dbutils.widgets.get("Bunit")
# CategoryID = dbutils.widgets.get("CategoryID")
# FinancialYear = dbutils.widgets.get("FinancialYear")
# AccountingStandardID = dbutils.widgets.get("AccountingStandardID")
# PeriodValue = dbutils.widgets.get("PeriodValue")
# ComparitivePeriodValue_param = dbutils.widgets.get("ComparitivePeriodValue")
# AmountsIn = dbutils.widgets.get("AmountsIn")
# Roundoff = dbutils.widgets.get("Roundoff")
# RoundTypeLevel = dbutils.widgets.get("RoundTypeLevel")
# IsDrillDown = dbutils.widgets.get("IsDrillDown")
# # Flag = dbutils.widgets.get("Flag")
# # Dim = dbutils.widgets.get("Dim")

In [0]:
%run ../../../BackgroundServices/src/FactIncrementalLoad/Configuration_dynamic

In [0]:
engagement = dbutils.widgets.text("engagement","TenantDB1Config1")
environment = dbutils.widgets.text("environment","Development")

In [0]:
configuration = Configuration(engagement)
# print(configuration)
adls_path = f"abfss://{configuration.container}@{configuration.storageaccountname}.dfs.core.windows.net"
folder_path = "/DataUpload/Multiproc/"
catalog_name = 'dtdf_assu_faas_uc'
catalog_schema_name = 'dtdf_assu_faas_uc.frh'

In [0]:
# # Need to remove
# def sql_read_table(table_name: str):    
#     """
#     Read a Delta table into a PySpark DataFrame.

#     :param path: The path to the Delta table.
#     :param format: The format of the table.
#     :return: A PySfgpark DataFrame containing the data from the Delta table.
#     """
#     username = "apsSqlDIGIQAAdmin"
#     password = "4&4skBzqt7Q32@k5afHmG2$!"
#     host = "apsqdigsql01.database.windows.net"
#     try:
#         df = spark.read.format("sqlserver") \
#     .option("host", "apsqdigsql01.database.windows.net") \
#     .option("port", "1433") \
#     .option("user", username) \
#     .option("password", password) \
#     .option("database", "Dev1ConfigDB") \
#     .option("dbtable", table_name) \
#     .load()
#         print(f"SQL table {table_name} data read is successful!")
#         return df
#     except Exception as e:
#         print(f"An error occurred: {str(e)}")
#         error_message = f"An error occurred while reading table {table_name}: {str(e)}"
#         logging.error(error_message)
#         raise Exception("[Error] Reading source table data Failed ...", e)
#         return None

#### 3. Read Data from source tables

In [0]:
trialbalance_tbl = "trialbalance"
castingadjustment_tbl = "castingadjustmenttable"
legacyadjustment_tbl = "legacyadjustment"
adjustments_tbl = "adjustments"
custom_dim_master_tbl = "custom_dim_master"
dimensiontable_tbl = "dimensiontable"
scoa_tbl = "scoa"
scoamappingtable_tbl = "scoamappingtable"
reportingparameters_tbl = "reportingparameters"
executioncontrol_tbl = "executioncontroltbl"
reportrunlog_tbl = "reportrunlog"
drilldown_tbl = get_tbl_nm('multiperiod_drilldown_'+str(parameterID))
staging_tbl = get_tbl_nm('multiperiod_staging_'+str(parameterID))

trialbalance = get_tbl_nm(trialbalance_tbl)
trialbalance_src_uc_df = spark.sql(f"SELECT * FROM {trialbalance}")
castingadjustment = get_tbl_nm(castingadjustment_tbl)
castingadjustment_src_uc_df = spark.sql(f"SELECT * FROM {castingadjustment}")
legacyadjustment = get_tbl_nm(legacyadjustment_tbl)
legacyadjustments_src_uc_df = spark.sql(f"SELECT * FROM {legacyadjustment}")
adjustments = get_tbl_nm(adjustments_tbl)
standaloneadjustments_src_uc_df = spark.sql(f"SELECT * FROM {adjustments}")
custom_dim_master = get_tbl_nm(custom_dim_master_tbl)
customdimmaster_src_uc_df = spark.sql(f"SELECT * FROM {custom_dim_master}")
dimensiontable = get_tbl_nm(dimensiontable_tbl)
dimensiontable_src_uc_df = spark.sql(f"SELECT * FROM {dimensiontable}")
scoa = get_tbl_nm(scoa_tbl)
scoa_src_uc_df = spark.sql(f"SELECT * FROM {scoa}")
scoamappingtable = get_tbl_nm(scoamappingtable_tbl)
masterscoamapping_src_uc_df = spark.sql(f"SELECT * FROM {scoamappingtable}")
reportingparameters = get_tbl_nm(reportingparameters_tbl)
reportingparameters_src_uc_df = spark.sql(f"SELECT * FROM {reportingparameters}")
executioncontrol = get_tbl_nm(executioncontrol_tbl)
executioncontrol_src_uc_df = spark.sql(f"SELECT * FROM {executioncontrol}")
reportrunlog = get_tbl_nm(reportrunlog_tbl)
reportrunlog_src_uc_df = spark.sql(f"SELECT * FROM {reportrunlog}")


In [0]:
# trialbalance_src_uc_df = spark.read.table("dtdf_assu_faas_uc.frh.trialbalance")
# castingadjustment_src_uc_df = spark.read.table("dtdf_assu_faas_uc.frh.castingadjustmenttable")
# legacyadjustments_src_uc_df = spark.read.table("dtdf_assu_faas_uc.frh.legacyadjustment")
# standaloneadjustments_src_uc_df = spark.read.table("dtdf_assu_faas_uc.frh.adjustments")
# customdimmaster_src_uc_df =  spark.read.table("dtdf_assu_faas_uc.frh.custom_dim_master")
# dimensiontable_src_uc_df = spark.read.table("dtdf_assu_faas_uc.frh.dimensiontable")
# scoa_src_uc_df = spark.read.table("dtdf_assu_faas_uc.frh.scoa")
# masterscoamapping_src_uc_df = spark.read.table("dtdf_assu_faas_uc.frh.scoamappingtable")
# reportingparameters_src_uc_df = spark.read.table("dtdf_assu_faas_uc.frh.reportingparameters")
# executioncontroltbl_src_uc_df = spark.read.table("dtdf_assu_faas_uc.frh.executioncontroltbl")
# reportrunlog_src_uc_df = spark.read.table("dtdf_assu_faas_uc.frh.reportrunlog")
periodicaldatesforextensionperiod_src_df = sql_read_table("master.PeriodicalDatesForExtensionPeriod")
entityperioddetails_src_df = sql_read_table("master.EntityPeriodDetails")
periodicaldates_src_df = sql_read_table("master.PeriodicalDates")
entitymaster_src_df = sql_read_table("master.entitymaster")
financialcycle_src_df = sql_read_table("master.financialcycle")
categorydropdown_src_df = sql_read_table("master.categorydropdown")

#### 4. Transformations

In [0]:
PeriodValuePrefix = PeriodValue.split("-")[0][:1]
if PeriodValuePrefix in ('Y','9','H','Q','M') :
    PeriodValueInYHQM=1
else :
    PeriodValueInYHQM=0
print("PeriodValuePrefix:",PeriodValuePrefix)
print("PeriodValueInYHQM:",PeriodValueInYHQM)

if ComparitivePeriodValue_param is None:
    print("ComparitivePeriodValue is None")
elif len(ComparitivePeriodValue_param) == 0:
    ComparitivePeriodValue = None
    print("ComparitivePeriodValue is Empty")
else:
    print("ComparitivePeriodValue is not empty")
    ComparitivePeriodValue = ComparitivePeriodValue_param
print(ComparitivePeriodValue)

entity_result = entitymaster_src_df.filter(entitymaster_src_df["entitymasterid"] == EntityID).select(
    "fk_fiscalperiodid",
    "fk_functionalcurrencyid",
    "entityname",
    "fk_fiscalperiodid"
).first()

# display(entity_result)

# Assigning values to variables
FK_FiscalPeriodID = entity_result["fk_fiscalperiodid"]
SDCurrencyid = entity_result["fk_functionalcurrencyid"]
EntitySDName = entity_result["entityname"]
FinancialSDCycleID = entity_result["fk_fiscalperiodid"]
print("FK_FiscalPeriodID:",FK_FiscalPeriodID)
print("SDCurrencyid:",SDCurrencyid)
print("EntitySDName:",EntitySDName)
print("FinancialSDCycleID:",FinancialSDCycleID)
# # Selecting values from financial_cycle_df
fiscal_cycle_result = financialcycle_src_df.filter(financialcycle_src_df["financialcycleid"] == FK_FiscalPeriodID).select(
    "financialcycle",
    "financialcycleid"
).first()

# # Assigning values to variables
FiscalPeriod = fiscal_cycle_result["financialcycle"]
FiscalPeriodID = fiscal_cycle_result["financialcycleid"]
print("FiscalPeriod:",FiscalPeriod)
print("FiscalPeriodID:",FiscalPeriodID)

CategorySDName = categorydropdown_src_df.filter(col("categoryid") == CategoryID).select("categoryname").first()
CategorySDName = CategorySDName["categoryname"]
print("CategorySDName:",CategorySDName)

extendedmonthdiff_result = entityperioddetails_src_df \
.filter(col("Fk_EntityMasterID") == EntityID) \
    .select(expr(
        "MONTH(endDate) - MONTH(startDate) + 1 + (YEAR(endDate) - YEAR(startDate)) * 12"
    ).alias("ExtendedMonthDiff")).collect()
    
    
if extendedmonthdiff_result:
    extendedmonthdiff = extendedmonthdiff_result[0]["ExtendedMonthDiff"]
    # Use extendedmonthdiff variable for further processing
else:
    extendedmonthdiff = 0
print("extendedmonthdiff:",extendedmonthdiff)

extflag = entityperioddetails_src_df.filter((col("fk_entitymasterid") == EntityID) & (col("isExtendedPeriod") == 1)).count() > 0
extflag = int(extflag)
print("extflag:",extflag)
extvalue = PeriodValue.split('-')[0]

print("extvalue:",extvalue)
extendedcycleyear_result = (
    periodicaldatesforextensionperiod_src_df
    .filter((col("Entityid") == EntityID) & (col("Value") == extvalue) & (col("FinancialCycleID") == FiscalPeriodID))
    .select(when(col("Year").isNotNull(), col("Year")).otherwise("").alias("ExtendedCycleYear"))
    .first()
)
# 

if extendedcycleyear_result:
    extendedcycleyear = extendedcycleyear_result['ExtendedCycleYear']
    # Use extendedmonthdiff variable for further processing
else:
    extendedcycleyear = extendedcycleyear_result

print("extendedcycleyear:",extendedcycleyear)

if extendedcycleyear is None:
    extendedcycleyear = 0

print("extendedcycleyear:",extendedcycleyear)

##### Get ParameterID, UploadChangedate and LastExecutionDate

In [0]:
'''
def get_parameterID(reportingparameters_src_uc_df):
    try:
        reportingparameters_df = reportingparameters_src_uc_df.where(
                                (col("EntityID_fk") == EntityID) &
                                (col("CategoryId_fk") == CategoryID) &
                                (col("Year") == FinancialYear) &
                                (col("AccountingStandard_fk") == AccountingStandardID) &
                                (col("Period") == PeriodValue) &
                                (col("ComparativePeriodValue") == ComparitivePeriodValue_param) &
                                (col("AmountsIn") == AmountsIn) &
                                (col("RoundUpto") == Roundoff)  &
                                (col("RoundUptoLevel") == RoundTypeLevel))
        parameterID = reportingparameters_df.select("ParameterID").first()[0]
        print("parameterID: ",parameterID)
        return parameterID
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_message = f"No ParameterID found for given {EntityID}"
        logging.error(error_message)
        raise Exception("[Error] No ParameterID found for given {EntityID} ...", e)
        return None
'''    
def get_uploadchangedate(executioncontrol_src_uc_df,parameterID):
    try:
        executioncontrol_df = executioncontrol_src_uc_df.filter((col("ParameterID_fk") == parameterID) & (col("Status") ==  "Pending"))
        if executioncontrol_df.count()==0:

            return  datetime.strptime("9999-12-25", "%Y-%m-%d") #datetime("9999-99-99 09:15:23.359000")
        
        else:
            print(" THIS SCENERIO WILL NEVER COME")
                       
    

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_message = f"No UploadChangeDate found for given {EntityID}"
        logging.error(error_message)
        raise Exception("[Error] No UploadChangeDate found for given {EntityID} ...", e)
        return None

def insertReportRunLog(reportrunlog_src_uc_df,reportrunlog,parameterID,CategoryID):
    #code changes
    if reportrunlog_src_uc_df.where(F.col("ParameterID_fk")==parameterID).count()==0:
        print("new parameterid FIRST TIME ",parameterID)
        spark.sql(f'''
                    INSERT into {reportrunlog} (ParameterID_fk, Categoryid_fk, ExecutionTime,ExecutionDate,ExecutedBy,IsReRun)
                        VALUES ({parameterID}, {CategoryID}, current_timestamp , current_timestamp,'System',true)
                                    ''')
    else:
        print("parameterid already present")
        

# def get_LastExecutionDate(reportrunlog_src_uc_df,parameterID):
#     try:
#         reportrunlog_df = reportrunlog_src_uc_df.filter(col("ParameterID_fk") == parameterID)
#         if reportrunlog_df.count()==0:
#             print("inside if of get_LastExecutionDate fxn ")
#             insertReportRunLog(reportrunlog_src_uc_df,reportrunlog,parameterID,CategoryID)
#             LastExecutionDate = reportrunlog_df.select("ExecutionTime").first()[0]
#             return LastExecutionDate 
#             #datetime.strptime("2000-01-01", "%Y-%m-%d") #datetime("2000-01-01 09:15:23.359000")
#         else:
#             print("inside ELSE of get_LastExecutionDate fxn ")
#             # reportrunlog_df.display()
#             # reportrunlog_df = reportrunlog_df.withColumn("ExecutionDate",lit("2024-04-15 09:15:23.359000")) # Need to delete later once data is available 
#             max_execution_date_row = reportrunlog_df.select("ExecutionDate").orderBy(col("ExecutionDate").desc()).first()
#             LastExecutionDate = max_execution_date_row['ExecutionDate']
#             print("LastExecutionDate: ",LastExecutionDate)
#             return LastExecutionDate
#     except Exception as e:
#         print(f"An error occurred: {str(e)}")
#         error_message = f"No LastExecutionDate found for given {EntityID}"
#         logging.error(error_message)
#         raise Exception("[Error] No LastExecutionDate found for given {EntityID} ...", e)
#         return None

def get_LastExecutionDate(reportrunlog_src_uc_df,executioncontrol_src_uc_df,parameterID):
    try:
        reportrunlog_df = reportrunlog_src_uc_df.filter(col("ParameterID_fk") == parameterID)
        if reportrunlog_df.count()==0:

            print("inside if of get_LastExecutionDate fxn ")
            executioncontrol_df = executioncontrol_src_uc_df.filter((col("ParameterID_fk") == parameterID) & (col("Status") ==  "Pending"))
            LastExecutionDate = executioncontrol_df.select("ExecutionDateTime").orderBy(col("ExecutionDateTime").desc()).collect()[0]["ExecutionDateTime"]
            print("LastExecutionDate: ",LastExecutionDate)
            # insertReportRunLog(reportrunlog_src_uc_df,reportrunlog,parameterID,CategoryID)                  
            return  LastExecutionDate #datetime("2000-01-01 09:15:23.359000")
        else:
            print("inside ELSE of get_LastExecutionDate fxn ")

            # reportrunlog_df.display()
            # reportrunlog_df = reportrunlog_df.withColumn("ExecutionDate",lit("2024-04-15 09:15:23.359000")) # Need to delete later once data is available 
            max_execution_date_row = reportrunlog_df.select("ExecutionDate").orderBy(col("ExecutionDate").desc()).first()
            LastExecutionDate = max_execution_date_row['ExecutionDate']
            print("LastExecutionDate: ",LastExecutionDate)
            return LastExecutionDate
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        error_message = f"No LastExecutionDate found for given {EntityID}"
        logging.error(error_message)
        raise Exception("[Error] No LastExecutionDate found for given {EntityID} ...", e)
        return None
    
#parameterID = get_parameterID(reportingparameters_src_uc_df)

#UploadChangeDate = get_uploadchangedate(executioncontrol_src_uc_df,parameterID) 
LastExecutionDate = get_LastExecutionDate(reportrunlog_src_uc_df,executioncontrol_src_uc_df,parameterID)
#print("UploadChangeDate",UploadChangeDate)
print("LastExecutionDate",LastExecutionDate)

In [0]:
## REMOVE IT
parameterID = 119
executioncontrol_tbl = "executioncontroltbl"
reportrunlog_tbl = "reportrunlog"
executioncontrol = get_tbl_nm(executioncontrol_tbl)
executioncontrol_src_uc_df = spark.sql(f"SELECT * FROM {executioncontrol}")
reportrunlog = get_tbl_nm(reportrunlog_tbl)
reportrunlog_src_uc_df = spark.sql(f"SELECT * FROM {reportrunlog}")
# reportrunlog_df = reportrunlog_src_uc_df.filter(col("ParameterID_fk") == parameterID)
# if reportrunlog_df.count()==0:
print("inside if of get_LastExecutionDate fxn ")
executioncontrol_df = executioncontrol_src_uc_df.filter((col("ParameterID_fk") == parameterID) & (col("Status") ==  "Pending"))
LastExecutionDate = executioncontrol_df.select("ExecutionDateTime").orderBy(col("ExecutionDateTime").desc()).collect()[0]["ExecutionDateTime"]
print("LastExecutionDate: ",LastExecutionDate)

In [0]:
alldimcols=[col_name for col_name in trialbalance_src_uc_df.columns if trialbalance_src_uc_df.select(col(col_name)).na.drop().count() > 0 if col_name.startswith("DimType")]

# udf to include dimesion columns dynamically
def add_dimension_columns(alldimcols, existing_schema):
    dim_cols = []
    processed_columns = set()

    for column in alldimcols:
        if column not in processed_columns:
            processed_columns.add(column)
            if column not in [field.name for field in existing_schema.fields]:
                dim_cols.append(StructField(column, IntegerType(), True))
    
    return StructType(existing_schema.fields + dim_cols)

schema_daterangeconverttoperiod= StructType([ \
    StructField("currentperiod",StringType(),True), \
    StructField("startdate",DateType(),True), \
    StructField("enddate", DateType(), True), \
    StructField("periodstart",StringType(),True), \
    StructField("monthdiff",IntegerType(),True), \
    StructField("periodmonthvalue",StringType(),True), \
    StructField("tbperiodvalue", StringType(), True)
  ])
 
df_daterangeconverttoperiod = spark.createDataFrame(data=[],schema=schema_daterangeconverttoperiod)

schema_daterangecustomperiod = StructType([ \
    StructField("id",IntegerType(),True),
    StructField("currentperiod",StringType(),True), \
    StructField("startdate",DateType(),True), \
    StructField("enddate", DateType(), True), \
    StructField("periodstart",StringType(),True), \
    StructField("monthdiff",IntegerType(),True), \
    StructField("periodmonthvalue",StringType(),True), \
    StructField("tbperiodvalue", StringType(), True),\
    StructField("year",StringType(),True), \
    StructField("periodvalue",StringType(),True), \
    StructField("customperiod",StringType(),True), \
    StructField("customdaterange", IntegerType(), True)
  ])
 
df_daterangecustomperiod = spark.createDataFrame(data=[],schema=schema_daterangecustomperiod)

schema_daterangetoperiod = StructType([ \
    StructField("id",IntegerType(),True),
    StructField("currentperiod",StringType(),True), \
    StructField("startdate",DateType(),True), \
    StructField("enddate", DateType(), True), \
    StructField("periodstart",StringType(),True), \
    StructField("monthdiff",IntegerType(),True), \
    StructField("periodmonthvalue",StringType(),True), \
    StructField("tbperiodvalue", StringType(), True),\
    StructField("year",StringType(),True), \
    StructField("periodvalue",StringType(),True), \
    StructField("customperiod",StringType(),True), \
    StructField("customdaterange", IntegerType(), True),
    StructField("openperiodvalue", StringType(), True), \
    StructField("opencyclevalue", StringType(), True) \
  ])

schema_daterangetoperiod1 = StructType([ \
    StructField("id",IntegerType(),True),
    StructField("cpid",StringType(),True), \
    StructField("currentperiod",StringType(),True), \
    StructField("startdate",DateType(),True), \
    StructField("enddate", DateType(), True), \
    StructField("periodstart",StringType(),True), \
    StructField("monthdiff",IntegerType(),True), \
    StructField("periodmonthvalue",StringType(),True), \
    StructField("tbperiodvalue", StringType(), True),\
    StructField("year",StringType(),True), \
    StructField("periodvalue",StringType(),True), \
    StructField("customperiod",StringType(),True), \
    StructField("customdaterange", IntegerType(), True),
    StructField("openperiodvalue", StringType(), True), \
    StructField("opencyclevalue", StringType(), True) \
  ])
 
df_daterangetoperiod = spark.createDataFrame(data = [],schema=schema_daterangetoperiod)

schema_df_usergivenmulticomp = StructType([
  StructField("compperiodname", StringType(), True),  
  StructField("compperiodvalue", StringType(), True),
  StructField("comptype", StringType(), True),
  StructField("compperiod", StringType(), True),
  StructField("Value", StringType(), True)])
df_usergivenmulticomp= spark.createDataFrame([],schema = schema_df_usergivenmulticomp)

schema_df_compperioddetails = StructType([
StructField("comptype", StringType(), True),
StructField("alias", StringType(), True)])
df_compperioddetails= spark.createDataFrame([],schema = schema_df_compperioddetails)


schema_df_legacysddates = StructType([
  #StructField('id', IntegerType(), True),
  StructField('periodtype', StringType(), True),  
  StructField('Value', StringType(), True),
  StructField('pcurrentvalue', StringType(), True),
  StructField('currentvalue', StringType(), True),
  StructField('legacyperiod', StringType(), True),
  StructField('categoryid', IntegerType(), True),
  StructField('financialyear', IntegerType(), True),
  StructField('openingperiodvalue', StringType(), True),
  StructField('castingperiod', StringType(), True),
  StructField('numericvalueofperiod', IntegerType(), True),
  StructField('ftpperiod', IntegerType(), True),
  StructField('periodformat', StringType(), True),
  StructField('p', StringType(), True),
  StructField('ppp', StringType(), True),
  StructField('py', StringType(), True),
  StructField('pypp', StringType(), True),
  StructField('ppy', StringType(), True),
  StructField('p3y', StringType(), True),
  StructField('p4y', StringType(), True)
  ])
df_legacysddates= spark.createDataFrame([],schema = schema_df_legacysddates)

allperiodftpscoa_schema = StructType([
    StructField("SCOAID", IntegerType(), nullable=True),
    StructField("Member", StringType(), nullable=True),
    StructField("Path", StringType(), nullable=True),
    StructField("Column27", StringType(), nullable=True),
    StructField("AccountSubType", StringType(), nullable=True),
    StructField("UltimateParent", StringType(), nullable=True)
])

SCOAMappingTable_schema = StructType([
    StructField("EntityId", IntegerType(), False),
    StructField("Period", StringType(), False),
    StructField("Accountingtype", IntegerType(), False),
    StructField("EYGLCode", StringType(), False),
    StructField("UserGLCode", StringType(), False),
    StructField("CategoryID", IntegerType(), True),
    StructField("UltimateParent", StringType(), False),
    StructField("Unit", StringType(), True),
    StructField("CreatedBy", StringType(), True),
    StructField("ModifiedBy", StringType(), True),
    StructField("CreatedDate", StringType(), True),
    StructField("ModifiedDate", StringType(), True),
    StructField("PresentationLayer", StringType(), True),
    StructField("UserGLDescription", StringType(), True)
])

legacydatesperiodtypelite_schema = StructType([
    StructField("pcurrentvalue", StringType(), True),
    StructField("PeriodType", StringType(), True)
])

legacydatesnotperiodtypelite_schema = StructType([
    StructField("pcurrentvalue", StringType(), True),
    StructField("PeriodType", StringType(), True)
])


priorityseq_schema = StructType([
    StructField("Period", StringType(), True),
    StructField("stdperiod", StringType(), True),
    StructField("PeriodType", StringType(), True),
    StructField("closing", StringType(), True),
    StructField("legacyperiod", StringType(), True),
    StructField("numericvalueofperiod", IntegerType(), True),
    StructField("priorityseq", IntegerType(), True),
    StructField("order", IntegerType(), True),
    StructField("ftpperiod", IntegerType(), True)
])


prioritylevel2_existing_schema = StructType([
    StructField("Period", StringType(), True),
    StructField("stdperiod", StringType(), True),
    StructField("PeriodType", StringType(), True),
    StructField("closing", StringType(), True),
    StructField("legacyperiod", StringType(), True),
    StructField("numericvalueofperiod", IntegerType(), True),
    StructField("priorityseq", IntegerType(), True),
    StructField("Order", IntegerType(), True)
])
prioritylevel2_schema = add_dimension_columns(alldimcols, prioritylevel2_existing_schema)

prioritylevel2_ext_schema = StructType([
    StructField("Period", StringType(), True),
    StructField("stdperiod", StringType(), True),
    StructField("PeriodType", StringType(), True),
    StructField("closing", StringType(), True),
    StructField("legacyperiod", StringType(), True),
    StructField("numericvalueofperiod", IntegerType(), True),
    StructField("priorityseq", IntegerType(), True),
    StructField("Order", IntegerType(), True)
])


tballperiod_schema = StructType([
    StructField("Period", StringType(), True),
    StructField("stdperiod", StringType(), True),
    StructField("PeriodType", StringType(), True),
    StructField("closing", StringType(), True),
    StructField("LegacyPeriod", StringType(), True),
    StructField("numericvalueofperiod", IntegerType(), True)
])


tblegacymatchperiod_existing_schema = StructType([
    StructField("Period", StringType(), True),
    StructField("stdperiod", StringType(), True),
    StructField("PeriodType", StringType(), True),
    StructField("closing", StringType(), True),
    StructField("LegacyPeriod", StringType(), True),
    StructField("numericvalueofperiod", IntegerType(), True)
])
tblegacymatchperiod_schema = add_dimension_columns(alldimcols, tblegacymatchperiod_existing_schema)

ytddates_schema = StructType([
    StructField("PeriodType", StringType(), True),
    StructField("CurrentValue", StringType(), True),
    StructField("FTPValue", StringType(), True)
])

tbfinal_existing_schema = StructType([
    StructField("PeriodType", StringType(), True),
    StructField("Period", StringType(), True),
    StructField("userglcode", StringType(), True),
    StructField("currency", IntegerType(), True),
    StructField("closingbalance", DecimalType(28, 8), True)
])


tbfinal_schema = add_dimension_columns(alldimcols, tbfinal_existing_schema)

tbfinalytd_existing_schema = StructType([
    StructField("PeriodType", StringType(), True),
    StructField("Period", StringType(), True),
    StructField("userglcode", StringType(), True),
    StructField("currency", IntegerType(), True),
    StructField("closingbalance", DecimalType(28, 8), True)
])

tbfinalytd_schema = add_dimension_columns(alldimcols, tbfinalytd_existing_schema)

tbfinaldd_existing_schema = StructType([
    StructField("PeriodType", StringType(), True),
    StructField("Period", StringType(), True),
    StructField("userglcode", StringType(), True),
    StructField("usergldescription", StringType(), True),
    StructField("currency", IntegerType(), True),
    StructField("closingbalance", DecimalType(28, 8), True),
    StructField("Source", StringType(), True)
])
tbfinaldd_schema = add_dimension_columns(alldimcols, tbfinaldd_existing_schema)

tbfinalddytd_existing_schema = StructType([
    StructField("PeriodType", StringType(), True),
    StructField("Period", StringType(), True),
    StructField("userglcode", StringType(), True),
    StructField("usergldescription", StringType(), True),
    StructField("currency", IntegerType(), True),
    StructField("closingbalance", DecimalType(28, 8), True),
    StructField("Source", StringType(), True)
])
tbfinalddytd_schema = add_dimension_columns(alldimcols, tbfinalddytd_existing_schema)

disclosures_existing_schema = StructType([
    StructField("PeriodType", StringType(), True),
    StructField("Period", StringType(), True),
    StructField("entityid", IntegerType(), True),
    StructField("ultimateparent", StringType(), True),
    StructField("categoryid", IntegerType(), True),
    StructField("member", StringType(), True),
    StructField("accounttype", StringType(), True),
    StructField("amount", DecimalType(28, 8), True),
    StructField("currencyid", IntegerType(), True),
    StructField("column27", StringType(), True),
    StructField("AccountSubType", StringType(), True)
])
disclosures_schema = add_dimension_columns(alldimcols, disclosures_existing_schema)

disclosuresytd_existing_schema = StructType([
    StructField("PeriodType", StringType(), True),
    StructField("Period", StringType(), True),
    StructField("entityid", IntegerType(), True),
    StructField("ultimateparent", StringType(), True),
    StructField("categoryid", IntegerType(), True),
    StructField("member", StringType(), True),
    StructField("accounttype", StringType(), True),
    StructField("amount", DecimalType(28, 8), True),
    StructField("currencyid", IntegerType(), True),
    StructField("column27", StringType(), True),
    StructField("AccountSubType", StringType(), True)
])
disclosuresytd_schema = add_dimension_columns(alldimcols, disclosuresytd_existing_schema)

castingsdadjustment_existing_schema = StructType([
    StructField("CastingAdjustmentsID", IntegerType(), True),
    StructField("Fk_AccountingStandardID", IntegerType(), True),
    StructField("Fk_ValidationID", IntegerType(), True),
    StructField("PeriodValue", StringType(), True),
    StructField("FinancialYear", StringType(), True),
    StructField("Fk_EntityID", IntegerType(), True),
    StructField("EntityName", StringType(), True),
    StructField("JournalNumber", IntegerType(), True),
    StructField("JournalDate", TimestampType(), True),
    StructField("UserGLCode", StringType(), True),
    StructField("UserGLDescription", StringType(), True),
    StructField("StandardGLCode", StringType(), True),
    StructField("StandardGLDescription", StringType(), True),
    StructField("DebitCredit", StringType(), True),
    StructField("Fk_CurrencyID", IntegerType(), True),
    StructField("Amount", DecimalType(18, 4), True),
    StructField("JournalType", StringType(), True),
    StructField("Narration", StringType(), True),
    StructField("Fk_BusinessUnitID", IntegerType(), True),
    StructField("CategoryID", IntegerType(), True),
    StructField("Roundoff", IntegerType(), True),
    StructField("AmountsIn", StringType(), True),
    StructField("ReportType", StringType(), True),
    StructField("CreatedDate", TimestampType(), True),
    StructField("CreatedBy", StringType(), True),
    StructField("ModifiedDate", TimestampType(), True),
    StructField("ModifiedBy", StringType(), True),
    StructField("IsUserGLCode", BooleanType(), True),
    StructField("RoundTypeLevel", StringType(), True),
    
])
castingsdadjustment_schema = add_dimension_columns(alldimcols, castingsdadjustment_existing_schema)
castingsdadjustment_df = spark.createDataFrame([], schema=castingsdadjustment_schema)

legacysddates_schema = StructType([
    StructField("id", IntegerType(), False),
    StructField("periodtype", StringType(), True),
    StructField("Value", StringType(), True),
    StructField("pcurrentvalue", StringType(), True),
    StructField("currentvalue", StringType(), True),
    StructField("legacyperiod", StringType(), True),
    StructField("categoryid", IntegerType(), True),
    StructField("financialyear", IntegerType(), True),
    StructField("openingperiodvalue", StringType(), True),
    StructField("castingperiod", StringType(), True),
    StructField("numericvalueofperiod", IntegerType(), True),
    StructField("ftpperiod", IntegerType(), True),
    StructField("periodformat", StringType(), True),
    StructField("p", StringType(), True),
    StructField("ppp", StringType(), True),
    StructField("py", StringType(), True),
    StructField("pypp", StringType(), True),
    StructField("ppy", StringType(), True),
    StructField("p3y", StringType(), True),
    StructField("p4y", StringType(), True)
])

legacysddates_union_df = spark.createDataFrame([], legacysddates_schema)


legacyadjustmentsdrilldown_schema = StructType([
    StructField("AccountingStdID_fk", IntegerType(), True),
    StructField("Period", StringType(), True),
    StructField("EntityID_Fk", IntegerType(), True),
    StructField("JournalNumber", IntegerType(), True),
    StructField("JournalDate", TimestampType(), True),
    StructField("UserGLCode", StringType(), True),
    StructField("Member", StringType(), True),
    StructField("DebitCredit", StringType(), True),
    StructField("CurrencyID_Fk", IntegerType(), True),
    StructField("Amount", DecimalType(20, 5), True),
    StructField("JournalType", StringType(), True),
    StructField("Narration", StringType(), True),
    StructField("CreatedDate", TimestampType(), True),
    StructField("CreatedBy", StringType(), True),
    StructField("ModifiedDate", TimestampType(), True),
    StructField("ModifiedBy", StringType(), True),
    StructField("BUID_fk", IntegerType(), True),
    StructField("IsImported", BooleanType(), True),
    StructField("CategoryID", IntegerType(), True),
    StructField("IsMappedGLCode", BooleanType(), True),
    StructField("EndPeriod", StringType(), True),
    StructField("NotApplicable", StringType(), True),
    StructField("DimType1ID_fk", IntegerType(), True),
    StructField("DimType2ID_fk", IntegerType(), True),
    StructField("DimType3ID_fk", IntegerType(), True),
    StructField("DimType4ID_fk", IntegerType(), True),
    StructField("DimType5ID_fk", IntegerType(), True),
    StructField("DimType6ID_fk", IntegerType(), True),
    StructField("DimType7ID_fk", IntegerType(), True),
    StructField("DimType8ID_fk", IntegerType(), True),
    StructField("DimType9ID_fk", IntegerType(), True),
    StructField("DimType10ID_fk", IntegerType(), True),
    StructField("DimType11ID_fk", IntegerType(), True),
    StructField("DimType12ID_fk", IntegerType(), True),
    StructField("DimType13ID_fk", IntegerType(), True),
    StructField("DimType14ID_fk", IntegerType(), True),
    StructField("DimType15ID_fk", IntegerType(), True),
    StructField("DimType16ID_fk", IntegerType(), True),
    StructField("DimType17ID_fk", IntegerType(), True),
    StructField("DimType18ID_fk", IntegerType(), True),
    StructField("DimType19ID_fk", IntegerType(), True),
    StructField("DimType20ID_fk", IntegerType(), True),
    StructField("DimType21ID_fk", IntegerType(), True),
    StructField("DimType22ID_fk", IntegerType(), True),
    StructField("DimType23ID_fk", IntegerType(), True),
    StructField("DimType24ID_fk", IntegerType(), True),
    StructField("DimType25ID_fk", IntegerType(), True)
])

# Create an empty DataFrame with the defined schema
legacyadjustmentsdrilldown_union_df = spark.createDataFrame([], legacyadjustmentsdrilldown_schema)

mappedsdtbytd_existing_schema = StructType([
    StructField("PeriodType", StringType(), True),
    StructField("Period", StringType(), True),
    StructField("UserGLCode", StringType(), True),
    StructField("Currency", IntegerType(), True),
    StructField("ClosingBalance", DecimalType(28, 8), True),
    StructField("CategoryID", StringType(), True),
    StructField("Member", StringType(), True),
    StructField("ytdtype", IntegerType(), True)
])
mappedsdtbytd_schema = add_dimension_columns(alldimcols, mappedsdtbytd_existing_schema)

mappedsdtbftp_existing_schema = StructType([
    StructField("PeriodType", StringType(), True),
    StructField("Period", StringType(), True),
    StructField("UserGLCode", StringType(), True),
    StructField("Currency", IntegerType(), True),
    StructField("ClosingBalance", DecimalType(28, 8), True),
    StructField("categoryid", StringType(), True),
    StructField("member", StringType(), True),
    StructField("ytdtype", IntegerType(), True)
])
mappedsdtbftp_schema = add_dimension_columns(alldimcols, mappedsdtbftp_existing_schema)

mappedtbddytd_existing_schema = StructType([
    StructField("PeriodType", StringType(), True),
    StructField("Period", StringType(), True),
    StructField("UserGLCode", StringType(), True),
    StructField("UserGLDescription", StringType(), True),
    StructField("closingbalance", DecimalType(28, 8), True),
    StructField("member", StringType(), True),
    StructField("Source", StringType(), True),
    StructField("ytdtype", IntegerType(), True)
])
mappedtbddytd_schema = add_dimension_columns(alldimcols, mappedtbddytd_existing_schema)

mappedtbddftp_existing_schema = StructType([
    StructField("PeriodType", StringType(), True),
    StructField("Period", StringType(), True),
    StructField("UserGLCode", StringType(), True),
    StructField("UserGLDescription", StringType(), True),
    StructField("closingbalance", DecimalType(28, 8), True),
    StructField("member", StringType(), True),
    StructField("Source", StringType(), True),
    StructField("ytdtype", IntegerType(), True)
])
mappedtbddftp_schema = add_dimension_columns(alldimcols, mappedtbddftp_existing_schema)

allperioddata_existing_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("PeriodType", StringType(), True),
    StructField("Period", StringType(), True),
    StructField("userglcode", StringType(), True),
    StructField("currency", IntegerType(), True),
    StructField("closingbalance", DecimalType(28, 8), True),
    StructField("categoryid", IntegerType(), True),
    StructField("member", StringType(), True),
    StructField("ytdtype", StringType(), True),
    StructField("usergldescription", StringType(), True),
    StructField("Source", StringType(), True),
    StructField("Value", StringType(), True),
    StructField("pcurrentvalue", StringType(), True),
    StructField("legacyperiod", StringType(), True),
    StructField("financialyear", StringType(), True),
    StructField("openingperiodvalue", StringType(), True),
    StructField("castingperiod", StringType(), True),
    StructField("p", StringType(), True),
    StructField("ppp", StringType(), True),
    StructField("py", StringType(), True),
    StructField("pypp", StringType(), True),
    StructField("ppy", StringType(), True),
    StructField("p3y", StringType(), True),
    StructField("p4y", StringType(), True),
    StructField("CastingAdjustmentsID", IntegerType(), True),
    StructField("Fk_AccountingStandardID", IntegerType(), True),
    StructField("Fk_ValidationID", IntegerType(), True),
    StructField("PeriodValue", StringType(), True),
    StructField("Fk_EntityID", IntegerType(), True),
    StructField("EntityName", StringType(), True),
    StructField("JournalNumber", IntegerType(), True),
    StructField("JournalDate", TimestampType(), True),
    StructField("StandardGLCode", StringType(), True),
    StructField("StandardGLDescription", StringType(), True),
    StructField("DebitCredit", StringType(), True),
    StructField("Fk_CurrencyID", IntegerType(), True),
    StructField("Amount", DecimalType(18, 4), True),
    StructField("JournalType", StringType(), True),
    StructField("Narration", StringType(), True),
    StructField("Fk_BusinessUnitID", IntegerType(), True),
    StructField("Roundoff", IntegerType(), True),
    StructField("AmountsIn", StringType(), True),
    StructField("ReportType", StringType(), True),
    StructField("CreatedDate", TimestampType(), True),
    StructField("CreatedBy", StringType(), True),
    StructField("ModifiedDate", TimestampType(), True),
    StructField("ModifiedBy", StringType(), True),
    StructField("IsUserGLCode", IntegerType(), True),
    StructField("RoundTypeLevel", StringType(), True),
    StructField("typeofdata", IntegerType(), True)
])
allperioddata_schema = add_dimension_columns(alldimcols, allperioddata_existing_schema)
allperioddata_df = spark.createDataFrame([], schema=allperioddata_schema)


In [0]:
def get_legacydates(periodicaldates_src_df,df_legacysddates,df_daterangetoperiod,schema_daterangetoperiod,df_usergivenmulticomp,df_daterangeconverttoperiod,df_daterangecustomperiod,df_compperioddetails,AccountingStandardID,PeriodValue):
    try:
        
        AccountingType = AccountingStandardID
        Start_TimestampMP = datetime.now()
        NumericalPeriodTypeValue =None
        customperiod = 0
        startVal = None
        print("custom period logic")
        list_col_daterangeconverttoperiod = ["id","currentperiod","startdate","enddate","periodstart","monthdiff","periodmonthvalue","tbperiodvalue"]

        list_col_daterangecustomperiod = ["id","currentperiod","startdate","enddate","periodstart","monthdiff","periodmonthvalue","tbperiodvalue","year","periodvalue","customperiod","customdaterange"]

        list_col_daterangetoperiod = ["id","currentperiod","startdate","enddate","periodstart","monthdiff","periodmonthvalue","tbperiodvalue","year","periodvalue","customperiod","customdaterange","openperiodvalue","opencyclevalue"]

        # Create Empty df_usergivenmulticomp and df_compperioddetails
        list_col_usergivenmulticomp = ["compperiodname","compperiodvalue","comptype","compperiod","Value"]

        list_col_legacysddates=["id","periodtype","Value","pcurrentvalue","currentvalue","legacyperiod","categoryid","financialyear","openingperiodvalue","castingperiod","numericvalueofperiod","ftpperiod","periodformat","p","ppp","py","pypp","ppy","p3y","p4y"]
        
        periodicaldates_df = periodicaldates_src_df.withColumnRenamed("Year","pd_Year")

        print("change date range period to standard period format")

        #function to get date in YYYY-MM-DD format 

        def changeToDateFormat(inputDate):
            date_list= inputDate.split("-")
            date_input = date_list[0]
            month_input = date_list[1]
            year_input = date_list[2]
            dateValue = datetime.date(int(year_input), int(month_input), int(date_input))
            return dateValue

        def diff_month(d1, d2):
            return (d1.year - d2.year) * 12 + d1.month - d2.month

        #for periodic table
        def getPeriodFirstValue(Period):
            v1= Period.split(" ")[0]
            #v2= Period.split(" ")[1]
            return v1
        getPeriodFirstValue_udf = F.udf(getPeriodFirstValue,StringType())

        #get period column of df_daterangecustomperiod
        def getperiodvalue(value,opening,closing):
            periodvalue= value + "-" + opening + "-" + closing
            return periodvalue

        getperiodvalue_udf = F.udf(getperiodvalue,StringType())

        #get period column of df_daterangecustomperiod
        def getcustomperiod(periodMonthValue,monthDiff,opening,startDate,closing):
            customperiod=None
            if periodMonthValue =='M12' and monthDiff == 12:
                customperiod= 'Year'
            elif periodMonthValue == 'M12' and monthDiff == 6:
                customperiod= 'H2'
            elif periodMonthValue =='M12' and monthDiff == 3:
                customperiod= 'Q4'
            elif periodMonthValue =='M3' and monthDiff == 3:
                customperiod= 'Q1'
            elif periodMonthValue =='M6' and monthDiff == 6:
                customperiod= 'H1'
            elif periodMonthValue =='M6' and monthDiff == 3:
                customperiod= 'Q2'
            elif periodMonthValue =='M9' and monthDiff == 9:
                customperiod= '9M'
            elif periodMonthValue =='M9' and monthDiff == 3:
                customperiod= 'Q3'
            else:
                customperiod = 'C-' + opening[:4] + startDate.strftime("%m") + '-' + closing
            return customperiod

        getcustomperiod_udf = F.udf(getcustomperiod,StringType())

        #get updated periodvalue column of df_daterangecustomperiod on the basis of customperiod
        def getupdatedperiodvalue(periodvalue,customperiod):
            if customperiod[:1]=='C':
                periodvalue=periodvalue
            else:
                tempList = periodvalue.split("-")
                periodvalue = customperiod+'-'+tempList[1]+'-'+tempList[2]
            return periodvalue

        getupdatedperiodvalue_udf = F.udf(getupdatedperiodvalue,StringType())
        #get updated customdaterange column of df_daterangecustomperiod on the basis of customperiod
        def getupdatedcustomdaterange(customperiod):
            if customperiod[:1]=='C':
                customdaterange=1
            else:
                customdaterange=0
            return customdaterange

        getupdatedcustomdaterange_udf = F.udf(getupdatedcustomdaterange,IntegerType())

        #get openperiodvalue of daterangetoperiod
        def getopenperiodvalue(startDate):
            openperiodvalue = startDate.strftime("%Y")+startDate.strftime("%m")
            return openperiodvalue

        getopenperiodvalue_udf = F.udf(getopenperiodvalue,StringType())

        def getperiodMonthValue(endDate,periodstart):
            periodMonthValue = 'M'+str((int(endDate.strftime("%m")) - int(periodstart) + 12 ) % 12 + 1)
            return periodMonthValue

        getperiodMonthValue_udf = F.udf(getperiodMonthValue,StringType())

        def gettbperiodvalue(endDate):
            return endDate.strftime("%Y")+endDate.strftime("%m")


        gettbperiodvalue_udf = F.udf(gettbperiodvalue,StringType())

        #this if condition, all required df for data in range format
        if PeriodValueInYHQM==0:
            print("inside if")
            startVal=4  # from master.financialcycle
            #INSERT INTO #daterangeconverttoperiod
            startDate= changeToDateFormat(PeriodValue[:10])
            endDate= changeToDateFormat(PeriodValue[11:])
            periodStart = str(startVal)  
            monthDiff=0
            periodMonthValue=None
            tbPeriodValue=None
            data_row_single = [[PeriodValue,startDate,endDate,periodStart,monthDiff,periodMonthValue,tbPeriodValue]]
            '''--------------------- df_daterangeconverttoperiod ---------------'''
            df_daterangeconverttoperiodTemp1= spark.createDataFrame(data=data_row_single,schema=schema_daterangeconverttoperiod)
           
            df_daterangeconverttoperiod = df_daterangeconverttoperiod.union(df_daterangeconverttoperiodTemp1)
            #display(df_daterangeconverttoperiod)
            if ComparitivePeriodValue is not None:
                print("inside if ComparitivePeriodValue is not None")
                list_ComparitivePeriodValue = ComparitivePeriodValue.split(",")  #comparativeperiod
                rows=[]
                for compPer in list_ComparitivePeriodValue:
                    startDate= changeToDateFormat(compPer[:10])
                    endDate= changeToDateFormat(compPer[11:])
                    print(startDate,endDate)
                    rows.append([compPer,startDate,endDate,periodStart,monthDiff,periodMonthValue,tbPeriodValue])
                df_daterangeconverttoperiodTemp2= spark.createDataFrame(data=rows,schema=schema_daterangeconverttoperiod)
                
                df_daterangeconverttoperiod = df_daterangeconverttoperiod.union(df_daterangeconverttoperiodTemp2)
                    
            else:
                print("ComparitivePeriodValue is  None")
        
            df_daterangeconverttoperiod = df_daterangeconverttoperiod.withColumn("periodmonthvalue",getperiodMonthValue_udf(F.col("enddate"),F.col("periodstart")))\
                                        .withColumn("monthdiff", F.months_between(F.col("enddate"),F.col("startdate")).cast(IntegerType()))\
                                        .withColumn("monthdiff", F.col("monthdiff")+1)\
                                        .withColumn("tbperiodvalue",gettbperiodvalue_udf(F.col("enddate")))
            
            # adding id column in df_daterangeconverttoperiod
            df_daterangeconverttoperiod = df_daterangeconverttoperiod.withColumn("monotonically_increasing_id",F.monotonically_increasing_id()+1).withColumn("DummyColForPartition",F.lit("DummyColForPartition"))
            #display(df1218)
            window = Window.partitionBy(F.col("DummyColForPartition")).orderBy(col('monotonically_increasing_id'))
            df_daterangeconverttoperiod = df_daterangeconverttoperiod.withColumn('id', F.row_number().over(window)).select(list_col_daterangeconverttoperiod)
            
            #df_daterangeconverttoperiod calculation end 
            
            
            '''--------------------- End of df_daterangeconverttoperiod ---------------'''
            '''--------------------- daterangecustomperiod ---------------'''
            #get daterangecustomperiod temp table 
            #creating new temp col in perioddates for joining condition, will drop tihs col after join
            periodicaldates_df = periodicaldates_df.withColumn("PeriodColFirstValue", getPeriodFirstValue_udf(F.col("Period")))

            df_daterangecustomperiod_new = df_daterangecustomperiod.select(list_col_daterangeconverttoperiod).union(df_daterangeconverttoperiod)\
                                            .withColumn("year",F.lit(None))\
                                            .withColumn("periodvalue",F.lit(None))\
                                            .withColumn("customperiod",F.lit(None))\
                                            .withColumn("customdaterange",F.lit(None))
            #df_daterangecustomperiod= spark.createDataFrame(df_daterangecustomperiod.rdd,schema = schema_daterangecustomperiod)
            df_daterangecustomperiod = df_daterangecustomperiod.union(df_daterangecustomperiod_new)
            
            joined_df_pt_pd = (df_daterangecustomperiod.join(periodicaldates_df,how="inner",on= [(periodicaldates_df.Closing==df_daterangecustomperiod.tbperiodvalue) & (df_daterangecustomperiod.periodmonthvalue == periodicaldates_df.PeriodColFirstValue) & (periodicaldates_df.FinancialCycleID==FiscalPeriodID)]))\
                    .withColumn("year",F.col("pd_Year"))\
                    .withColumn("periodvalue",getperiodvalue_udf(F.col("Value"),F.col("Opening"),F.col("Closing")))\
                    .withColumn("customperiod",getcustomperiod_udf(F.col("periodmonthvalue"),F.col("monthdiff"),F.col("opening"),F.col("startdate"),F.col("closing")))\
                    .withColumn("customdaterange",F.lit(0))
            df_daterangecustomperiod = joined_df_pt_pd.select(list_col_daterangecustomperiod)
            
            periodicaldates_df = periodicaldates_df.drop("PeriodColFirstValue") #droppping dervied col 
            # get updated customperiod and customdaterange on the basis of customperiod
            df_daterangecustomperiod =df_daterangecustomperiod.withColumn("periodvalue",getupdatedperiodvalue_udf(F.col("periodvalue"),F.col("customperiod"))).withColumn("customdaterange",getupdatedcustomdaterange_udf(F.col("customperiod")))
            
            '''--------------------- daterangetoperiod ---------------'''
            #daterangetoperiod
            df_daterangetoperiod_new = df_daterangetoperiod.select(list_col_daterangecustomperiod).union(df_daterangecustomperiod).withColumn("openperiodvalue",F.lit(None)).withColumn("opencyclevalue",F.lit(None))
            
            df_daterangetoperiod = df_daterangetoperiod.union(df_daterangetoperiod_new).withColumn("cpid",F.concat(F.lit("CP"),F.col("id")-1))

            joined_df_ptb_pd = df_daterangetoperiod.join(periodicaldates_df,how="inner", on=[(df_daterangetoperiod.periodmonthvalue==periodicaldates_df.Value)& (df_daterangetoperiod.year==periodicaldates_df.pd_Year) ]).where( (F.col("financialcycleid")==F.lit(FiscalPeriodID)) )\
                                .withColumn("openperiodvalue",getopenperiodvalue_udf(F.col("startdate")))\
                                .withColumn("opencyclevalue",F.col("Opening")).dropDuplicates(list_col_daterangecustomperiod)\
                                
            print("------------df_daterangetoperiod after joined_df_ptb_pd---")
            df_daterangetoperiod =joined_df_ptb_pd.select(list_col_daterangetoperiod[:1]+["cpid"]+list_col_daterangetoperiod[1:]).orderBy("id")
            
            PeriodValue = df_daterangetoperiod.select("periodvalue").where(F.col("id")==1).collect()[0][0]
            print("update value of PeriodValue to:",PeriodValue)
            

            #update monthdiff column When inYHQM format
        df_daterangetoperiod = df_daterangetoperiod.withColumn( "monthdiff", when(col("periodmonthvalue").substr(1, 1) == 'Y', 12)\
                .when(col("periodmonthvalue").substr(1, 1) == '9', 9)
                .when(col("periodmonthvalue").substr(1, 1) == 'H', 6)
                .when(col("periodmonthvalue").substr(1, 1) == 'Q', 3)
                .when(col("periodmonthvalue").substr(1, 1) == 'M', 1).otherwise(F.col("monthdiff"))).orderBy("id")        

        # adding id column in df_daterangetoperiod in PeriodValueInYHQM==1 case
        if PeriodValueInYHQM==1:  
            df_daterangetoperiod = df_daterangetoperiod.withColumn("monotonically_increasing_id",F.monotonically_increasing_id()+1).withColumn("DummyColForPartition",F.lit("DummyColForPartition"))
            window = Window.partitionBy(F.col("DummyColForPartition")).orderBy(col('monotonically_increasing_id'))
            df_daterangetoperiod = df_daterangetoperiod.withColumn('id', F.row_number().over(window)).withColumn("cpid",F.concat(F.lit("CP"),F.col("id")-1)).select(list_col_daterangetoperiod[:1]+["cpid"]+list_col_daterangetoperiod[1:])
        
        df_daterangetoperiod.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"daterangetoperiod")  
        df_daterangetoperiod = spark.read.parquet(adls_path+folder_path+"daterangetoperiod",schema = schema_daterangetoperiod1)

        # local variable calculation
        PeriodValuePrefix = PeriodValue[:1]
        PeriodType = PeriodValuePrefix
        endperiod = PeriodValue[-6:]
        if PeriodValueInYHQM ==0:
            print("inside if")
            NumericalValueofPeriodType = -int((df_daterangetoperiod.select("monthdiff").where(F.col("id")==1).collect()[0][0]))  
        else:
            if PeriodType== 'Y':
                NumericalValueofPeriodType= -12
            elif PeriodType== '9':
                NumericalValueofPeriodType= -9
            elif PeriodType== 'H':
                NumericalValueofPeriodType= -6
            elif PeriodType== 'Q':
                NumericalValueofPeriodType= -3
            elif PeriodType== 'M':
                NumericalValueofPeriodType= -1

        print("addition for custom date range")

        # from datetime import datetime
        def getYearMonthFromDate(input_date):
            return input_date.strftime("%Y")+input_date.strftime("%m")
            
        YearEndinDateFormat  = datetime.strptime(FinancialYear + '-' +  FiscalPeriod[-3:] + '-' + '01', '%Y-%b-%d').date()
        PeriodEndInDateFormat = datetime.strptime(endperiod[:4] + '-' + endperiod[-2:] + '-' + '01' , '%Y-%m-%d').date()
        print("YearEndinDateFormat:",YearEndinDateFormat)
        print("PeriodEndInDateFormat:",PeriodEndInDateFormat)

        CurrentPeriod	=	None
        CurrentPeriod_YTD	=	None
        PreviousPeriod	=	None
        PrevPreviousPeriod	=	None
        PreviousYear	=	None
        PrevPreviousYear	=	None
        PrevYearCurrPeriod	=	None
        PrevYearCurrPeriod_YTD	=	None
        PrevYearPrevPeriod	=	None
        PPrevPreviousYear	=	None
        P4PreviousYear	=	None

        CurrentPeriod = endperiod
        CurrentPeriod_YTD = endperiod
        if PeriodValuePrefix !=9:
            PreviousPeriod = getYearMonthFromDate(PeriodEndInDateFormat + relativedelta(months = int(NumericalValueofPeriodType)))
        else:
            PreviousPeriod = getYearMonthFromDate(PeriodEndInDateFormat  + relativedelta(months = -12))
        PrevPreviousPeriod = getYearMonthFromDate(PeriodEndInDateFormat + relativedelta(months = 2 * int(NumericalValueofPeriodType) ))
        PreviousYear	=	 getYearMonthFromDate(YearEndinDateFormat  + relativedelta(months = -12))
        PrevPreviousYear	=getYearMonthFromDate(YearEndinDateFormat  + relativedelta(months = -24))
        PrevYearCurrPeriod	=getYearMonthFromDate(PeriodEndInDateFormat + relativedelta(months = -12 ))
        PrevYearCurrPeriod_YTD	=getYearMonthFromDate(PeriodEndInDateFormat + relativedelta(months = -12 ))
        PrevYearPrevPeriod	=	 getYearMonthFromDate(PeriodEndInDateFormat + relativedelta(months = -12 + int(NumericalValueofPeriodType) ))
        PPrevPreviousYear	=	getYearMonthFromDate(YearEndinDateFormat  + relativedelta(months = -36))
        P4PreviousYear	=	getYearMonthFromDate(YearEndinDateFormat  + relativedelta(months = -48))

        print("comparative Period calc")

        CompYearEndinDateFormat = None     #DATE,
        CompPeriodEndInDateFormat  = None     # DATE,
        CompFinancialyear  = None            # string
        gapincompperiod   = None             # INT,
        countofcomp    = None                # INT;

        # df_complist :Empty
        schema_complist= StructType([ StructField("compperiod",StringType(),True) ])
        df_complist = spark.createDataFrame(data=[],schema=schema_complist)

        if PeriodValueInYHQM == 1:
            print('period format is not daterange')
            if ComparitivePeriodValue is not None:
                list_ComparitivePeriodValue  = ComparitivePeriodValue.split(",")
                data_in_row_list=[]
                for cp in list_ComparitivePeriodValue:
                    data_in_row_list.append([cp])
                print("data_in_row_list",data_in_row_list)
                df_comp_temp = spark.createDataFrame(data=data_in_row_list,schema=schema_complist)
                df_complist = df_complist.union(df_comp_temp)
        else:
            print('period format is daterange')
            df_complist = df_complist.union(df_daterangetoperiod.withColumn("complist_compperiod",F.when(F.col("customdaterange")==1,F.concat_ws('-',F.col("periodmonthvalue"),F.col("openperiodvalue"),F.col("tbperiodvalue") ) )\
                            .otherwise(F.concat_ws('-',F.col("customperiod"),F.col("openperiodvalue"),F.col("tbperiodvalue"))) ).where(F.col("id")!=1).select("complist_compperiod"))

        
        countofcomp = df_complist.count()
        print("countofcomp",countofcomp)

        #required functions
        def getcompperiodvalue(PeriodValueInYHQM,compperiodvalue,customdaterange,periodmonthvalue,customperiod,openperiodvalue,tbperiodvalue):
            if PeriodValueInYHQM == 1:
                compperiodvalue=compperiodvalue
            else:
                if customdaterange==1:
                    compperiodvalue=periodmonthvalue + '-' + openperiodvalue + '-' + tbperiodvalue
                else:
                    compperiodvalue=customperiod + '-' + openperiodvalue + '-' + tbperiodvalue
            return compperiodvalue

        getcompperiodvalue_udf  = F.udf(getcompperiodvalue,StringType())


        def getcomptype(PeriodValueInYHQM,compperiodvalue,tbperiodvalue):
            if PeriodValueInYHQM == 1:
                compperiod = compperiodvalue[-6:]
            else:
                compperiod= tbperiodvalue
            return compperiod

        getcomptype_udf  = F.udf(getcomptype,StringType())
        def getValue(compperiodvalue):
            return compperiodvalue.split("-")[0]

        getValue_udf  = F.udf(getValue,StringType())

        if ComparitivePeriodValue is not None:
            if countofcomp == 1:
                df_compperioddetails = df_compperioddetails.union(spark.createDataFrame([('CP1', 'PreviousPeriod'),('CP1', 'PrevYearCurrPeriod_YTD'),('CP1', 'PrevYearCurrPeriod'),('CP1', 'PreviousYear')],schema = schema_df_compperioddetails))

            elif countofcomp == 2:
                df_compperioddetails = df_compperioddetails.union(spark.createDataFrame([('CP1', 'PreviousPeriod'),('CP1', 'PreviousYear'),('CP2', 'PrevPreviousYear')],schema = schema_df_compperioddetails))

            elif countofcomp ==3:
                df_compperioddetails = df_compperioddetails.union(spark.createDataFrame([('CP1', 'PreviousPeriod'),('CP1', 'PreviousYear'),('CP2', 'PrevPreviousYear'), ('CP3', 'PPrevPreviousYear')],schema = schema_df_compperioddetails))

            elif countofcomp ==4:
                df_compperioddetails = df_compperioddetails.union(spark.createDataFrame([('CP1', 'PreviousPeriod'),('CP1', 'PreviousYear'),('CP2', 'PrevPreviousYear'),('CP3', 'PPrevPreviousYear'),('CP4','P4PreviousYear')],schema = schema_df_compperioddetails))
            elif countofcomp ==5:
                df_compperioddetails = df_compperioddetails.union(spark.createDataFrame([('CP1', 'PreviousPeriod'),('CP1', 'PreviousYear'),('CP2', 'PrevPreviousYear'),('CP3', 'PPrevPreviousYear'),('CP4','P4PreviousYear'), ('CP5','P5PreviousYear')],schema = schema_df_compperioddetails))
            elif countofcomp ==6:
                df_compperioddetails = df_compperioddetails.union(spark.createDataFrame([('CP1', 'PreviousPeriod'),('CP1', 'PreviousYear'),('CP2', 'PrevPreviousYear'),('CP3', 'PPrevPreviousYear'),('CP4','P4PreviousYear'), ('CP5','P5PreviousYear'),('CP6','P6PreviousYear')],schema = schema_df_compperioddetails))
            elif countofcomp ==7:
                df_compperioddetails = df_compperioddetails.union(spark.createDataFrame([('CP1', 'PreviousPeriod'),('CP1', 'PreviousYear'),('CP2', 'PrevPreviousYear'),('CP3', 'PPrevPreviousYear'),('CP4','P4PreviousYear'), ('CP5','P5PreviousYear'),('CP6','P6PreviousYear'),('CP7','P7PreviousYear')],schema = schema_df_compperioddetails))
            elif countofcomp ==8:
                df_compperioddetails = df_compperioddetails.union(spark.createDataFrame([('CP1', 'PreviousPeriod'),('CP1', 'PreviousYear'),('CP2', 'PrevPreviousYear'),('CP3', 'PPrevPreviousYear'),('CP4','P4PreviousYear'), ('CP5','P5PreviousYear'),('CP6','P6PreviousYear'),('CP7','P7PreviousYear'),('CP8','P8PreviousYear')],schema = schema_df_compperioddetails))
            print("df_compperioddetails")
            
            comparative_periods_df = spark.createDataFrame([(ComparitivePeriodValue,)], ["value"]).withColumn("comparative_periods", split(col("value"), ',')).selectExpr("explode(comparative_periods) as compperiodvalue")
            #ctecomp
            df_ctecomp = comparative_periods_df.withColumn("comptype", F.concat(F.lit('CP'), F.row_number().over(Window.orderBy(monotonically_increasing_id()))))
            print("---ctecomp---")
            
            df_joined_cte_pt_dp = df_ctecomp.join(df_compperioddetails,how="inner",on="comptype").join(df_daterangetoperiod,how="left",on=df_ctecomp.comptype==df_daterangetoperiod.cpid)\
            .withColumn("compperiodvalue",getcompperiodvalue_udf(F.lit(PeriodValueInYHQM),F.col("compperiodvalue"),F.col("customdaterange"),F.col("periodmonthvalue"),F.col("customperiod"),F.col("openperiodvalue"),F.col("tbperiodvalue")) )\
            .withColumn("compperiod",getcomptype_udf(F.lit(PeriodValueInYHQM),F.col("compperiodvalue"),F.col("tbperiodvalue")))\
            .withColumn("Value",getValue_udf(F.col("compperiodvalue"))).select(["alias","compperiodvalue",'comptype', 'compperiod', 'Value'])
            print("df_joined_cte_pt_dp")
            
            df_usergivenmulticomp = df_usergivenmulticomp.union(df_joined_cte_pt_dp)
            df_usergivenmulticomp = df_usergivenmulticomp.withColumn("monotonically_increasing_id",F.monotonically_increasing_id())
            window = Window.orderBy(col('monotonically_increasing_id'))
            df_usergivenmulticomp = df_usergivenmulticomp.withColumn('id', row_number().over(window)).select(["id"]+list_col_usergivenmulticomp)
            print("df_usergivenmulticomp final dataframe")
            

            compperiodvalue = df_usergivenmulticomp.where(df_usergivenmulticomp.id==1).select("compperiodvalue").collect()[0][0]
            
            if FiscalPeriod=='Jan-Dec':
                CompFinancialyear = str(compperiodvalue[-6:-2])
            else:
                CompFinancialyear = str(int(compperiodvalue.split("-")[1][:4])+1)

            ComparitiveSelectionEndPeriod= compperiodvalue[-6:]
            CompYearEndinDateFormat = datetime.strptime(CompFinancialyear + '-' +  FiscalPeriod[-3:] + '-' + '01', '%Y-%b-%d').date()
            CompPeriodEndInDateFormat =  datetime.strptime(ComparitiveSelectionEndPeriod[:4] + '-' +  ComparitiveSelectionEndPeriod[4:]+ '-' + '01', '%Y-%m-%d').date()
            gapincompperiod = (int(YearEndinDateFormat.strftime("%Y")) - int(CompYearEndinDateFormat.strftime("%Y")) )-1
            print("CompYearEndinDateFormat",CompYearEndinDateFormat)
            print("CompPeriodEndInDateFormat",CompPeriodEndInDateFormat)
            print('CompFinancialyear',CompFinancialyear)
            print("ComparitiveSelectionEndPeriod",ComparitiveSelectionEndPeriod)
            print("gapincompperiod",gapincompperiod)
            #---------to remove normal period data when  one comparative is selected
            if countofcomp==1 and PeriodValueInYHQM==1:
                print("---------to remove normal period data when  one comparative is selected----")
                print("TRUNCATE TABLE df_usergivenmulticomp")
                df_usergivenmulticomp = df_usergivenmulticomp.limit(0)

                PreviousPeriod = ComparitiveSelectionEndPeriod  #CP1
                PrevPreviousPeriod = getYearMonthFromDate(CompYearEndinDateFormat + relativedelta(months = int(NumericalValueofPeriodType) ))
                PreviousYear	=	getYearMonthFromDate(CompYearEndinDateFormat  + relativedelta(months = 0))
                PrevPreviousYear	=	getYearMonthFromDate(CompYearEndinDateFormat  + relativedelta(months = -12))       
                PrevYearCurrPeriod = ComparitiveSelectionEndPeriod
                PrevYearCurrPeriod_YTD = ComparitiveSelectionEndPeriod
                PrevYearPrevPeriod = getYearMonthFromDate(CompYearEndinDateFormat  + relativedelta(months = -24))
                PPrevPreviousYear	=	getYearMonthFromDate(CompYearEndinDateFormat  + relativedelta(months = -24))
                P4PreviousYear	=	getYearMonthFromDate(CompYearEndinDateFormat  + relativedelta(months = -36))
        # legacydates
        schema_data_tmp_dates = StructType([StructField("periodtype",StringType(),True), StructField("currentvalue",StringType(),True), StructField("legacyperiod",StringType(),True) ])
        df_tmpdates = spark.createDataFrame(data = [],schema=schema_data_tmp_dates)
        schema_daterangetoperiod = StructType([ StructField("periodtype",StringType(),True), StructField("currentvalue",StringType(),True), StructField("legacyperiod",StringType(),True) ])
        df_dates = spark.createDataFrame(data = [],schema=schema_daterangetoperiod)

        def getLeagacyPeriod(periodtype,currentvalue,PeriodValuePrefix):
            legacyperiod=""
            if periodtype[-4:]=='Year' :
                legacyperiod= str(currentvalue)+'-12'
            else:
                #legacyperiod="elsepart"
                #NumericalvalueofPeriod=0
                if PeriodValuePrefix =='Y':
                    legacyperiod=currentvalue + '-12'
                elif PeriodValuePrefix =='9':
                    legacyperiod=currentvalue + '-09'
                elif PeriodValuePrefix =='H':
                    legacyperiod=currentvalue + '-06'
                elif PeriodValuePrefix =='Q':
                    legacyperiod=currentvalue + '-03'
                elif PeriodValuePrefix =='M':
                    legacyperiod=currentvalue + '-01'
            return legacyperiod
        getLeagacyPeriod_udf = F.udf(getLeagacyPeriod, StringType()) 

        #update legacyperiod on the basis of compperiod value
        def getLeagacyPeriod2(legacyperiod,compperiod,currentvalue):
            if compperiod is not None:
                PeriodValuePrefix =  compperiod.split("-")[0][:1]
                if PeriodValuePrefix =='Y':
                    legacyperiod=currentvalue + '-12'
                elif PeriodValuePrefix =='9':
                    legacyperiod=currentvalue + '-09'
                elif PeriodValuePrefix =='H':
                    legacyperiod=currentvalue + '-06'
                elif PeriodValuePrefix =='Q':
                    legacyperiod=currentvalue + '-03'
                elif PeriodValuePrefix =='M':
                    legacyperiod=currentvalue + '-01'
                return legacyperiod
            else:
                return legacyperiod
        getLeagacyPeriod2_udf = F.udf(getLeagacyPeriod2, StringType()) 

        #update legacyperiod value where it is Null
        def getLeagacyPeriod3(legacyperiod,NumericalPeriodTypeValue,currentvalue):
            if legacyperiod is None:
                if len(NumericalPeriodTypeValue)==1:
                    legacyperiod = currentvalue + '-' + '0' + NumericalPeriodTypeValue
                else:
                    legacyperiod = currentvalue + '-' + NumericalPeriodTypeValue
                
            return legacyperiod
        getLeagacyPeriod3_udf = F.udf(getLeagacyPeriod3, StringType()) 

        #update legacyperiod value where PeriodValuePrefix =1 i.e. Y9HQM format
        def getLeagacyPeriod4(currentvalue,PeriodValuePrefix):
            legacyperiod=""
            if PeriodValuePrefix =='Y':
                legacyperiod=currentvalue + '-12'
            elif PeriodValuePrefix =='9':
                legacyperiod=currentvalue + '-09'
            elif PeriodValuePrefix =='H':
                legacyperiod=currentvalue + '-06'
            elif PeriodValuePrefix =='Q':
                legacyperiod=currentvalue + '-03'
            elif PeriodValuePrefix =='M':
                legacyperiod=currentvalue + '-01'
            return legacyperiod
        getLeagacyPeriod4_udf = F.udf(getLeagacyPeriod4, StringType())

        #update legacyperiod on the basis of compperiod value-Y9HQM format
        def getLeagacyPeriod5(compperiod,Value):
            legacyperiod=""
            Value=Value[:1]
            if Value =='Y':
                legacyperiod=compperiod + '-12'
            elif Value =='9':
                legacyperiod=compperiod + '-09'
            elif Value =='H':
                legacyperiod=compperiod + '-06'
            elif Value =='Q':
                legacyperiod=compperiod + '-03'
            elif Value =='M':
                legacyperiod=compperiod + '-01'
            return legacyperiod
        getLeagacyPeriod5_udf = F.udf(getLeagacyPeriod5, StringType())

        
        def getExtractLastValue(compperiod):
            return compperiod.split("-")[2]
        getExtractLastValue_udf = F.udf(getExtractLastValue,StringType())
        df_complist = df_complist.withColumn("Right6_compperiod",getExtractLastValue_udf(F.col("compperiod")))
        

        #df_dates 
        if ( (countofcomp ==0 or countofcomp ==1) and PeriodValueInYHQM ==1):  
            print("inside if")
            data_tmp_dates2 = [("CurrentPeriod",CurrentPeriod,"")\
            ,("CurrentPeriod_YTD",CurrentPeriod_YTD,"")\
            ,("PreviousPeriod",PreviousPeriod,"")\
            ,("PreviousYear",PreviousYear,"")\
            ,("PrevYearCurrPeriod",PrevYearCurrPeriod,"")\
            ,("PrevYearCurrPeriod_YTD",PrevYearCurrPeriod_YTD,"")]
            df_tmpdates2 = spark.createDataFrame(data = data_tmp_dates2,schema=schema_data_tmp_dates)
            df_tmpdates = df_tmpdates.union(df_tmpdates2)
            
            list_compperiodname_ColData = [x["compperiodname"] for x in df_usergivenmulticomp.select("compperiodname").collect()]
            print(list_compperiodname_ColData)
            df_tmpdates = df_tmpdates.where(~df_tmpdates.periodtype.isin(list_compperiodname_ColData))  
            
            df_dates2 = df_tmpdates.union(df_usergivenmulticomp.select("compperiodname","compperiod").withColumn("EmptyCol",F.lit("")))
            df_dates = df_dates.union(df_dates2)            

            #update legacy col
            df_dates = df_dates.withColumn("legacyperiod",getLeagacyPeriod_udf(F.col("periodtype"),F.col("currentvalue"),F.lit(PeriodValuePrefix)))            

            #update legacy col
            df_dates =  df_dates.join(df_complist,how="left",on= df_dates.currentvalue == df_complist.Right6_compperiod).withColumn("legacyperiod",getLeagacyPeriod2_udf(F.col("legacyperiod"),F.col("compperiod"),F.col("currentvalue"))).select("periodtype","currentvalue","legacyperiod")
        
            if NumericalValueofPeriodType >= 0:
                NumericalPeriodTypeValue = NumericalValueofPeriodType
            else:
                NumericalPeriodTypeValue = -NumericalValueofPeriodType
            print("outside NumericalValueofPeriodType")
            df_dates = df_dates.withColumn("legacyperiod",getLeagacyPeriod3_udf(F.col("legacyperiod"),F.lit(NumericalPeriodTypeValue),F.col("currentvalue")))
            
        else:
            print("inside else part")
            data_tmp_dates2 = [("CurrentPeriod",CurrentPeriod,"")]
            df_tmpdates2 = spark.createDataFrame(data = data_tmp_dates2,schema=schema_data_tmp_dates)
            df_tmpdates = df_tmpdates.union(df_tmpdates2)
            
            df_temp1 = df_tmpdates.withColumn("legacyperiod",getLeagacyPeriod4_udf(F.col("currentvalue"),F.lit(PeriodValuePrefix)))
            df_temp2 = df_usergivenmulticomp.withColumn("legacyperiod",getLeagacyPeriod5_udf(F.col("compperiod"),F.col("Value"))).select("compperiodname","compperiod","legacyperiod")
            
            df_dates = df_temp1.union(df_temp2)
       
        print("1")
        # df_daterangetoperiodtype
        df_daterangetoperiodtype = df_daterangetoperiod.join(df_compperioddetails,how="left",on= df_daterangetoperiod.cpid==df_compperioddetails.comptype)
        df_daterangetoperiodtype = df_daterangetoperiodtype.withColumn("alias",F.when(F.col("alias").isNull(),F.lit("CurrentPeriod")).otherwise(F.col("alias")))\
                                                            .withColumnRenamed("alias","daterangeperiodtype").withColumnRenamed("id","daterangetoperiodtype_id").drop("comptype")
        print("------df_daterangetoperiodtype.display()-------")
        df_daterangetoperiodtype.display()
        print("2")
        #Get df_legacysddates from df_dates
        df_legacysddates_new = df_legacysddates.select("periodtype", "currentvalue", "legacyperiod","pcurrentvalue", "categoryid", "castingperiod").\
                                            union(df_dates.withColumn("pcurrentvalue",F.col("currentvalue")).withColumn("categoryid",F.lit(0)).withColumn("castingperiod",F.lit("12")))   \
            .withColumn("categoryid",F.lit(0)).withColumn("castingperiod",F.lit("12")).withColumn("id",F.lit(None)).withColumn("Value",F.lit(None)).withColumn("financialyear",F.lit(None)).withColumn("openingperiodvalue",F.lit(None)).withColumn("numericvalueofperiod",F.lit(None)).withColumn("ftpperiod",F.lit(None)).withColumn("periodformat",F.lit(None)).withColumn("p",F.lit(None)).withColumn("ppp",F.lit(None)).withColumn("py",F.lit(None)).withColumn("pypp",F.lit(None)).withColumn("ppy",F.lit(None)).withColumn("p3y",F.lit(None)).withColumn("p4y",F.lit(None)).select(list_col_legacysddates[1:])
        print(" --- df_legacysddates_new   222 ---")     
        df_legacysddates_new.display()
        # applying schema on df_legacysddates
        print(" df_legacysddates schema :")
        df_legacysddates.printSchema()
        print(" df_legacysddates_new schema     NEW:")
        df_legacysddates_new.printSchema()
        print()
        df_legacysddates = df_legacysddates.union(df_legacysddates_new) #issue
        print("list_col_legacysddates",list_col_legacysddates)
        # adding id column in df_legacysddates
        df_legacysddates = df_legacysddates.withColumn("monotonically_increasing_id",F.monotonically_increasing_id()+1)
        window = Window.orderBy(col('monotonically_increasing_id'))
        df_legacysddates = df_legacysddates.withColumn('id', F.row_number().over(window)).select(list_col_legacysddates)   
        print(" --- df_legacysddates ---")     
        df_legacysddates.display()
        print("3")
        def getperiodformat(customdaterange,legacyperiod):
            periodformat=""
            if customdaterange !=1 or customdaterange is None:
                if legacyperiod[7:]=='12':
                    periodformat='Yearly'
                elif legacyperiod[7:]=='06':
                    periodformat='Halfyearly'
                elif legacyperiod[7:]=='03':
                    periodformat='Quarterly'
                elif legacyperiod[7:]=='01':
                    periodformat='Monthly'
                elif legacyperiod[7:]=='09':
                    periodformat='9 Months'
                else:
                    periodformat='Monthly'
            elif customdaterange==1:
                periodformat='Monthly'
            return periodformat

        getperiodformat_udf = F.udf(getperiodformat, StringType())

        #getnumericvalueofperiod
        def getnumericvalueofperiod(Value,periodtype,openperiodvalue,monthdiff):
            numericvalueofperiod=None
            if PeriodValueInYHQM == 1:
                if Value is None:
                    return numericvalueofperiod
                else:
                    if Value[:1] == "Y" :
                        numericvalueofperiod= -12
                    elif Value[:1] == "9" :
                        numericvalueofperiod= -9
                    elif Value[:1] == "H" :
                        numericvalueofperiod= -6
                    elif Value[:1] == "Q" :
                        numericvalueofperiod= -3
                    elif Value[:1] == "M" :
                        numericvalueofperiod= -1
            elif PeriodValueInYHQM == 0:
                print("inside elif PeriodValueInYHQM==0")
                if ( (periodtype.find('Year') != -1) and (openperiodvalue is None)):
                    print("inside if")
                    numericvalueofperiod = -12
                    #return numericvalueofperiod
                else:
                    if monthdiff is None:
                        print("monthdiff is None")
                        numericvalueofperiod= None
                        return numericvalueofperiod 
                    print("inside else")
                    monthdiff = - monthdiff
                    numericvalueofperiod = monthdiff
                    #return numericvalueofperiod
            return numericvalueofperiod

        getnumericvalueofperiod_udf = F.udf(getnumericvalueofperiod, IntegerType()) 

        def getcastingperiod(Value,openingperiodvalue,currentvalue):
            castingperiod=None
            if Value is None or openingperiodvalue is None or currentvalue is None:
                return None
            else:
                castingperiod = Value + '-' + openingperiodvalue + '-' + currentvalue

            return castingperiod
        getgetcastingperiod_udf = F.udf(getcastingperiod, StringType())    

        def getFTPPeriod(Value):
            FTPPeriod=None
            if PeriodValueInYHQM == 1:
                if Value is None:
                    FTPPeriod = 1
                    return FTPPeriod
                else:
                    if Value[:1] == "Y" :
                        FTPPeriod= 5
                    elif Value[:1] == "9" :
                        FTPPeriod = 4
                    elif Value[:1] == "H" :
                        FTPPeriod = 3
                    elif Value[:1] == "Q" :
                        FTPPeriod = 2
                    elif Value[:1] == "M" :
                        FTPPeriod = 1
                    
            elif PeriodValueInYHQM == 0:
                FTPPeriod = 1
            return int(FTPPeriod)

        getFTPPeriod_udf = F.udf(getFTPPeriod, IntegerType())

        # from datetime import datetime, timedelta
        def getpy(currentvalue):
            if currentvalue is None:
                    return None
            else:
                    dayValue="01"
                    monthsValue = currentvalue[4:]
                    yearValue = currentvalue[:4]
                    date_currentvalue = datetime(int(yearValue), int(monthsValue), int(dayValue))
                    new_datevalue = date_currentvalue - relativedelta(months = 12) # year-1
                    req_yr = new_datevalue.strftime("%Y")
                    req_month= new_datevalue.strftime("%m")
                    return req_yr+req_month

        getpy_udf = F.udf(getpy, StringType())

        def getppy(currentvalue):
            if currentvalue is None:
                return None
            else:
                dayValue="01"
                monthsValue = currentvalue[4:]
                yearValue = currentvalue[:4]
                date_currentvalue = datetime(int(yearValue), int(monthsValue), int(dayValue))
                new_datevalue = date_currentvalue - relativedelta(months = 24) # year-2
                req_yr = new_datevalue.strftime("%Y")
                req_month= new_datevalue.strftime("%m")
                return req_yr+req_month

        getppy_udf = F.udf(getppy, StringType())

        def getp3y(currentvalue):
            if currentvalue is None:
                return None
            else:
                dayValue="01"
                monthsValue = currentvalue[4:]
                yearValue = currentvalue[:4]
                date_currentvalue = datetime(int(yearValue), int(monthsValue), int(dayValue))
                new_datevalue = date_currentvalue - relativedelta(months = 36) # year-3
                req_yr = new_datevalue.strftime("%Y")
                req_month= new_datevalue.strftime("%m")
                return req_yr+req_month

        getp3y_udf = F.udf(getp3y, StringType()) 


        def getp4y(currentvalue):
            if currentvalue is None:
                return None
            else:
                dayValue="01"
                monthsValue = currentvalue[4:]
                yearValue = currentvalue[:4]
                date_currentvalue = datetime(int(yearValue), int(monthsValue), int(dayValue))
                new_datevalue = date_currentvalue - relativedelta(months = 48) # year-4
                req_yr = new_datevalue.strftime("%Y")
                req_month= new_datevalue.strftime("%m")
                return req_yr+req_month

        getp4y_udf = F.udf(getp4y, StringType())


        def getp(currentvalue,numericvalueofperiod):
            if currentvalue is None or numericvalueofperiod is None:
                return None
            else:
                dayValue="01"
                monthsValue = currentvalue[4:]
                yearValue = currentvalue[:4]
                date_currentvalue = datetime(int(yearValue), int(monthsValue), int(dayValue))
                new_datevalue = date_currentvalue + relativedelta(months = int(numericvalueofperiod))
                req_yr = new_datevalue.strftime("%Y")
                req_month= new_datevalue.strftime("%m")
                return req_yr+req_month

        getp_udf = F.udf(getp, StringType())

        def getppp(currentvalue,numericvalueofperiod):
            if currentvalue is None or numericvalueofperiod is None:
                return None
            else:
                dayValue="01"
                monthsValue = currentvalue[4:]
                yearValue = currentvalue[:4]
                date_currentvalue = datetime(int(yearValue), int(monthsValue), int(dayValue))
                new_datevalue = date_currentvalue + relativedelta(months = 2 * int(numericvalueofperiod))
                req_yr = new_datevalue.strftime("%Y")
                req_month= new_datevalue.strftime("%m")
                return req_yr+req_month

        getppp_udf = F.udf(getppp, StringType()) 

        def getpypp(currentvalue,numericvalueofperiod):
            if currentvalue is None or numericvalueofperiod is None:
                return None
            else:
                dayValue="01"
                monthsValue = currentvalue[4:]
                yearValue = currentvalue[:4]
                date_currentvalue = datetime(int(yearValue), int(monthsValue), int(dayValue))
                new_datevalue = date_currentvalue + relativedelta(months = -12 + int(numericvalueofperiod))
                req_yr = new_datevalue.strftime("%Y")
                req_month= new_datevalue.strftime("%m")
                return req_yr+req_month

        getpypp_udf = F.udf(getpypp, StringType())

        #update  periodformat
        df_joined_ld_dp= df_legacysddates.alias("ld").join(df_daterangetoperiodtype.alias("dp"),how="left",
        on= [( F.col("ld.currentvalue") == F.col("dp.tbperiodvalue")) & (F.col("ld.periodtype") == F.col("dp.daterangeperiodtype"))])
        df_legacysddates = df_joined_ld_dp.withColumn("periodformat", getperiodformat_udf(F.col("customdaterange"),F.col("legacyperiod"))).select(list_col_legacysddates)

        # update Value , financialyear and openingperiodvalue by joining periodicaldates_df , df_legacysddates and df_daterangetoperiodtype 
        #Renaming column Name to avoid ambiguity
        periodicaldates_df = periodicaldates_df.withColumnRenamed("Value","periodicaldates_Value").withColumnRenamed("Opening","periodicaldates_Opening").withColumnRenamed("Year","periodicaldates_Year")

        df_joined_pd_ld_drtpt = periodicaldates_df.alias("pds").join(df_legacysddates.alias("lds"), col("pds.Closing") == col("lds.currentvalue"), "inner") \
            .join(df_daterangetoperiodtype.alias("dpt"), (col("lds.currentvalue") == col("dpt.tbperiodvalue")) & (col("lds.periodtype") == col("dpt.daterangeperiodtype")),how= "left_outer") .filter((periodicaldates_df["financialcycleid"] == FiscalPeriodID) & (periodicaldates_df["Type"] == df_legacysddates["periodformat"]))
        

        #Renaming column Name to avoid ambiguity
        for columnsName in df_joined_pd_ld_drtpt.columns:
            df_joined_pd_ld_drtpt= df_joined_pd_ld_drtpt.withColumnRenamed(columnsName,"joined_"+columnsName)
        

        legacysddates_df2_temp = df_legacysddates.alias("ld").join(df_joined_pd_ld_drtpt.alias("pdlddrtpt"),how="left",on= (col("ld.periodtype")==col("pdlddrtpt.joined_periodtype") ) )\
                            .withColumn("Value", col("joined_periodicaldates_Value")) \
                            .withColumn("financialyear", col("joined_pd_Year")) \
                            .withColumn("openingperiodvalue", when(col("joined_openperiodvalue").isNotNull(), col("joined_openperiodvalue")).otherwise(col("joined_periodicaldates_Opening")))
        
        df_legacysddates = legacysddates_df2_temp.select(list_col_legacysddates)
        

        #update numericvalueofperiod 
        legacysddates_df_temp_ld_pt = df_legacysddates.alias("ld2").join(df_daterangetoperiodtype.alias("dpt2"), on=[ (col("ld2.currentvalue") == col("dpt2.tbperiodvalue")) & (col("ld2.periodtype") == col("dpt2.daterangeperiodtype"))], how="left")\
                                    .withColumn("numericvalueofperiod", getnumericvalueofperiod_udf(F.col("Value"),F.col("periodtype"),F.col("openperiodvalue"),F.col("monthdiff")))
        df_legacysddates= legacysddates_df_temp_ld_pt.select(list_col_legacysddates)

        # update numericvalueofperiod column , where numericvalueofperiod is null replace numericvalueofperiod by -monthDiff value
        update_expr = -col("monthdiff")
        legacysddates_df4 = df_legacysddates.join(df_daterangetoperiod.filter(col("id") == 1), "id", "left_outer").withColumn("numericvalueofperiod", when(col("numericvalueofperiod").isNull(), update_expr).otherwise(col("numericvalueofperiod"))) 
        df_legacysddates = legacysddates_df4.select(list_col_legacysddates)

        # Update castingperiod, FTPPeriod , p , py, ppy, p3y, p4y, ppp , pypp
        df_legacysddates = df_legacysddates.withColumn("castingperiod",getgetcastingperiod_udf(F.col("Value"),F.col("openingperiodvalue"),F.col("currentvalue")))
        df_legacysddates = df_legacysddates.withColumn("ftpperiod",getFTPPeriod_udf(F.col("Value")))
        legacysddates_src_df = df_legacysddates.withColumn("p",getp_udf(F.col("currentvalue"),F.col("numericvalueofperiod")))\
                                            .withColumn("py",getpy_udf(F.col("currentvalue")))\
                                            .withColumn("ppy",getppy_udf(F.col("currentvalue")))\
                                            .withColumn("p3y",getp3y_udf(F.col("currentvalue")))\
                                            .withColumn("p4y",getp4y_udf(F.col("currentvalue")))\
                                            .withColumn("ppp",getppp_udf(F.col("currentvalue"),F.col("numericvalueofperiod")))\
                                            .withColumn("pypp",getpypp_udf(F.col("currentvalue"),F.col("numericvalueofperiod")))
        legacysddates_src_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates")
        return legacysddates_src_df,countofcomp
    except Exception as e:
        return "error"

In [0]:
legacysddates_src_df,countofcomp = get_legacydates(periodicaldates_src_df,df_legacysddates,df_daterangetoperiod,schema_daterangetoperiod,df_usergivenmulticomp,df_daterangeconverttoperiod,df_daterangecustomperiod,df_compperioddetails,AccountingStandardID,PeriodValue)

legacysddates_src_df = spark.read.parquet(adls_path+folder_path+"legacysddates",schema = legacysddates_schema)
period_value_ld = legacysddates_src_df.where(col("periodtype")=='CurrentPeriod').select("legacyperiod").first()[0][:-3]
print("period_value_ld",period_value_ld)

In [0]:
print("inside get_legacydates done")

##### Read data from trialbalance, castingadjustments, adjustments and legacyadjustments

In [0]:
print("Read data from trialbalance, castingadjustments, adjustments and legacyadjustments")

In [0]:
reportrunlog_df = reportrunlog_src_uc_df.filter(col("ParameterID_fk") == parameterID)
if reportrunlog_df.count()==0:
    print("if no entry found in reportlog means first run")
    # trialbalance_src_uc_df.display()

    trialbalance_src_uc_df = trialbalance_src_uc_df.where(col("Period").substr(1, 6)==period_value_ld)
    trialbalance_src_df = trialbalance_src_uc_df.filter((trialbalance_src_uc_df['EntityID_fk'] == EntityID) & (trialbalance_src_uc_df['AccountingStdID_fk'] == AccountingStandardID) & ((trialbalance_src_uc_df['period'].substr(1, 6)).isin([str(row.closing) for row in periodicaldates_src_df.filter((col("financialcycleid") == FiscalPeriodID) & (col("year") == FinancialYear)).select("closing").collect()])) & (col("IsActive") == True))

    # print("++++++++++ trialbalance_src_df ++++++")
    # trialbalance_src_df.display()

    trialbalance_dim_df = trialbalance_src_df.filter((col("Createddate") <= LastExecutionDate)).select("TrailbalanceID","Period","EntityID_fk","AccountingStdID_fk","Userglcode","Usergldescription","Member","CurrencyID_fk","openingbalance","debitamount","creditamount","closingbalance","Createddate",*alldimcols)
    print("++++++++++ trialbalance_dim_df ++++++")
    trialbalance_dim_df.display()

    castingadjustment_dim_df = castingadjustment_src_uc_df.filter((col("Fk_EntityID") == EntityID) & (col("Createddate") <= LastExecutionDate)).select("CastingAdjustmentsID","Fk_AccountingStandardID","Fk_ValidationID","PeriodValue","FinancialYear","Fk_EntityID","EntityName", "JournalNumber", "JournalDate", "UserGLCode", "UserGLDescription", "StandardGLCode","StandardGLDescription","DebitCredit","Fk_CurrencyID", "Amount", "JournalType", "Narration", "Fk_BusinessUnitID", "CategoryID", "Roundoff", "AmountsIn", "ReportType", "CreatedDate", "CreatedBy", "ModifiedDate", "ModifiedBy", "IsUserGLCode", "RoundTypeLevel",*alldimcols)


    standaloneadjustments_dim_df = standaloneadjustments_src_uc_df.filter((col("EntityID_fk")== EntityID) & (col("Createddate") <= LastExecutionDate) & (col("IsActive") == True)).select("AdjustmentID", "CategoryID_fk", "AdjustmentDisclosureType", "AccountingStdID_fk", "Period", "Year", "EntityID_fk", "UserGLcode", "Member", "DebitCredit", "CurrencyID_fk", "Amount", "IsImported","Journalnumber","Journaldate","Journaltype","narration","IsUserGLCodeAdjustment","Disclosureremarks","AccountType","AccountSubType","Createdby","Createddate","Modifiedby","Modifieddate","IsActive",*alldimcols)

    legacyadjustments_dim_df = legacyadjustments_src_uc_df.filter((col("EntityID_fk")== EntityID) & (col("Createddate") <= LastExecutionDate) & (col("IsActive") == True)).select("LegacyAdjustmentID", "Period", "EntityID_fk", "AccountingStdID_fk","Journalnumber","Journaldate","Userglcode","Member","debitcredit","CurrencyID_fk","Amount","Journaltype","Narration","Createdby","Createddate","Modifiedby","Modifieddate","BUID_fk","IsImported","CategoryID","IsMappedGLCode","EndPeriod","IsActive","NotApplicable",*alldimcols)

    insertReportRunLog(reportrunlog_src_uc_df,reportrunlog,parameterID,CategoryID)

else:
    print("else entry found in reportlog means not a first run")
    # trialbalance_src_uc_df.display()

    trialbalance_src_uc_df = trialbalance_src_uc_df.where(col("Period").substr(1, 6)==period_value_ld)
    trialbalance_src_df = trialbalance_src_uc_df.filter((trialbalance_src_uc_df['EntityID_fk'] == EntityID) & (trialbalance_src_uc_df['AccountingStdID_fk'] == AccountingStandardID) & ((trialbalance_src_uc_df['period'].substr(1, 6)).isin([str(row.closing) for row in periodicaldates_src_df.filter((col("financialcycleid") == FiscalPeriodID) & (col("year") == FinancialYear)).select("closing").collect()])) & (col("IsActive") == True))

    # print("++++++++++ trialbalance_src_df ++++++")
    # trialbalance_src_df.display()

    trialbalance_dim_df = trialbalance_src_df.filter((col("Createddate") >= LastExecutionDate) | (col("Modifieddate") >= LastExecutionDate)).select("TrailbalanceID","Period","EntityID_fk","AccountingStdID_fk","Userglcode","Usergldescription","Member","CurrencyID_fk","openingbalance","debitamount","creditamount","closingbalance","Createddate",*alldimcols)
    print("++++++++++ trialbalance_dim_df ++++++")
    trialbalance_dim_df.display()

    castingadjustment_dim_df = castingadjustment_src_uc_df.filter((col("Fk_EntityID") == EntityID) & (col("Createddate") >= LastExecutionDate) |(col("Modifieddate") >= LastExecutionDate)).select("CastingAdjustmentsID","Fk_AccountingStandardID","Fk_ValidationID","PeriodValue","FinancialYear","Fk_EntityID","EntityName", "JournalNumber", "JournalDate", "UserGLCode", "UserGLDescription", "StandardGLCode","StandardGLDescription","DebitCredit","Fk_CurrencyID", "Amount", "JournalType", "Narration", "Fk_BusinessUnitID", "CategoryID", "Roundoff", "AmountsIn", "ReportType", "CreatedDate", "CreatedBy", "ModifiedDate", "ModifiedBy", "IsUserGLCode", "RoundTypeLevel",*alldimcols)

    standaloneadjustments_dim_df = standaloneadjustments_src_uc_df.filter((col("EntityID_fk")== EntityID) & (col("Createddate") >= LastExecutionDate) | (col("Modifieddate") >= LastExecutionDate) & (col("IsActive") == True)).select("AdjustmentID", "CategoryID_fk", "AdjustmentDisclosureType", "AccountingStdID_fk", "Period", "Year", "EntityID_fk", "UserGLcode", "Member", "DebitCredit", "CurrencyID_fk", "Amount", "IsImported","Journalnumber","Journaldate","Journaltype","narration","IsUserGLCodeAdjustment","Disclosureremarks","AccountType","AccountSubType","Createdby","Createddate","Modifiedby","Modifieddate","IsActive",*alldimcols)

    legacyadjustments_dim_df = legacyadjustments_src_uc_df.filter((col("EntityID_fk")== EntityID) & (col("Createddate") >= LastExecutionDate) | (col("Modifieddate") >= LastExecutionDate) & (col("IsActive") == True)).select("LegacyAdjustmentID", "Period", "EntityID_fk", "AccountingStdID_fk","Journalnumber","Journaldate","Userglcode","Member","debitcredit","CurrencyID_fk","Amount","Journaltype","Narration","Createdby","Createddate","Modifiedby","Modifieddate","BUID_fk","IsImported","CategoryID","IsMappedGLCode","EndPeriod","IsActive","NotApplicable",*alldimcols)

In [0]:
def get_legacy_adjustment_for_mapping(legacyadjustments_dim_df,legacyadjustmentsdrilldown_union_df,catalog_schema_name):
    try:
        templegcat_df = spark.createDataFrame(
        [(category,) for category in CategoryID.split(',')],["categoryid"])
        tmplegacywithoutendperiod_df = legacyadjustments_dim_df \
        .filter((col("BUID_fk") == 0) &
                (col("CategoryID").isin([row.CategoryID for row in templegcat_df.select("CategoryID").collect()])) &
                (col("EntityID_fk") == EntityID) &
                (col("AccountingStdID_fk") == AccountingStandardID) &
                (coalesce(col("EndPeriod"),lit("999999-99")) > col("Period"))).drop("LegacyAdjustmentID")
        
        templegacy_df1 = tmplegacywithoutendperiod_df.withColumn("split_column",F.split(F.coalesce(F.col("NotApplicable"), F.lit('')), ','))

        # Explode the array created by split to mimic CROSS APPLY
        templegacy_df = templegacy_df1.select("*", F.explode("split_column").alias("value"))
        tmplegacywithoutna_df = templegacy_df.withColumn(
        "IsmappedGLUpdatedCode",
        when((col("Period") == col("value")), 0)
        .when((col("NotApplicable").isNotNull()) & (col("NotApplicable") != '')
            & (col("value") == '')
            , 0)
        .otherwise(1)
        )
        
        templegacy_df = tmplegacywithoutna_df.filter(
        (F.col("IsmappedGLUpdatedCode") == 1) &
        (~F.col("Period").isin([row.value for row in tmplegacywithoutna_df.select("value").distinct().collect()]))
        ).select("AccountingStdID_fk", "Period", "EntityID_fk",
            "JournalNumber", "JournalDate", "UserGLCode","Member",
            "DebitCredit", "CurrencyID_Fk", "Amount", "JournalType",
            "Narration", "CreatedDate", "CreatedBy", "ModifiedDate", "ModifiedBy",
            "BUID_fk", "IsImported", "CategoryID", "IsMappedGLCode",
            "EndPeriod", "NotApplicable",*alldimcols).distinct()
        
        for col_name in legacyadjustmentsdrilldown_union_df.columns:
                    if col_name not in templegacy_df.columns:
                        templegacy_df = templegacy_df.withColumn(col_name, lit(None).cast(legacyadjustmentsdrilldown_union_df.schema[col_name].dataType))

        legacyadjustmentsdrilldown_df = templegacy_df.select("AccountingStdID_fk","Period","EntityID_Fk","JournalNumber","JournalDate","UserGLCode","Member","DebitCredit","CurrencyID_Fk","Amount","JournalType","Narration","CreatedDate","CreatedBy","ModifiedDate","ModifiedBy","BUID_fk","IsImported","CategoryID","IsMappedGLCode","EndPeriod","NotApplicable","DimType1ID_fk","DimType2ID_fk","DimType3ID_fk","DimType4ID_fk","DimType5ID_fk","DimType6ID_fk","DimType7ID_fk","DimType8ID_fk","DimType9ID_fk","DimType10ID_fk","DimType11ID_fk","DimType12ID_fk","DimType13ID_fk","DimType14ID_fk","DimType15ID_fk","DimType16ID_fk","DimType17ID_fk","DimType18ID_fk","DimType19ID_fk","DimType20ID_fk","DimType21ID_fk","DimType22ID_fk","DimType23ID_fk","DimType24ID_fk","DimType25ID_fk")
        
        legacyadjustmentsdrilldown_df.write.format("delta").mode("overwrite").saveAsTable(catalog_schema_name+"."+"legacyadjustmentsdrilldown")
    except Exception as e:
        return "error"

def get_SCOA_mapping(scoa_src_uc_df,standaloneadjustments_dim_df,trialbalance_dim_df,adls_path,folder_path,catalog_schema_name):
    try:
        legacyadjustmentsdrilldown_df = spark.read.table(catalog_schema_name+"."+"legacyadjustmentsdrilldown")
        AccountingType = AccountingStandardID
        allperiodftpscoa_df = scoa_src_uc_df.filter((scoa_src_uc_df["mcoayear"] == FinancialYear) & (scoa_src_uc_df["accountingtype"] == AccountingType))

        allperiodftpscoa_df =  allperiodftpscoa_df.select("SCOAID","Member","Path","Column27","AccountSubType","UltimateParent")
        allperiodftpscoa_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"allperiodftpscoa")
        legrecords_df =  legacyadjustmentsdrilldown_df
        standaloneadj_df = standaloneadjustments_dim_df.filter((col("EntityID_fk") == EntityID) & (col("AccountingStdID_fk") == AccountingStandardID) & (col("CategoryID_fk") == CategoryID ))

        # ---------TB data-----

        trialbalance_df = trialbalance_dim_df.select(
            col("UserGLCode"),
            col("UserGLDescription"),
            col("EntityID_fk")
        ).filter(
            (col("EntityID_fk") == EntityID) &
            (col("period").substr(1, 6).isin( [row["closing"] for row in periodicaldates_src_df
                .filter((col("financialcycleid") == FiscalPeriodID) & (col("year") == FinancialYear))
                .select("closing")
                .collect()]
            ))
        )

        #combined tb,adj and leagcy adj
        tb_df= trialbalance_df.select('UserGLCode', 'UserGLDescription', 'EntityID_fk') \
            .union(standaloneadj_df.select('UserGLCode',lit(None).alias('UserGLDescription'), 'EntityID_fk')) \
            .union(legrecords_df.select('UserGLCode', lit(None).alias('UserGLDescription'), 'EntityID_fk')) \
            .groupBy("UserGLCode","EntityID_fk") \
            .agg(first(col("UserGLDescription"), True).alias("UserGLDescription"))

        # first time tb upload
        year_period= periodicaldates_src_df \
            .filter((col("FinancialCycleID") == FiscalPeriodID) & (col("Year") == FinancialYear) & (col("Type") == 'Yearly')) \
            .select("closing") \
            .collect()

        year_period_value = year_period[0]["closing"] if year_period else ""

        isautomapped_df = categorydropdown_src_df.filter(categorydropdown_src_df["CategoryID"] == CategoryID) \
        .select("IsAutomapped").collect()

        if isautomapped_df:
            isautomapped_df = isautomapped_df[0][0]
        else:
            print("No rows found for the given CategoryID.")
        

        if isautomapped_df == 1:
            print("IsAutomapped is 1")
            SCOAMappingTable1_df = tb_df.join(
            allperiodftpscoa_df,
            tb_df["UserGLCode"] == allperiodftpscoa_df["Member"],
            "inner").select(
            col("EntityID_fk").alias("EntityId"),
            concat(lit(year_period_value), lit('-12')).alias("Period"),
            lit(AccountingStandardID).alias("accountingtype"),
            col("Member").alias("EyGlcode"),
            col("UserGLCode"),
            lit(CategoryID).alias("CategoryID"),
            col("UltimateParent"),
            lit("admin").alias("createdby"),
            lit("admin").alias("modifiedby"),
            lit(current_timestamp()).alias("createdDate"),
            lit(current_timestamp()).alias("modifiedDate"),
            lit(None).alias("PresentationLayer"),
            col("UserGLDescription")
            ).join(
            masterscoamapping_src_uc_df.filter((col("EntityId") == EntityID) & (col("CategoryID") == CategoryID) & (col("mapped") == 1)),
            tb_df["UserGLCode"] == masterscoamapping_src_uc_df["UserGLCode"],
            "left_anti"
            )

            SCOAMappingTable_df = SCOAMappingTable1_df.join(
            masterscoamapping_src_uc_df.filter((col("CategoryID") == CategoryID) & (col("unmapped") == 1)), \
            SCOAMappingTable1_df["UserGLCode"] == masterscoamapping_src_uc_df["UserGLCode"], \
            "left_anti") \
            .select("EntityId", "Period", "accountingtype", "EYGLCode", "UserGLCode", "CategoryID", "UltimateParent", lit("admin").alias("CreatedBy"), lit("admin").alias("ModifiedBy"), lit(current_timestamp()).alias("CreatedDate"), lit(current_timestamp()).alias("ModifiedDate"), "PresentationLayer", "UserGLDescription") \
            .union(
                masterscoamapping_src_uc_df
                .filter((col("EntityId") == EntityID) & (col("CategoryID") == CategoryID) & (col("mapped") == 1))
                .select("EntityID","ReportingFrequency","accountingtype","EYGLCode","UserGLCode","CategoryID","UltimateParent","CreatedBy","modifiedby","CreatedDate","ModifiedDate","PresentationLayer","UserGLDescription")
            )
    
            SCOAMappingTable_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"SCOAMappingTable")
            
        else:
            print("IsAutomapped is 0")
            SCOAMappingTable1_df = masterscoamapping_src_uc_df \
                .filter((col("EntityId") == EntityID) & (col("CategoryID") == CategoryID) & (col("mapped") == 1)) \
                .select("EntityId",col("ReportingFrequency").alias("Period"),"accountingtype","EYGLCode","UserGLCode","CategoryID","UltimateParent","CreatedBy","modifiedby","CreatedDate","ModifiedDate","PresentationLayer","UserGLDescription")

            SCOAMappingTable_df =  SCOAMappingTable1_df \
                .filter((col("EntityId") == EntityID) & (col("CategoryID") == CategoryID) & (col("mapped") == 1)) \
                .select("EntityId","Period","accountingtype","EYGLCode","UserGLCode","CategoryID","UltimateParent","CreatedBy","modifiedby","CreatedDate","ModifiedDate","PresentationLayer","UserGLDescription")

            SCOAMappingTable_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"SCOAMappingTable")
    
        SCOAMappingTable_df.display()
    except Exception as e:
        return "error"

def get_legacy_adjustment_by_category_drilldown(legacyadjustments_dim_df,legacyadjustmentsdrilldown_union_df,AccountingStandardID,EntityID,LCurrperiod,LCategoryID):
    try:
        tmplegacywithoutendperiod_df = legacyadjustments_dim_df \
        .filter((col("BUID_fk") == 0) &
                (col("CategoryID") == LCategoryID) &
                (col("EntityID_fk") == EntityID) &
                (col("AccountingStdID_fk") == AccountingStandardID) & (col("Period") <= LCurrperiod) &
                (coalesce(col("EndPeriod"),lit("999999-99")) > LCurrperiod)).drop("LegacyAdjustmentID")

        templegacy_df1 = tmplegacywithoutendperiod_df.withColumn(
            "split_column",
            F.split(F.coalesce(F.col("NotApplicable"), F.lit('')), ',')
        )


        # Explode the array created by split to mimic CROSS APPLY
        templegacy_df = templegacy_df1.select("*", F.explode("split_column").alias("value"))


        tmplegacywithoutna_df = templegacy_df.withColumn(
        "IsmappedGLUpdatedCode",
        when((col("Period") == col("value")), 0)
        .when((col("NotApplicable").isNotNull()) & (col("NotApplicable") != '')
            & (col("value") == '')
            , 0)
        .otherwise(1)
        )

        # reset  ismappedglcode to 0 for na periods
        templegacy_df = tmplegacywithoutna_df.filter(
        (F.col("IsmappedGLUpdatedCode") == 1) &
        (~F.col("Period").isin([row.value for row in tmplegacywithoutna_df.select("value").distinct().collect()]))
        ).select(
            "AccountingStdID_fk", "Period", "EntityID_fk",
            "JournalNumber", "JournalDate", "UserGLCode","Member",
            "DebitCredit", "CurrencyID_Fk", "Amount", "JournalType",
            "Narration", "CreatedDate", "CreatedBy", "ModifiedDate", "ModifiedBy",
            "BUID_fk", "IsImported", "CategoryID", "IsMappedGLCode",
            "EndPeriod", "NotApplicable",*alldimcols
        ).distinct()

        for col_name in legacyadjustmentsdrilldown_union_df.columns:
                    if col_name not in templegacy_df.columns:
                        templegacy_df = templegacy_df.withColumn(col_name, lit(None).cast(legacyadjustmentsdrilldown_union_df.schema[col_name].dataType))


        legacyadjustmentsdrilldown_df = templegacy_df.select("AccountingStdID_fk", "Period", "EntityID_fk",
            "JournalNumber", "JournalDate", "UserGLCode","Member",
            "DebitCredit", "CurrencyID_Fk", "Amount", "JournalType",
            "Narration", "CreatedDate", "CreatedBy", "ModifiedDate", "ModifiedBy",
            "BUID_fk", "IsImported", "CategoryID", "IsMappedGLCode",
            "EndPeriod", "NotApplicable","DimType1ID_fk","DimType2ID_fk","DimType3ID_fk","DimType4ID_fk","DimType5ID_fk","DimType6ID_fk","DimType7ID_fk","DimType8ID_fk","DimType9ID_fk","DimType10ID_fk","DimType11ID_fk","DimType12ID_fk","DimType13ID_fk","DimType14ID_fk","DimType15ID_fk","DimType16ID_fk","DimType17ID_fk","DimType18ID_fk","DimType19ID_fk","DimType20ID_fk","DimType21ID_fk","DimType22ID_fk","DimType23ID_fk","DimType24ID_fk","DimType25ID_fk")

        return legacyadjustmentsdrilldown_df

    except Exception as e:
        return "error"


In [0]:
get_legacy_adjustment_for_mapping(legacyadjustments_dim_df,legacyadjustmentsdrilldown_union_df,catalog_schema_name)
get_SCOA_mapping(scoa_src_uc_df,standaloneadjustments_dim_df,trialbalance_dim_df,adls_path,folder_path,catalog_schema_name)

In [0]:
if extflag != 1 or (extflag == 1 and extendedmonthdiff <= 12) or (extflag == 1 and extendedmonthdiff > 12 and int(extendedcycleyear) != int(FinancialYear)): 
    print("standard period")
    periodicaldateswithlargeperiod_df = legacysddates_src_df.alias("ld").join(periodicaldates_src_df.alias("mp"),
    (col("ld.ftpperiod") >= col("mp.Order")),
    "inner"
    ).filter((col("financialcycleid") == FiscalPeriodID) & (col("mp.closing") <= col("ld.currentvalue")) & (col("mp.closing") >= col("ld.openingperiodvalue"))).select(
    col("ld.PeriodType"),
    col("mp.closing"),
    col("ld.LegacyPeriod"),
    col("ld.numericvalueofperiod")
    ).distinct()

    periodicaldateswithequalperiod_df = legacysddates_src_df.alias("ld").join(periodicaldates_src_df.alias("mp"),
    col("mp.closing") <= col("ld.currentvalue"),
    "inner"
    ).filter((col("financialcycleid") == FiscalPeriodID) & (col("mp.closing") >= col("ld.openingperiodvalue")) & (col("ld.ftpperiod") == col("mp.Order"))).select(
    col("ld.PeriodType"),
    col("mp.closing"),
    col("ld.LegacyPeriod"),
    col("ld.numericvalueofperiod")
    ).distinct()

    if PeriodValueInYHQM == 0: 
        print("inside PeriodValueInYHQM == 0 ")
        legacysddates_df = legacysddates_src_df.filter(~col("periodtype").like('%YTD'))
        legacysddates_period_df = legacysddates_df.withColumn("PeriodType", concat(col("PeriodType"), lit("_YTD")))

        tballperiod_df = (
        trialbalance_dim_df.alias("tb")
        .join(periodicaldateswithlargeperiod_df.alias("ld"), (col("tb.Period").substr(1, 6) == col("ld.closing")) &
        (col("tb.period").substr(-2, 2) <= abs(col("numericvalueofperiod"))),"inner")
        .where(col("EntityID_fk") == EntityID)
        .select("tb.Period", col("tb.Period").substr(1, 6).alias("stdperiod"),"ld.*",*alldimcols)
        .orderBy("PeriodType")
        )
        tballperiod_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tballperiod")
        tballperiod_df = spark.read.parquet(adls_path+folder_path+"tballperiod", schema = tballperiod_schema)

        
        tblegacymatchperiod_df = (trialbalance_dim_df.alias("tb")
        .join(periodicaldateswithequalperiod_df.alias("ld"), (col("tb.Period") == concat(col("closing"),lit("-"),col("legacyperiod").substr(-2, 2))),"inner")
        .where(col("tb.EntityID_fk") == EntityID)
        .select("Period", col("Period").substr(1, 6).alias("stdperiod"),"ld.*",*alldimcols)
        .orderBy("PeriodType")
        )

        
        tblegacymatchperiod_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tblegacymatchperiod")
        tblegacymatchperiod_df = spark.read.parquet(adls_path+folder_path+"tblegacymatchperiod", schema = tblegacymatchperiod_schema)
        
        priorityseq1_df = (
        tballperiod_df.alias("tb").join(
            (
                legacysddates_period_df.alias("ld")
                .join(
                    tblegacymatchperiod_df.alias("mp"),
                    (col("ld.LegacyPeriod") == col("mp.LegacyPeriod")) & (col("ld.periodtype") == col("mp.periodtype")),
                    "left"
                )
                .where(col("mp.LegacyPeriod").isNull())
                .select("ld.LegacyPeriod", "ld.periodtype", "ld.ftpperiod")
                .distinct()
            ),
            (col("tb.Period").substr(1, 6) <= col("ld.LegacyPeriod").substr(1, 6)) &
            (col("tb.periodtype") == col("ld.periodtype")) &
            (col("tb.Period").substr(-2, 2) != col("ld.LegacyPeriod").substr(-2, 2)),
            "inner"
        )
        .select("tb.*", "ld.ftpperiod")
        )

        
        window_spec = Window.partitionBy("stdperiod", "periodtype").orderBy(col("Period").desc())

        priorityseq_df = priorityseq1_df \
        .withColumn("priorityseq", row_number().over(window_spec)) \
        .withColumn("order", when(col("Period").substr(-2, 2) >= '12', 4)
        .when(col("Period").substr(-2, 2) == '06', 3)
        .when(col("Period").substr(-2, 2) == '03', 2)
        # .when(col("Period").substr(-2, 2) == '01', 1)
        .otherwise(1)) \
        .select(
            col("Period"),
            col("stdperiod"),
            col("PeriodType"),
            col("closing"),
            col("legacyperiod"),
            col("numericvalueofperiod"),
            col("priorityseq"),
            col("order"),
            col("ftpperiod")
        ) \
        .orderBy("periodtype")
            
        priorityseq_df = priorityseq_df.filter(col("priorityseq") == 1)

        priorityseq_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"priorityseq")
        priorityseq_df = spark.read.parquet(adls_path+folder_path+"priorityseq", schema = priorityseq_schema)

        legacysddates_final_df = legacysddates_period_df.alias("ld") \
        .join(priorityseq_df.alias("ps"),(col("ld.PeriodType") == col("ps.PeriodType")) & (col("ld.currentvalue") == col("ps.stdperiod")),"left_outer") \
        .select("ld.*",col("ps.PeriodType").alias("ps_PeriodType"),col("ps.stdperiod").alias("ps_stdperiod"),col("ps.Period")) \
        .withColumn("LegacyPeriod",when((col("ld.periodtype") == col("ps_PeriodType")) & (col("ld.currentvalue") == col("ps_stdperiod")),col("ps.Period"))
                .otherwise(col("ld.LegacyPeriod"))) \
    .drop("ps_PeriodType", "ps_stdperiod", col("ps.Period"))

        legacysddates_final_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates")  

        ytddates_df = (
                tblegacymatchperiod_df
                .filter((col("PeriodType").like("%YTD")) | (col("PeriodType").like("%Year")))
                .select("PeriodType", col("legacyperiod").substr(1, 6).alias("CurrentValue"), col("Period").alias("FTPValue"))
                .union(
                    priorityseq_df
                    .filter((col("PeriodType").like("%YTD")) | (col("PeriodType").like("%Year")))
                    .select("PeriodType", col("legacyperiod").substr(1, 6).alias("CurrentValue"), col("Period").alias("FTPValue"))
                )
            )

        ytddates_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"ytddates") 
    elif PeriodValueInYHQM == 1:   
        print("inside PeriodValueInYHQM == 1")
        tballperiod_df = (
        trialbalance_dim_df.alias("tb")
        .join(
            periodicaldateswithlargeperiod_df.alias("ld"),
            (
                (col("tb.Period").substr(1, 6) == col("ld.closing")) &
                (col("tb.Period").substr(-2, 2) <= abs(col("ld.numericvalueofperiod")))
            ),
            "inner"
        )
        .where(col("EntityID_fk") == EntityID)
        .select("Period", col("Period").substr(1, 6).alias("stdperiod"),"ld.*",*alldimcols)
        .orderBy("PeriodType")
        ).distinct()
        tballperiod_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tballperiod")

        tballperiod_df = spark.read.parquet(adls_path+folder_path+"tballperiod", schema = tballperiod_schema)

        tblegacymatchperiod_df = (trialbalance_dim_df.alias("tb").join(
        periodicaldateswithequalperiod_df.alias("ld"),
        (
            col("tb.Period") == concat(col("closing"),lit("-"),col("legacyperiod").substr(-2, 2))
        ),"inner")
        .where(col("EntityID_fk") == EntityID)
        .select("Period", col("Period").substr(1, 6).alias("stdperiod"),"ld.*",*alldimcols)
        .orderBy("PeriodType")
        ).distinct()

        tblegacymatchperiod_df = tblegacymatchperiod_df.filter(~(
            (F.expr("substr(Period, 1, 6)") == F.expr("substr(LegacyPeriod, 1, 6)")) &
            (F.expr("cast(substr(Period, -2) as int)") != F.abs(col("NumericValueofPeriod")))
        ))

        tblegacymatchperiod_df = tblegacymatchperiod_df.filter(~(
            (F.expr("substr(Period, 1, 6)") == F.expr("substr(LegacyPeriod, 1, 6)")) &
            (F.expr("cast(substr(Period, -2) as int)") < F.abs(col("NumericValueofPeriod")))
        ))

        tblegacymatchperiod_df1 = tblegacymatchperiod_df

        tblegacymatchperiod_df1 = tblegacymatchperiod_df1.filter(~(
            (F.expr("substr(Period, 1, 6)") != F.expr("substr(LegacyPeriod, 1, 6)")) &
            (F.expr("cast(substr(Period, -2) as int)") == F.abs(col("NumericValueofPeriod")))
        ))
            
        legacysddates_tmp_df = legacysddates_src_df.alias("ls").join(tblegacymatchperiod_df1.alias("mp"),((col("ls.LegacyPeriod") == col("mp.LegacyPeriod")) & (col("ls.periodtype") == col("mp.periodtype"))),"left") \
        .filter(col("mp.LegacyPeriod").isNull()) \
                    .select("ls.LegacyPeriod","ls.periodtype","ls.ftpperiod").withColumn("fromperiod",concat(F.year(F.add_months(F.to_date(F.concat(F.substring(col('LegacyPeriod'), 1, 4),F.lit('-'),F.substring(col('ls.LegacyPeriod'), 5, 2),F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(LegacyPeriod, 2) AS INT)"))), 
            F.substring(F.date_format(F.add_months(F.to_date(F.concat(F.substring(col('LegacyPeriod'), 1, 4),F.lit('-'),F.substring(col('LegacyPeriod'), 5, 2),
                            F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(LegacyPeriod, 2) AS INT)")),'yyyy-MM-dd' ),6,2)))


        priorityseq_df1 = tballperiod_df.alias("tb").join(legacysddates_tmp_df.alias("ld"), ((col("tb.Period").substr(1, 6) <= col("ld.LegacyPeriod").substr(1, 6)) &
                        (col("tb.Period").substr(1, 6) > col("fromperiod")) & (col("tb.PeriodType") == col("ld.periodtype"))),"inner").filter((col("tb.Period").substr(-2, 2) != col("ld.LegacyPeriod").substr(-2, 2))).select("tb.*","ld.ftpperiod")

        window_spec = Window().partitionBy("stdperiod", "periodtype").orderBy(col("Period").desc())

        priorityseq_df = priorityseq_df1 \
                .withColumn("priorityseq", row_number().over(window_spec)) \
                .withColumn("order", when(col("Period").substr(-2, 2) >= '12', 4)
                .when(col("Period").substr(-2, 2) >= '12', 5)
                .when(col("Period").substr(-2, 2) >= '09', 4)
                .when(col("Period").substr(-2, 2) >= '06', 3)
                .when(col("Period").substr(-2, 2) >= '03', 2)
                .when(col("Period").substr(-2, 2) >= '01', 1)
                .otherwise(1)) \
                .select(
                    col("Period"),
                    col("stdperiod"),
                    col("PeriodType"),
                    col("closing"),
                    col("legacyperiod"),
                    col("numericvalueofperiod"),
                    col("priorityseq"),
                    col("order"),
                    col("ftpperiod")
                ) \
                .orderBy("periodtype")

        # #################PRIORITY : fetch data for Q3 & Q4 (lower period)in case data for H1 Present but H2 not present #####################

        prioritylevel2temp_1_df = tblegacymatchperiod_df.alias("tbm").join(legacysddates_src_df.alias("ld"),(col("ld.periodtype") == col("tbm.periodtype")) & 
                      (col("ld.legacyperiod") == col("tbm.legacyperiod")), "inner") \
            .groupBy("tbm.periodtype", "ld.FinancialYear", "ld.LegacyPeriod", "ftpperiod") \
            .agg(
            max(col("tbm.Period")).alias("Period"))
        prioritylevel2temp_2_df = prioritylevel2temp_1_df.withColumn("ftpperiod",
            when(col("ftpperiod") == 1, col("ftpperiod"))
            .otherwise(col("ftpperiod") - 1)) \
            .withColumn("fromperiod",
            concat(F.year(F.add_months(F.to_date(F.concat(F.substring(col('ld.LegacyPeriod'), 1, 4),F.lit('-'),F.substring(col('ld.LegacyPeriod'), 5, 2),F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(ld.LegacyPeriod, 2) AS INT)"))), 
            F.substring(F.date_format(F.add_months(F.to_date(F.concat(F.substring(col('ld.LegacyPeriod'), 1, 4),F.lit('-'),F.substring(col('ld.LegacyPeriod'), 5, 2),
            F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(ld.LegacyPeriod, 2) AS INT)")),'yyyy-MM-dd' ),6,2)).alias("fromperiod"))

        prioritylevel2temp1_df = periodicaldates_src_df.alias("pd").join(prioritylevel2temp_2_df.alias("ld"),(col("pd.closing") > col("ld.fromperiod")) &
        (col("pd.closing") <= col("ld.LegacyPeriod").substr(1, 6)) & 
        (col("pd.Order") == col("ld.ftpperiod")) &
        (col("pd.Year") == col("ld.financialyear")) &
        (col("FinancialCycleID") == FiscalPeriodID) &
        (col("ld.legacyperiod") != col("ld.Period"))
        ,"inner")
        prioritylevel2temp_df = prioritylevel2temp1_df.withColumn(
        "closing",
        col("closing") +
        when(col("ftpperiod") == 5, "-12")
        .when(col("ftpperiod") == 4, "-09")
        .when(col("ftpperiod") == 3, "-06")
        .when(col("ftpperiod") == 2, "-03")
        .when(col("ftpperiod") == 1, "-01")
        .otherwise("")) \
        .select("periodtype","ld.Period","ld.legacyperiod","ftpperiod","closing")
     
        prioritylevel2temp_df = trialbalance_dim_df.alias("tb").join(prioritylevel2temp_df.alias("pt"),col("tb.Period") == col("pt.closing"),"inner").where( col("EntityID_fk") == EntityID).select("tb.Period",
							col("tb.Period").substr(1,6).alias("stdperiod"),
							"periodtype",
							col("tb.Period").substr(1,6).alias("closing"),
							"legacyperiod",
							"ftpperiod",*alldimcols)

        window_spec = Window.partitionBy("stdperiod", "periodtype").orderBy(col("Period").desc())

        prioritylevel2_df = prioritylevel2temp_df \
                .withColumn("priorityseq", row_number().over(window_spec)) \
                .withColumn("order", when(col("Period").substr(-2, 2) >= '12', 4)
                .when(col("Period").substr(-2, 2) >= '12', 5)
                .when(col("Period").substr(-2, 2) >= '09', 4)
                .when(col("Period").substr(-2, 2) >= '06', 3)
                .when(col("Period").substr(-2, 2) >= '03', 2)
                .when(col("Period").substr(-2, 2) >= '01', 1)
                .otherwise(1)) \
                .select("*") \
                .orderBy("periodtype")

        priorityseq_upt_df = priorityseq_df.filter(col("priorityseq") == 1)
        prioritylevel2_df = prioritylevel2_df.select("Period","stdperiod","PeriodType","closing","legacyperiod",col("ftpperiod").alias("numericvalueofperiod"),"priorityseq","Order",*alldimcols).filter(col("priorityseq") == 1)

        prioritylevel2_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"prioritylevel2")
        prioritylevel2_df = spark.read.parquet(adls_path+folder_path+"prioritylevel2", schema = prioritylevel2_schema)

        # ######################### For Next level priority check ##############################

        order_df = (
        priorityseq_upt_df
        .groupBy("PeriodType")
        .agg({"Order": "max"})
        .withColumnRenamed("max(Order)", "order")
        .exceptAll(
        priorityseq_upt_df.alias("ps")
        .join(
            legacysddates_src_df.alias("ld"),
            (col("ps.PeriodType") == col("ld.PeriodType")) &
            (col("ps.LegacyPeriod") == col("ld.LegacyPeriod")) &
            (col("ps.Order") == col("ld.ftpperiod") - 1),
            "inner"
        )
        .select("ps.PeriodType", "ps.Order")
        )
        )

        if not order_df.isEmpty():
            print("if order not empty")
            print("if count is greater than 0")
            ctePS_df = priorityseq_upt_df.groupBy("PeriodType","LegacyPeriod").agg(max(col("Order")).alias("Order")).select("PeriodType","LegacyPeriod","Order")

            legacysddates_df = legacysddates_src_df.alias("ld").join(ctePS_df.alias("ps"),
            (col("ps.periodtype") == col("ld.PeriodType")) &
            (col("ps.legacyperiod") == col("ld.legacyperiod")),"left_outer").select("ld.*",col("ps.Order").alias("ps_Order"),col("ps.periodtype").alias("ps_periodtype"),col("ps.legacyperiod").alias("ps_legacyperiod")) \
            .withColumn("ftpperiod",when((col("periodtype") == col("ps_periodtype")) & (col("legacyperiod") == col("ps_legacyperiod")),col("ps_Order") + 1).otherwise(col("ftpperiod"))).drop("ps_Order","ps_periodtype","ps_legacyperiod")
            legacysddates_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates") 

        legacysddates_df = spark.read.parquet(adls_path+folder_path+"legacysddates", schema = legacysddates_schema)
        tblegacymatchperiod_df = (
        tblegacymatchperiod_df.alias("tbm")
        .join(
            prioritylevel2_df.alias("pl"),
            (col("tbm.PeriodType") == col("pl.PeriodType")) &
            (col("tbm.LegacyPeriod") == col("pl.LegacyPeriod")),
            "left_anti"
        )
        )

        priorityseq_upt1_df = (
        priorityseq_upt_df.alias("ps")
        .join(
            legacysddates_df.alias("ld"),
            (col("ps.PeriodType") == col("ld.PeriodType")) &
            (col("ps.LegacyPeriod") == col("ld.LegacyPeriod")) &
            (col("ps.Order") != col("ld.ftpperiod") - 1),
            "left_anti"
        )
        )   
        
        priorityseq_upt1_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"priorityseq")
        priorityseq_upt1_df = spark.read.parquet(adls_path+folder_path+"priorityseq", schema = priorityseq_schema)
        
        bspriority_df = (
        legacysddates_df.alias("ldd")
        .join(priorityseq_upt1_df.alias("pseq"), (col("ldd.PeriodType") == col("pseq.PeriodType")) & (col("ldd.LegacyPeriod") == col("pseq.LegacyPeriod")), "left")
        .join(prioritylevel2_df.alias("pl2"), (col("ldd.PeriodType") == col("pl2.PeriodType")) & (col("ldd.LegacyPeriod") == col("pl2.LegacyPeriod")), "left")
        .join(tblegacymatchperiod_df.alias("lm"), (col("ldd.PeriodType") == col("lm.PeriodType")) & (col("ldd.LegacyPeriod") == col("lm.LegacyPeriod")), "left")
        .groupBy("ldd.PeriodType", "ldd.LegacyPeriod", "ldd.CurrentValue")
        .agg(
            max(coalesce("pseq.Period", "pl2.Period", "lm.Period")).alias("Period"),
            max(coalesce("pseq.StdPeriod", "pl2.StdPeriod", "lm.StdPeriod")).alias("StdPeriod")
        )
        .filter(col("Period").isNotNull())
        .groupBy("PeriodType", "LegacyPeriod", "CurrentValue")
        .agg(
            max("Period").alias("Period"),
            max("StdPeriod").alias("StdPeriod")
        )
        )

        legacysddates_final_df = legacysddates_df.alias("ld") \
        .join(
            bspriority_df.alias("ps"),
            (col("ld.PeriodType") == col("ps.PeriodType")) &
            (col("ld.LegacyPeriod") == col("ps.LegacyPeriod")),
            "left_outer").select("ld.*",col("ps.Period").alias("ps_Period"),col("ps.stdperiod").alias("ps_stdperiod")).withColumn("legacyperiod",F.coalesce("ps_Period","ld.legacyperiod")) \
        .withColumn("currentvalue",F.coalesce("ps_stdperiod","ld.currentvalue")).drop("ps_Period","ps_stdperiod")
        
        legacysddates_final_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates") 
        legacysddates_final_df = spark.read.parquet(adls_path+folder_path+"legacysddates", schema = legacysddates_schema)
        
        tblegacymatchperiod_df2 = tblegacymatchperiod_df.filter((col("Period").substr(1,6) != col("LegacyPeriod").substr(1,6)) & (col("Period").substr(-2, 2).cast("int") == abs(col("numericvalueofperiod")))).alias("a").join(tblegacymatchperiod_df.alias("b"),col("a.LegacyPeriod") == col("b.Period"),"left_anti").select("a.*")

        tblegacymatchperiod_updated1_df = tblegacymatchperiod_df.alias("tb").join(tblegacymatchperiod_df2.alias("del"),col("tb.LegacyPeriod")==col("del.LegacyPeriod"),"inner").select("tb.*")

        tblegacymatchperiod_df = tblegacymatchperiod_df.exceptAll(tblegacymatchperiod_updated1_df)
        
        tblegacymatchperiod_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tblegacymatchperiod")
        tblegacymatchperiod_df = spark.read.parquet(adls_path+folder_path+"tblegacymatchperiod", schema = tblegacymatchperiod_schema)
        

        if countofcomp == 0 or countofcomp ==1:
            print("inside PeriodValueInYHQM == 1 and countofcomp == 0 or countofcomp ==1")
            ytddates_df = tblegacymatchperiod_df.filter((col("Periodtype").like('%YTD')) | (col("Periodtype").like('%Year'))).select("PeriodTYpe",col("legacyperiod").substr(1,6).alias("CurrentValue"),col("Period").alias("FTPValue")) \
            .union(priorityseq_upt1_df.filter((col("Periodtype").like('%YTD')) | (col("Periodtype").like('%Year'))).select("PeriodTYpe",col("legacyperiod").substr(1,6).alias("CurrentValue"),col("Period").alias("FTPValue"))) \
            .union(prioritylevel2_df.filter((col("Periodtype").like('%YTD')) | (col("Periodtype").like('%Year'))).select("PeriodTYpe",col("legacyperiod").substr(1,6).alias("CurrentValue"),col("Period").alias("FTPValue"))).distinct()

            ytddates_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"ytddates") 
           
        if countofcomp > 1:
            print("inside PeriodValueInYHQM == 1 and countofcomp > 1")
            
            ytddates_df = (
            legacysddates_final_df
            .filter(~(col("periodtype").isin("currentPeriod", "PreviousPeriod"))).select("periodtype","pcurrentvalue","legacyperiod")
            .exceptAll(
            legacysddates_final_df
            .filter(col("legacyperiod").substr(1, 6) != col("pcurrentvalue")).alias("a")
            .join(
                tblegacymatchperiod_df.alias("b"),
                (col("a.legacyperiod") == col("b.Period")) &
                (col("a.periodtype") == col("b.PeriodType")),
                "left_anti"
            )
            .select("a.periodType", col("a.pcurrentvalue"), col("a.legacyperiod"))
            )
            .union(
                priorityseq_upt1_df
                .select("PeriodType", col("legacyperiod").substr(1, 6).alias("pcurrentvalue"), col("Period").alias("legacyperiod"))
            )
            .union(
                prioritylevel2_df
                .select("PeriodType", col("legacyperiod").substr(1, 6).alias("pcurrentvalue"), col("Period").alias("legacyperiod"))
            ).distinct()
            )
                        
            ytddates_df = ytddates_df.withColumnRenamed("pcurrentvalue","CurrentValue").withColumnRenamed("legacyperiod","FTPValue")
            ytddates_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"ytddates") 

In [0]:
legacysddates_df = spark.read.parquet(adls_path+folder_path+"legacysddates", schema = legacysddates_schema)
prioritylevel2_df = spark.read.parquet(adls_path+folder_path+"prioritylevel2", schema = prioritylevel2_schema)
ytddates_df = spark.read.parquet(adls_path+folder_path+"ytddates", schema = ytddates_schema)

In [0]:
############################ extended financial cycle changes #############################

if extflag == 1 and extendedmonthdiff <= 12:
    print("extflag == 1 and extendedmonthdiff <= 12")
    print("extended financial cycle changes")
    print("less than 12 month cycle")
    legacysddates_updated_df1 = legacysddates_df.alias("ld").join(periodicaldatesforextensionperiod_src_df.alias("pd"),
            (col("pd.closing") == col("ld.pcurrentvalue")) &
            (col("pd.Value") == col("ld.Value")),"left_outer").filter((col("ld.financialyear") == extendedcycleyear) &
        (col("pd.entityid") == EntityID)) \
            .select("ld.*")
    
    legacysddates_updated_df2 = legacysddates_df.exceptAll(legacysddates_updated_df1)
    
    legacysddates_updated_df3 = legacysddates_updated_df1.alias("ld").join(
        periodicaldatesforextensionperiod_src_df.filter(col("entityid") == EntityID).alias("pd"),
        (col("pd.closing") == col("ld.pcurrentvalue")) & (col("pd.Value") == col("ld.Value"))
    ).select(
        ("ld.*")
    ).withColumn(
        "p", lit(None)
    ).withColumn(
        "ppp", lit(None)
    ).withColumn(
        "py", lit(None)
    ).withColumn(
        "pypp", lit(None)
    ).withColumn(
        "ppy", lit(None)
    ).withColumn(
        "p3y", lit(None)
    ).withColumn(
        "p4y", lit(None)
    )

    legacysddates_updated_df = legacysddates_updated_df2.union(legacysddates_updated_df3)
    
    min_currentvalue = legacysddates_updated_df.filter(col("p").isNull()).selectExpr("MIN(currentvalue)").collect()[0][0]

    legacysddates_curr_df = legacysddates_updated_df.filter(col("currentvalue") < min_currentvalue)
    legacysddates_updt_df = legacysddates_updated_df.exceptAll(legacysddates_curr_df)
    legacysddates_updt_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates") 

In [0]:
def get_extended_period_legacydates(legacysddates_df,periodicaldatesforextensionperiod_src_df,extvalue,FiscalPeriodID,FinancialYear,EntityID,extopen,extclose,extendedcycleyear):
    print("inside get_extended_period_legacydates function")
    legacysddates_ext1_df =legacysddates_df.alias("ld").join(
        periodicaldatesforextensionperiod_src_df.alias("pd"),(col("pd.closing") == col("ld.pcurrentvalue")) & (col("pd.Value") == col("ld.Value")),how="inner").filter((col("entityid") == EntityID) & (col("financialcycleid") == FiscalPeriodID) & (col("financialyear") == extendedcycleyear)).select("ld.*")
    
    legacysddates_ext2_df = legacysddates_df.exceptAll(legacysddates_ext1_df)
    
    legacysddates_ext3_df = legacysddates_ext1_df.alias("ld").join(
        periodicaldatesforextensionperiod_src_df.filter((col("EntityID") == EntityID)).alias("pd"),
        (col("pd.closing") == col("ld.pcurrentvalue")) &
        (col("pd.Value") == col("ld.Value")),how="inner").select("ld.*",col("pd.Value").alias("pd_Value"),col("pd.Year").alias("pd_Year"),col("pd.opening").alias("pd_opening")) \
            .withColumn("Value",col("pd_Value"))\
                .withColumn("financialyear",col("pd_Year")) \
                    .withColumn("openingperiodvalue",col("pd_opening")) \
                        .withColumn("CurrentValue",col("ld.pcurrentvalue")).drop("pd_Value","pd_Year","pd_opening")
   
    legacysddates_ext_df =  legacysddates_ext2_df.union(legacysddates_ext3_df)

    legacysddates_updated_df1 = legacysddates_ext_df.alias("ld").join(periodicaldatesforextensionperiod_src_df.alias("pd"),
            (col("pd.closing") == col("ld.pcurrentvalue")) &
            (col("pd.Value") == col("ld.Value")),"left_outer").filter((col("ld.financialyear") == extendedcycleyear) &
        (col("pd.entityid") == EntityID)) \
            .select("ld.*")

    legacysddates_updated_df2 = legacysddates_ext_df.exceptAll(legacysddates_updated_df1)

    legacysddates_updated_df3 = legacysddates_updated_df1.alias("ld").join(
        periodicaldatesforextensionperiod_src_df.filter(col("entityid") == EntityID).alias("pd"),
        (col("pd.closing") == col("ld.pcurrentvalue")) & (col("pd.Value") == col("ld.Value"))
    ).select(
        ("ld.*"),
        col("pd.Value").alias("pd_Value"),
        col("pd.closing").alias("pd_closing"),
        col("pd.opening").alias("pd_opening")
    ).withColumn(
        "openingperiodvalue",
        when(
            (col("pd_closing") == col("ld.pcurrentvalue")) & (col("pd_Value") == col("ld.Value")),
            lit(extopen)
        ).otherwise(col("ld.openingperiodvalue"))
    ).withColumn(
        "CastingPeriod",
        when(
            (col("pd_closing") == col("ld.pcurrentvalue")) & (col("pd_Value") == col("ld.Value")),
            concat(col("Value"), lit('-'), lit(extopen), lit('-'), col("currentvalue"))
        ).otherwise(col("ld.CastingPeriod"))
    ).withColumn(
        "p", lit('')
    ).withColumn(
        "ppp", lit('')
    ).withColumn(
        "py", lit('')
    ).withColumn(
        "pypp", lit('')
    ).withColumn(
        "ppy", lit('')
    ).withColumn(
        "p3y", lit('')
    ).withColumn(
        "p4y", lit('')
    ).withColumn(
        "legacyperiod",
        when(
            col("ld.Value").isin("H1", "Q1", "Year", "9M"),
            concat(
                col("pcurrentvalue"), 
                lit("-"),
                when(
                    length(months_between(to_date(concat(lit("01-"),col("pd_closing").substr(-2, 2),lit("-"),col("pd_closing").substr(1, 4)),"dd-MM-yyyy"
            ),to_date(concat(lit("01-"),col("pd_opening").substr(-2, 2),lit("-"),col("pd_opening").substr(1, 4)),"dd-MM-yyyy"))+1
                ).cast("int") == 1,
                concat(lit('0'), months_between(
                    to_date(concat(lit("01-"), col("pd_closing").substr(-2, 2), lit("-"), col("pd_closing").substr(1, 4)), 
                    "dd-MM-yyyy"), 
                    to_date(concat(lit("01-"), col("pd_opening").substr(-2, 2), lit("-"), col("pd_opening").substr(1, 4)), 
                    "dd-MM-yyyy")).cast("int") + 1
                )).otherwise(months_between(
                    to_date(concat(lit("01-"), col("pd_closing").substr(-2, 2), lit("-"), col("pd_closing").substr(1, 4)), 
                    "dd-MM-yyyy"), 
                    to_date(concat(lit("01-"), col("pd_opening").substr(-2, 2), lit("-"), col("pd_opening").substr(1, 4)), 
                    "dd-MM-yyyy")).cast("int") + 1
                )
            )
        ).when(
            col("Value").isin("H2"),
            concat(col("currentvalue"), lit("-"), lit("06"))
        ).when(
            col("Value").isin('Q2', 'Q3', 'Q4'),
            concat(col("currentvalue"), lit("-"), lit("03"))
        ).otherwise(None)
    ).drop("pd_value", "pd_closing", "pd_opening")

    legacysddates_updated_df = legacysddates_updated_df2.union(legacysddates_updated_df3)

    min_currentvalue = legacysddates_updated_df.filter(col("p") == lit('')).selectExpr("MIN(currentvalue)").collect()[0][0]

    # Filter the DataFrame based on the condition
    legacysddates_curr_df = legacysddates_updated_df.filter(col("currentvalue") < min_currentvalue)

    legacysddates_updt1_df = legacysddates_updated_df.exceptAll(legacysddates_curr_df)

    legacysddates_num_df = legacysddates_updt1_df.withColumn("numericvalueofperiod",-expr("CAST(SUBSTR(LegacyPeriod, -2) AS INT)"))

    legacysddates_num_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates_num") 
    print("legacyddates write successful")
    return legacysddates_num_df

In [0]:
if extflag == 1 and extendedmonthdiff > 12:
    print("extflag == 1 and extendedmonthdiff > 12")
    print("more than 12 month cycle")
    extvalue = spark.createDataFrame([(PeriodValue,)], ["PeriodValue"]) \
    .withColumn("ExtValue", expr("substring(PeriodValue, 1, locate('-', PeriodValue)-1)")) \
    .select("ExtValue") \
    .collect()[0][0]
    print(extvalue)

    selected_data = periodicaldatesforextensionperiod_src_df.where(
        (col("entityid") == EntityID) &
        (col("Value") == extvalue) &
        (col("financialcycleid") == FiscalPeriodID)
        ).select("opening","closing","Year").collect()
    
    extopen = selected_data[0]["opening"]
    extclose = selected_data[0]["closing"]
    extendedcycleyear = selected_data[0]["Year"]

    print(extopen)
    print(extclose)
    print(extendedcycleyear)
   
    # --------Handle YTD calc for ext financial cycle ----------
    legacysddates_df = spark.read.parquet(adls_path+folder_path+"legacysddates", schema=legacysddates_schema)
    if extendedcycleyear == int(FinancialYear):

        print("extended year")
        
        get_extended_period_legacydates(legacysddates_df,periodicaldatesforextensionperiod_src_df,extvalue,FiscalPeriodID,FinancialYear,EntityID,extopen,extclose,extendedcycleyear)
        legacysddates_num_df = spark.read.parquet(adls_path+folder_path+"legacysddates_num", schema=legacysddates_schema)

        legacymatchperiodforlargerftp_df = legacysddates_num_df.alias("ld").join(periodicaldatesforextensionperiod_src_df.alias("mp"),(col("mp.closing") <= col("ld.currentvalue")) & (col("mp.opening") >= col("ld.openingperiodvalue")) & (col("ld.ftpperiod") >= col("mp.Order"))).where((col("financialcycleid") == FiscalPeriodID) & (col("Entityid") == EntityID)).select(
        "PeriodType",
        "closing",
        "LegacyPeriod",
        "numericvalueofperiod"
        ).distinct()

        legacymatchperiodforequalftp_df = legacysddates_num_df.alias("ld").join(periodicaldatesforextensionperiod_src_df.alias("mp"),(col("mp.closing") <= col("ld.currentvalue")) & (col("mp.opening") >= col("ld.openingperiodvalue")) & (col("ld.ftpperiod") == col("mp.Order"))).where((col("financialcycleid") == FiscalPeriodID) & (col("Entityid") == EntityID)).select(
        "PeriodType",
        "closing",
        "LegacyPeriod",
        "numericvalueofperiod"
        ).distinct()

        tballperiod_df = (
            trialbalance_dim_df.alias("tb")
            .join(legacymatchperiodforlargerftp_df.alias("ld"), (F.substring("tb.Period", 1, 6) == col("ld.closing")))
            .where(col("EntityID_fk") == EntityID)
            .select(
                col("Period"),
                F.substring("Period", 1, 6).alias("stdperiod"),
                col("ld.*"),*alldimcols
            ).distinct()
            .orderBy("PeriodType")
        )

        tballperiod_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tballperiod")
        tblegacymatchperiod_df = (
            trialbalance_dim_df.alias("tb")
            .join(legacymatchperiodforequalftp_df.alias("ld"), (F.substring("tb.Period", 1, 6) == col("ld.closing")))
            .where(col("EntityID_fk") == EntityID)
            .select(
                col("Period"),
                F.substring("Period", 1, 6).alias("stdperiod"),
                col("ld.*"),*alldimcols
            ).distinct()
            .orderBy("PeriodType")
        )

        tblegacymatchperiod_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tblegacymatchperiod")
        tblegacymatchperiod_df = spark.read.parquet(adls_path+folder_path+"tblegacymatchperiod", schema = tblegacymatchperiod_schema)
        
         # # ------table contain the number of months in ectended period----

        numperiod_df = legacysddates_num_df.alias("ld") \
        .join(tblegacymatchperiod_df.alias("md"), ["periodtype", "legacyperiod"]) \
        .withColumn(
        "val_numericvalueofperiod",
        when(expr("left(ld.Value, 1)") == 'Y', extendedmonthdiff)
        .when(expr("left(ld.Value, 1)") == '9', extendedmonthdiff - 3)
        .when(expr("left(ld.Value, 1)") == 'H', extendedmonthdiff - 6)
        .when(expr("left(ld.Value, 1)") == 'Q', extendedmonthdiff - 9)
        .otherwise(1)
        ) \
        .groupBy("periodtype", "LegacyPeriod") \
        .agg(max("val_numericvalueofperiod").alias("val_numericvalueofperiod")) \
        .select("periodtype","LegacyPeriod",col("val_numericvalueofperiod").alias("numericvalueofperiod"))
        

        tblegacymatchperiod_del1_df = tblegacymatchperiod_df.filter(~(
        (col("Period").substr(1, 6) == col("LegacyPeriod").substr(1, 6)) &
        (col("Period").substr(-2, 2).cast("int") != abs(col("numericvalueofperiod")))
        ))

        tblegacymatchperiod_del2_df = tblegacymatchperiod_del1_df.filter(~(
            (col("Period").substr(1, 6) != col("LegacyPeriod").substr(1, 6)) &
            (col("Period").substr(-2, 2).cast("int") < abs(col("NumericValueofPeriod")))
        ))

        tblegacymatchperiod_del3_df = tblegacymatchperiod_del2_df.alias("tb1").join(numperiod_df.alias("nm"), ["PeriodType", "LegacyPeriod"]) \
        .filter((col("tb1.Period").substr(1, 6) != col("tb1.LegacyPeriod").substr(1, 6)) & (col("tb1.Period").substr(-2, 2).cast("int") > abs(col("nm.numericvalueofperiod")))
        ).select("tb1.*")

        tblegacymatchperiod_updated_df = tblegacymatchperiod_del2_df.exceptAll(tblegacymatchperiod_del3_df)
        
        legacysddates_tmp_df = legacysddates_num_df.alias("ls").join(tblegacymatchperiod_updated_df.alias("mp"),((col("ls.LegacyPeriod") == col("mp.LegacyPeriod")) & (col("ls.periodtype") == col("mp.periodtype"))),"left") \
                            .filter(col("mp.LegacyPeriod").isNull()) \
                                        .select("ls.LegacyPeriod","ls.periodtype","ls.ftpperiod").withColumn("fromperiod",concat(F.year(F.add_months(F.to_date(F.concat(F.substring(col('LegacyPeriod'), 1, 4),F.lit('-'),F.substring(col('LegacyPeriod'), 5, 2),F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(LegacyPeriod, 2) AS INT)"))),  
                                F.substring(F.date_format(F.add_months(F.to_date(F.concat(F.substring(col('LegacyPeriod'), 1, 4),F.lit('-'),F.substring(col('LegacyPeriod'), 5, 2),
                                                F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(LegacyPeriod, 2) AS INT)")),'yyyy-MM-dd' ),6,2)))
        
        tballperiod_df = spark.read.parquet(adls_path+folder_path+"tballperiod", schema = tballperiod_schema )
        priorityseq_df1 = tballperiod_df.alias("tb").join(legacysddates_tmp_df.alias("ld"), ((col("tb.Period").substr(1, 6) <= col("ld.LegacyPeriod").substr(1, 6)) &
                                    (col("tb.Period").substr(1, 6) > col("fromperiod")) & (col("tb.PeriodType") == col("ld.periodtype"))),"inner").filter((col("tb.Period").substr(-2, 2) != col("ld.LegacyPeriod").substr(-2, 2)) & ((substring(col("Period"), -2, 2).cast("int")) <= abs(col("numericvalueofperiod")))).select("tb.*","ld.ftpperiod")
        
        window_spec = Window().partitionBy("stdperiod", "periodtype").orderBy(col("Period").desc())

        priorityseq_df = priorityseq_df1 \
                            .withColumn("priorityseq", row_number().over(window_spec)) \
                            .withColumn("order", lit(0)) \
                            .select(
                                col("Period"),
                                col("stdperiod"),
                                col("PeriodType"),
                                col("closing"),
                                col("legacyperiod"),
                                col("numericvalueofperiod"),
                                col("priorityseq"),
                                col("order"),
                                col("ftpperiod")
                            ) \
                            .orderBy("periodtype")                  
        
        #---populate order for ext cycle periods---
        mp_df = periodicaldatesforextensionperiod_src_df \
                                .withColumn("FromPeriod",substring(to_date(concat(lit("01-"),substring(col("period"),instr(col("period"), " ") + 1,when(instr(col("period"), "-") == 0,
                                length(col("period")) - instr(col("period"), " "))
                            .otherwise(instr(col("period"), "-") - instr(col("period"), " ") - 1)
                        ),lit("-"),substring(col("closing"), 1, 4)), "dd-MMM-yyyy"),6,2))
        ps_df = priorityseq_df.withColumn("fromperiod",
                    concat(F.year(F.add_months(F.to_date(F.concat(F.substring(col('Period'), 1, 4),F.lit('-'),F.substring(col('Period'), 5, 2),F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(Period, 2) AS INT)"))), 
                    F.substring(F.date_format(F.add_months(F.to_date(F.concat(F.substring(col('Period'), 1, 4),F.lit('-'),F.substring(col('Period'), 5, 2),
                    F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(Period, 2) AS INT)")),'yyyy-MM-dd' ),6,2)))

        cteorder_df = mp_df.alias("mp").join(ps_df.alias("ps"),(col("ps.closing") == col("mp.closing")) & (col("ps.ftpperiod") > col("mp.Order")) & (col("ps.fromperiod") == concat(col("ps.FromPeriod").substr(1,4),col("mp.FromPeriod")))).filter((col("financialcycleid") == FiscalPeriodID) & (col("entityid") == EntityID)) \
                        .groupBy("ps.Period","ps.PeriodType","mp.closing","ps.legacyperiod","priorityseq") \
                        .agg(F.max(col("mp.Order")).alias("Order"))
        # Perform the update by joining the original DataFrame with the CTE results
        priority_cte_joined_df = priorityseq_df.alias("ps").join(cteorder_df.alias("cte"), on="Period", how="left_outer").select("ps.*",col("cte.period").alias("cte_period"),col("cte.order").alias("cte_order"))

        priorityseq_updated_df = priority_cte_joined_df.withColumn(
            "order",
            when(col("ps.Period") == col("cte_Period"), col("cte_order")).otherwise(col("ps.order"))
        ).drop("cte_Period","cte_order")

        
        ld1_df = legacysddates_num_df.alias("ld").join(tblegacymatchperiod_updated_df.alias("tbm"),(col("ld.periodtype") == col("tbm.periodtype")) & 
                            (col("ld.legacyperiod") == col("tbm.legacyperiod")), "inner") \
                            .groupBy("tbm.periodtype", "ld.FinancialYear", "ld.LegacyPeriod", "ftpperiod") \
                            .agg(
                            max(col("tbm.Period")).alias("Period")) \
                            .withColumn("ftpperiod",when(col("ftpperiod") == 1,col("ftpperiod")).otherwise(col("ftpperiod")-1)) \
                            .withColumn("fromperiod",
                    concat(F.year(F.add_months(F.to_date(F.concat(F.substring(col('ld.LegacyPeriod'), 1, 4),F.lit('-'),F.substring(col('ld.LegacyPeriod'), 5, 2),F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(ld.LegacyPeriod, 2) AS INT)"))), 
                    F.substring(F.date_format(F.add_months(F.to_date(F.concat(F.substring(col('ld.LegacyPeriod'), 1, 4),F.lit('-'),F.substring(col('ld.LegacyPeriod'), 5, 2),
                    F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(ld.LegacyPeriod, 2) AS INT)")),'yyyy-MM-dd' ),6,2))) \
                    .select("tbm.periodtype","Period","ld.FinancialYear","ld.LegacyPeriod", "ftpperiod","fromperiod")
        
        ld_df = ld1_df.alias("ld1").join(periodicaldatesforextensionperiod_src_df.alias("pd"),(col("pd.closing") > col("ld1.fromperiod")) & (col("pd.closing") <= col("ld1.LegacyPeriod").substr(1, 6))
                        & (col("pd.Order") == col("ld1.ftpperiod")) & (col("pd.Year") == col("FinancialYear"))).filter((col("financialcycleid") == FiscalPeriodID) & (col("Entityid") == EntityID)) \
                        .withColumn("FromOpeningPeriod",concat(substring(col("FromPeriod"), 1, 4),substring(to_date(concat(lit("01-"),substring(
                                col("pd.Period"),
                                instr(col("pd.Period"), " ") + 1,  # Starting index
                                when(instr(col("pd.Period"), "-") == 0, 
                                    length(col("pd.Period")) - instr(col("pd.Period"), " ")
                                    )
                                .otherwise(instr(col("pd.Period"), "-") - instr(col("pd.Period"), " ") - 1)
                            ),lit("-"),substring(col("pd.closing"), 1, 4)),"dd-MMM-yyyy"),6,2))) \
                        .select("ld1.PeriodType","ld1.Period","ld1.FinancialYear","ld1.LegacyPeriod","ld1.ftpperiod"
                            ,"pd.Value","pd.closing","FromOpeningPeriod")
                        
        
        prioritylevel2temp = ld_df.withColumn("monthdiff",
        when(col("Value").isin("H1", "Q1", "Year", "9M"),
            (
        months_between(
            to_date(
            concat(
                lit("01-"), 
                col("closing").substr(-2, 2), 
                lit("-"), 
                col("closing").substr(1, 4)
            ), 
            "dd-MM-yyyy"
        ), 
        to_date(
            concat(
                lit("01-"), 
                col("FromOpeningPeriod").substr(-2, 2), 
                lit("-"), 
                col("FromOpeningPeriod").substr(1, 4)
            ), 
            "dd-MM-yyyy"
        )
        )+1
        ).cast("int"))
        .when(~(col("Value").isin("H1", "Q1", "Year", "9M")),
            when(col("Value").substr(1, 1) == "9", "9")
            .when(col("Value").substr(1, 1) == "H", "6")
            .when(col("Value").substr(1, 1) == "Q", "3")
            .when(col("Value").substr(1, 1) == "M", "1")
            .otherwise(None))
        ).select("periodtype","Period","legacyperiod","ftpperiod","closing","monthdiff").distinct()
           
        prioritylevel2temp_df = trialbalance_dim_df.alias("tb").join(prioritylevel2temp.alias("pt"),(col("tb.Period").substr(1,6) == col("pt.closing")) & ((abs(col("tb.Period").substr(-2, 2))) == col("pt.monthdiff")),"inner").where( col("EntityID_fk") == EntityID).select("tb.Period",
                                col("tb.Period").substr(1,6).alias("stdperiod"),
                                "periodtype",
                                col("tb.Period").substr(1,6).alias("closing"),
                                "legacyperiod",
                                col("ftpperiod").alias("numericvalueofperiod"),*alldimcols).distinct()
                
        
        window_spec = Window().partitionBy("stdperiod", "periodtype").orderBy(col("Period").desc())

        prioritylevel2_inst_df = prioritylevel2temp_df \
                            .withColumn("priorityseq", row_number().over(window_spec)) \
                            .withColumn("order", lit(0)) \
                            .select(
                                col("Period"),
                                col("stdperiod"),
                                col("PeriodType"),
                                col("closing"),
                                col("legacyperiod"),
                                col("numericvalueofperiod"),
                                col("priorityseq"),
                                col("order")
                            ) \
                            .orderBy("periodtype")
                
        priorityseq_final_df = priorityseq_updated_df.filter(col("priorityseq") == 1)

        prioritylevel2_final_df = prioritylevel2_inst_df.filter(col("priorityseq") == 1)


        ######################## For Next level priority check ##############################

        if_exists_df = (
        priorityseq_final_df
        .groupBy("PeriodType")
        .agg({"Order": "max"})
        .withColumnRenamed("max(Order)", "order")
        .exceptAll(
        priorityseq_final_df.alias("ps")
        .join(
        legacysddates_num_df.alias("ld"),
        (col("ps.PeriodType") == col("ld.PeriodType")) &
        (col("ps.LegacyPeriod") == col("ld.LegacyPeriod")) &
        (col("ps.Order") == col("ld.ftpperiod") - 1),
        "inner"
        )
        .select("ps.PeriodType", "ps.Order")
        ))
        if not if_exists_df.isEmpty():
            print("if exists not empty")
            # If the DataFrame is not empty, perform further operations
            ctePS_df = (
                priorityseq_final_df.groupBy("PeriodType","LegacyPeriod").agg(max(col("Order")).alias("Order")).select("PeriodType","LegacyPeriod","Order")
            )
            legacysddates_num1_df = legacysddates_num_df.alias("ld").join(ctePS_df.alias("ps"),
                    (col("ps.periodtype") == col("ld.PeriodType")) &
                    (col("ps.legacyperiod") == col("ld.legacyperiod")) & (col("ps.Order") != col("ld.ftpperiod")) ,"left_outer").select("ld.*",col("ps.Order").alias("ps_Order"),col("ps.legacyperiod").alias("ps_legacyperiod"),col("ps.periodtype").alias("ps_periodtype")) \
                    .withColumn("ftpperiod",when((col("ps_periodtype") == col("PeriodType")) &
                    (col("ps_legacyperiod") == col("legacyperiod")) & (col("ps_Order") != col("ftpperiod")), col("ps_Order") + 1).otherwise(col("ftpperiod"))).drop("ps_Order","ps_legacyperiod","ps_periodtype")
            legacysddates_num1_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates_num")
            
        else:
            print("else exists is empty")
            legacysddates_num1_df = legacysddates_num_df
            legacysddates_num1_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates_num")

        
        legacysddates_num1_df = spark.read.parquet(adls_path+folder_path+"legacysddates_num",schema=legacysddates_schema)
        deleteplev2_df = tblegacymatchperiod_updated_df.filter(col("Period") == col("LegacyPeriod")).select("LegacyPeriod","PeriodType")

        prioritylevel2_final_del_df = prioritylevel2_final_df.alias("pl").join(
            deleteplev2_df.alias("tbm"),
            (col("tbm.PeriodType") == col("pl.PeriodType")) &
            (col("tbm.LegacyPeriod") == col("pl.LegacyPeriod")),
            "left_anti"
        )
        
        prioritylevel2_final_del_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"prioritylevel2_ext")
       
        prioritylevel2_final_del_df = spark.read.parquet(adls_path+folder_path+"prioritylevel2_ext",schema=prioritylevel2_ext_schema)

        tblegacymatchperiod_del_df = (
        tblegacymatchperiod_updated_df.alias("tbm")
        .join(
        prioritylevel2_final_del_df.alias("pl"),
        (col("tbm.PeriodType") == col("pl.PeriodType")) &
        (col("tbm.LegacyPeriod") == col("pl.LegacyPeriod")),
        "left_anti"
        )
        )

        priorityseq_final_del_df = (
        priorityseq_final_df.alias("ps")
        .join(
        legacysddates_num1_df.alias("ld"),
        (col("ps.PeriodType") == col("ld.PeriodType")) &
        (col("ps.LegacyPeriod") == col("ld.LegacyPeriod")) &
        (col("ps.Order") != col("ld.ftpperiod") - 1),
        "left_anti"
        )
        )
        priorityseq_final_del_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"priorityseq")
        
        priorityseq_final_del_df = spark.read.parquet(adls_path+folder_path+"priorityseq",schema=priorityseq_schema)

        bspriority_tmp_df = (
                        legacysddates_num1_df.alias("ld").join(priorityseq_final_del_df.alias("ps"), (col("ld.PeriodType") == col("ps.PeriodType")) & (col("ld.LegacyPeriod") == col("ps.LegacyPeriod"))).select("ld.PeriodType","ld.legacyperiod","ld.currentvalue","ps.Period","ps.stdperiod") \
                        .union(
                            legacysddates_num1_df.alias("ld").join(prioritylevel2_final_del_df.alias("ps"), (col("ld.PeriodType") == col("ps.PeriodType")) & (col("ld.LegacyPeriod") == col("ps.LegacyPeriod"))).select("ld.PeriodType","ld.legacyperiod","ld.currentvalue","ps.Period","ps.stdperiod")))

        tblegacymatchperiod_del_grouped_df =  tblegacymatchperiod_del_df.withColumn("currentvalue",col("legacyperiod").substr(1,6)).select("PeriodType","legacyperiod","currentvalue","Period","stdperiod")


        bspriority_df = bspriority_tmp_df.union(tblegacymatchperiod_del_grouped_df) \
                        .groupBy("PeriodType", "legacyperiod", "currentvalue") \
                        .agg(F.max("Period").alias("Period"),
                            F.max("stdperiod").alias("stdperiod")).select("PeriodType","legacyperiod","currentvalue","Period","stdperiod")
                        
        bspriority_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"bspriority")
        bspriority_df = spark.read.parquet(adls_path+folder_path+"bspriority")
        legacysddates_updt_df = legacysddates_num1_df.alias("ld") \
        .join(
        bspriority_df.alias("ps"),
        (col("ld.PeriodType") == col("ps.PeriodType")) &
        (col("ld.LegacyPeriod") == col("ps.LegacyPeriod")),
        "left_outer").select("ld.*",col("ps.Period").alias("ps_Period"),col("ps.stdperiod").alias("ps_stdperiod"),col("ps.PeriodType").alias("ps_PeriodType"),col("ps.LegacyPeriod").alias("ps_LegacyPeriod")).withColumn("legacyperiod",when((col("PeriodType") == col("ps_PeriodType")) &
        (col("LegacyPeriod") == col("ps_LegacyPeriod")),col("ps_Period")).otherwise(col("legacyperiod"))) \
        .withColumn("currentvalue",when((col("PeriodType") == col("ps_PeriodType")) &
        (col("LegacyPeriod") == col("ps_LegacyPeriod")),col("ps_stdperiod")).otherwise(col("currentvalue"))).drop("ps_Period","ps_stdperiod","ps_PeriodType","ps_LegacyPeriod")
        
        legacysddates_updt_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates")
        legacysddates_updt_df = spark.read.parquet(adls_path+folder_path+"legacysddates",schema=legacysddates_schema)

        #### ---delete from legacyperiod -----

        tblegacymatchperiod_del_df1 = tblegacymatchperiod_del_df.filter(col("Period").substr(1, 6) != col("LegacyPeriod").substr(1, 6)).alias("a").join(tblegacymatchperiod_del_df.alias("b"),col("a.LegacyPeriod") == col("b.Period"),"left_anti")

        tblegacymatchperiod_final_del_df = tblegacymatchperiod_del_df.alias("tb").join(tblegacymatchperiod_del_df1.alias("del"),col("tb.LegacyPeriod") ==col("del.LegacyPeriod"),"left_anti")
        tblegacymatchperiod_final_del_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tblegacymatchperiod")
        tblegacymatchperiod_final_del_df = spark.read.parquet(adls_path+folder_path+"tblegacymatchperiod",schema=tblegacymatchperiod_schema)
     
        print("count of comp",countofcomp)
        if countofcomp == 0 or countofcomp ==1:
            print("count equal to 0 or 1")
            ytddates_df = tblegacymatchperiod_final_del_df.filter(col("Periodtype").like('%YTD') | col("Periodtype").like('%Year')).select("PeriodType",col("legacyperiod").substr(1,6).alias("CurrentValue"),col("Period").alias("FTPValue")) \
            .union(priorityseq_final_del_df.filter(col("Periodtype").like('%YTD') | col("Periodtype").like('%Year')).select("PeriodTYpe",col("legacyperiod").substr(1,6).alias("CurrentValue"),col("Period").alias("FTPValue"))) \
            .union(prioritylevel2_final_del_df.filter(col("Periodtype").like('%YTD') | col("Periodtype").like('%Year')).select("PeriodTYpe",col("legacyperiod").substr(1,6).alias("CurrentValue"),col("Period").alias("FTPValue"))).distinct()
            
            ytddates_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"ytddates")

        if countofcomp > 1:
            print("count greater than 1")
            ytddates_df1 = (
            legacysddates_updt_df.select("periodtype","pcurrentvalue","legacyperiod")
            .filter(~col("periodtype").isin("currentPeriod", "PreviousPeriod"))
            .exceptAll(
            legacysddates_updt_df.filter((col("legacyperiod").substr(1, 6) != col("pcurrentvalue"))).alias("a")
            .join(
                tblegacymatchperiod_final_del_df.alias("b"),    
                (col("a.legacyperiod") == col("b.Period")) &
                (col("a.periodtype") == col("b.PeriodType")),
                "left_anti"
            )
            .select("a.periodtype", "a.pcurrentvalue", "a.legacyperiod")
            )
            .union(
                priorityseq_final_del_df
                .select("PeriodType", col("legacyperiod").substr(1, 6).alias("legacyperiod"), "Period")
            )
            .union(
                prioritylevel2_final_del_df
                .select("PeriodType", col("legacyperiod").substr(1, 6).alias("legacyperiod"), "Period")
            )
            .union(
            tblegacymatchperiod_final_del_df
            .select("PeriodType", col("LegacyPeriod").substr(1, 6).alias("pcurrentvalue"), "Period")
            .exceptAll(
                legacysddates_updt_df
                .select("PeriodType", "pcurrentvalue", "legacyperiod"))
                .select("PeriodType", col("pcurrentvalue"), "Period")
            
        ).distinct()
            )
            print("ytddates table result final")
            ytddates_df = ytddates_df1.withColumnRenamed("pcurrentvalue","CurrentValue") \
            .withColumnRenamed("legacyperiod","FTPValue").distinct()
            
            ytddates_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"ytddates")
    else:
        print("not extended year")
        legacysddates_num_df = get_extended_period_legacydates(legacysddates_df,periodicaldatesforextensionperiod_src_df,extvalue,FiscalPeriodID,FinancialYear,EntityID,extopen,extclose,extendedcycleyear)    
        
        legacysddates_num_df = spark.read.parquet(adls_path+folder_path+"legacysddates_num",schema=legacysddates_schema)
        
        priority_rows_to_delete = priorityseq_df.join(legacysddates_num_df, (priorityseq_df["periodtype"] == legacysddates_num_df["periodtype"]) & \
                 (legacysddates_num_df["financialyear"] == extendedcycleyear), how="inner")
        
        priorityseq_del_df = priorityseq_df.join(priority_rows_to_delete, on=priorityseq_df.columns, how="left_anti")
        
        prioritylevel2_rows_to_delete = prioritylevel2_df.join(legacysddates_num_df, (prioritylevel2_df["periodtype"] == legacysddates_num_df["periodtype"]) & \
                        (legacysddates_num_df["financialyear"] == extendedcycleyear), how="inner").select(prioritylevel2_df["*"])
        
        prioritylevel2_del_df = prioritylevel2_df.join(prioritylevel2_rows_to_delete, on=prioritylevel2_df.columns, how="left_anti")
        
        
        ld_tmp_df = legacysddates_num_df.alias("ld").join(periodicaldatesforextensionperiod_src_df.alias("mp"),(col("mp.closing") <= col("ld.CurrentValue")) & (col("mp.opening") >= col("ld.openingperiodvalue")) & (col("ld.ftpperiod") >= col("mp.Order"))).where((col("financialcycleid") == FiscalPeriodID) & (col("Entityid") == EntityID)) \
                .select("PeriodType","closing","LegacyPeriod","numericvalueofperiod").distinct()

        tballperiod_df = trialbalance_dim_df.alias("tb").join(ld_tmp_df.alias("ld"),(col("tb.Period").substr(1, 6) == col("ld.closing"))).where(col("EntityID_fk") == EntityID) \
            .select("Period",col("tb.Period").substr(1, 6).alias("stdperiod"),"ld.*",*alldimcols).distinct() \
            .orderBy("PeriodType")

        ld1_tmp_df = legacysddates_num_df.alias("ld").join(periodicaldatesforextensionperiod_src_df.alias("mp"),(col("mp.closing") <= col("ld.CurrentValue")) & (col("mp.opening") >= col("ld.openingperiodvalue")) & (col("ld.ftpperiod") == col("mp.Order"))).where((col("financialcycleid") == FiscalPeriodID) & (col("Entityid") == EntityID)) \
        .select("PeriodType","closing","LegacyPeriod","numericvalueofperiod").distinct()

        tblegacymatchperiod_df = trialbalance_dim_df.alias("tb").join(ld1_tmp_df.alias("ld"),(col("Period").substr(1, 6) == col("ld.closing"))).where(col("EntityID_fk") == EntityID) \
            .select("Period",col("tb.Period").substr(1, 6).alias("stdperiod"),"ld.*",*alldimcols).distinct() \
            .orderBy("PeriodType")
        
        tblegacymatchperiod_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tblegacymatchperiod")
        tblegacymatchperiod_df = spark.read.parquet(adls_path+folder_path+"tblegacymatchperiod",schema=tblegacymatchperiod_schema)
        
        numperiod_df = legacysddates_num_df.alias("ld") \
            .join(tblegacymatchperiod_df.alias("md"), ["periodtype", "legacyperiod"]) \
            .withColumn(
            "val_numericvalueofperiod",
            when(expr("left(ld.Value, 1)") == 'Y', extendedmonthdiff)
            .when(expr("left(ld.Value, 1)") == '9', extendedmonthdiff - 3)
            .when(expr("left(ld.Value, 1)") == 'H', extendedmonthdiff - 6)
            .when(expr("left(ld.Value, 1)") == 'Q', extendedmonthdiff - 9)
            .otherwise(1)
            ) \
            .groupBy("periodtype", "LegacyPeriod") \
            .agg(max("val_numericvalueofperiod").alias("val_numericvalueofperiod")) \
            .select("periodtype","LegacyPeriod",col("val_numericvalueofperiod").alias("numericvalueofperiod"))
        
        
        tblegacymatchperiod_del1_df = tblegacymatchperiod_df.filter(~(
                    (col("Period").substr(1, 6) == col("LegacyPeriod").substr(1, 6)) &
                    (col("Period").substr(-2, 2).cast("int") != abs(col("numericvalueofperiod")))
                    ))
        
        tblegacymatchperiod_del2_df = tblegacymatchperiod_del1_df.filter(~(
                (col("Period").substr(1, 6) != col("LegacyPeriod").substr(1, 6)) &
                (col("Period").substr(-2, 2).cast("int") < abs(col("NumericValueofPeriod")))
            ))
        

        tblegacymatchperiod_del3_df = tblegacymatchperiod_del2_df.alias("tb1").join(numperiod_df.alias("nm"), ["PeriodType", "LegacyPeriod"]) \
            .filter((col("tb1.Period").substr(1, 6) != col("tb1.LegacyPeriod").substr(1, 6)) & (col("tb1.Period").substr(-2, 2).cast("int") > abs(col("nm.numericvalueofperiod")))
            ).select("tb1.*")
        
        tblegacymatchperiod_updated_df = tblegacymatchperiod_del2_df.exceptAll(tblegacymatchperiod_del3_df)
        
        legacysddates_tmp_df = legacysddates_num_df.alias("ls").join(tblegacymatchperiod_updated_df.alias("mp"),((col("ls.LegacyPeriod") == col("mp.LegacyPeriod")) & (col("ls.periodtype") == col("mp.periodtype"))),"left") \
                            .filter(col("mp.LegacyPeriod").isNull()) \
                                        .select("ls.LegacyPeriod","ls.periodtype","ls.ftpperiod").withColumn("fromperiod",concat(F.year(F.add_months(F.to_date(F.concat(F.substring(col('LegacyPeriod'), 1, 4),F.lit('-'),F.substring(col('LegacyPeriod'), 5, 2),F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(LegacyPeriod, 2) AS INT)"))),  
                                F.substring(F.date_format(F.add_months(F.to_date(F.concat(F.substring(col('LegacyPeriod'), 1, 4),F.lit('-'),F.substring(col('LegacyPeriod'), 5, 2),
                                                F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(LegacyPeriod, 2) AS INT)")),'yyyy-MM-dd' ),6,2)))
        

        priorityseq_df1 = tballperiod_df.alias("tb").join(legacysddates_tmp_df.alias("ld"), ((col("tb.Period").substr(1, 6) <= col("ld.LegacyPeriod").substr(1, 6)) &
                                    (col("tb.Period").substr(1, 6) > col("fromperiod")) & (col("tb.PeriodType") == col("ld.periodtype"))),"inner").filter((col("tb.Period").substr(-2, 2) != col("ld.LegacyPeriod").substr(-2, 2)) & ((substring(col("Period"), -2, 2).cast("int")) <= abs(col("numericvalueofperiod")))).select("tb.*","ld.ftpperiod")
        
        window_spec = Window().partitionBy("stdperiod", "periodtype").orderBy(col("Period").desc())
        
        priorityseq_df2 = priorityseq_df1 \
                            .withColumn("priorityseq", row_number().over(window_spec)) \
                            .withColumn("order", lit(0)) \
                            .select(
                                col("Period"),
                                col("stdperiod"),
                                col("PeriodType"),
                                col("closing"),
                                col("legacyperiod"),
                                col("numericvalueofperiod"),
                                col("priorityseq"),
                                col("order"),
                                col("ftpperiod")
                            ) \
                            .orderBy("periodtype")
        priorityseq_inst_df = priorityseq_del_df.union(priorityseq_df2)                    
        
        mp_df_expr = concat(lit("01-"),substring(col("period"),instr(col("period"), " ") + 1,when(instr(col("period"), "-") == 0,
                                length(col("period")) - instr(col("period"), " "))
                            .otherwise(instr(col("period"), "-") - instr(col("period"), " ") - 1)
                        ),lit("-"),substring(col("closing"), 1, 4))

        ps_df_expr = F.add_months(F.to_date(F.concat(F.substring(col('Period'), 1, 4),F.lit('-'),F.substring(col('Period'), 5, 2),F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(Period, 2) AS INT)"))

        mp_df = periodicaldatesforextensionperiod_src_df.withColumn("FromPeriod",substring(to_date(mp_df_expr, "dd-MMM-yyyy"),6,2))
                                

        ps_df = priorityseq_inst_df.withColumn("fromperiod",concat(F.year(ps_df_expr),F.substring(F.date_format(ps_df_expr,'yyyy-MM-dd' ),6,2)))
        
        cteorder_df = mp_df.alias("mp").join(ps_df.alias("ps"),(col("ps.closing") == col("mp.closing")) & (col("ps.ftpperiod") > col("mp.Order")) & (col("ps.fromperiod") == concat(col("ps.FromPeriod").substr(1,4),col("mp.FromPeriod")))).filter((col("financialcycleid") == FiscalPeriodID) & (col("entityid") == EntityID)) \
                        .groupBy("ps.Period","ps.PeriodType","mp.closing","ps.legacyperiod","priorityseq") \
                        .agg(F.max(col("mp.Order")).alias("Order"))

        priority_cte_joined_df = priorityseq_inst_df.alias("ps").join(cteorder_df.alias("cte"), on="Period", how="left_outer").select("ps.*",col("cte.period").alias("cte_period"),col("cte.order").alias("cte_order"))

        priorityseq_updated_df = priority_cte_joined_df.withColumn(
            "order",
            when(col("ps.Period") == col("cte_Period"), col("cte_order")).otherwise(col("ps.order"))
        ).drop("cte_Period","cte_order")
        
        ld1_df_expr = F.add_months(F.to_date(F.concat(F.substring(col('ld.LegacyPeriod'), 1, 4),F.lit('-'),F.substring(col('ld.LegacyPeriod'), 5, 2),F.lit('-01')),'yyyy-MM-dd'),-F.expr("CAST(RIGHT(ld.LegacyPeriod, 2) AS INT)"))
        
        ld1_df = legacysddates_num_df.alias("ld").join(tblegacymatchperiod_updated_df.alias("tbm"),(col("ld.periodtype") == col("tbm.periodtype")) & 
                    (col("ld.legacyperiod") == col("tbm.legacyperiod")), "inner") \
                    .groupBy("tbm.periodtype", "ld.FinancialYear", "ld.LegacyPeriod", "ftpperiod") \
                    .agg(
                    max(col("tbm.Period")).alias("Period")) \
                    .withColumn("ftpperiod",when(col("ftpperiod") == 1,col("ftpperiod")).otherwise(col("ftpperiod")-1)) \
                    .withColumn("fromperiod",
            concat(F.year(ld1_df_expr), 
            F.substring(F.date_format(ld1_df_expr,'yyyy-MM-dd' ),6,2))) \
            .select("tbm.periodtype","Period","ld.FinancialYear","ld.LegacyPeriod", "ftpperiod","fromperiod")
        
        ld_df = ld1_df.alias("ld1").join(periodicaldatesforextensionperiod_src_df.alias("pd"),(col("pd.closing") > col("ld1.fromperiod")) & (col("pd.closing") <= col("ld1.LegacyPeriod").substr(1, 6))
                        & (col("pd.Order") == col("ld1.ftpperiod")) & (col("pd.Year") == col("FinancialYear"))).filter((col("financialcycleid") == FiscalPeriodID) & (col("Entityid") == EntityID)) \
                        .withColumn("FromOpeningPeriod",concat(substring(col("FromPeriod"), 1, 4),substring(to_date(concat(lit("01-"),substring(
                                col("pd.Period"),
                                instr(col("pd.Period"), " ") + 1,  # Starting index
                                when(instr(col("pd.Period"), "-") == 0, 
                                    length(col("pd.Period")) - instr(col("pd.Period"), " ")
                                    )
                                .otherwise(instr(col("pd.Period"), "-") - instr(col("pd.Period"), " ") - 1)
                            ),lit("-"),substring(col("pd.closing"), 1, 4)),"dd-MMM-yyyy"),6,2))) \
                        .select("ld1.PeriodType","ld1.Period","ld1.FinancialYear","ld1.LegacyPeriod","ld1.ftpperiod"
                            ,"pd.Value","pd.closing","FromOpeningPeriod")
                        
        
        prioritylevel2temp = ld_df.withColumn("monthdiff",
        when(col("Value").isin("H1", "Q1", "Year", "9M"),
            (
        months_between(
            to_date(
            concat(
                lit("01-"), 
                col("closing").substr(-2, 2), 
                lit("-"), 
                col("closing").substr(1, 4)
            ), 
            "dd-MM-yyyy"
        ), 
        to_date(
            concat(
                lit("01-"), 
                col("FromOpeningPeriod").substr(-2, 2), 
                lit("-"), 
                col("FromOpeningPeriod").substr(1, 4)
            ), 
            "dd-MM-yyyy"
        )
        )+1
        ).cast("int"))
        .when(~(col("Value").isin("H1", "Q1", "Year", "9M")),
            when(col("Value").substr(1, 1) == "9", "9")
            .when(col("Value").substr(1, 1) == "H", "6")
            .when(col("Value").substr(1, 1) == "Q", "3")
            .when(col("Value").substr(1, 1) == "M", "1")
            .otherwise(None))
        ).select("periodtype","Period","legacyperiod","ftpperiod","closing","monthdiff").distinct()
       
        prioritylevel2temp_df = trialbalance_dim_df.alias("tb").join(prioritylevel2temp.alias("pt"),(col("tb.Period").substr(1,6) == col("pt.closing")) & ((abs(col("tb.Period").substr(-2, 2))) == col("pt.monthdiff")),"inner").where( col("EntityID_fk") == EntityID).select("tb.Period",
                                col("tb.Period").substr(1,6).alias("stdperiod"),
                                "periodtype",
                                col("tb.Period").substr(1,6).alias("closing"),
                                "legacyperiod",
                                col("ftpperiod").alias("numericvalueofperiod"),*alldimcols).distinct()
        

        window_spec = Window().partitionBy("stdperiod", "periodtype").orderBy(col("Period").desc())

        prioritylevel2_del_df2 = prioritylevel2temp_df \
                            .withColumn("priorityseq", row_number().over(window_spec)) \
                            .withColumn("order", lit(0)) \
                            .select(
                                col("Period"),
                                col("stdperiod"),
                                col("PeriodType"),
                                col("closing"),
                                col("legacyperiod"),
                                col("numericvalueofperiod"),
                                col("priorityseq"),
                                col("order")
                            ) \
                            .orderBy("periodtype")
        prioritylevel2_del_df = prioritylevel2_del_df.select(col("Period"),
                                col("stdperiod"),
                                col("PeriodType"),
                                col("closing"),
                                col("legacyperiod"),
                                col("numericvalueofperiod"),
                                col("priorityseq"),
                                col("order"))
        
        prioritylevel2_inst_df = prioritylevel2_del_df.union(prioritylevel2_del_df2)                    
        

        priorityseq_final_df = priorityseq_updated_df.filter(col("priorityseq") == 1)
        
        prioritylevel2_final_df = prioritylevel2_inst_df.filter(col("priorityseq") == 1)
        
        # # #     ######################## For Next level priority check ##############################
        
        if_exists_df = (
        priorityseq_final_df
        .groupBy("PeriodType")
        .agg({"Order": "max"})
        .withColumnRenamed("max(Order)", "order")
        .exceptAll(
        priorityseq_final_df.alias("ps")
        .join(
        legacysddates_num_df.alias("ld"),
        (col("ps.PeriodType") == col("ld.PeriodType")) &
        (col("ps.LegacyPeriod") == col("ld.LegacyPeriod")) &
        (col("ps.Order") == col("ld.ftpperiod") - 1),
        "inner"
        )
        .select("ps.PeriodType", "ps.Order")
        ))
        
        if not if_exists_df.isEmpty():
            print("inside if exists")
            
            ctePS_df = (
                priorityseq_final_df.groupBy("PeriodType","LegacyPeriod").agg(max(col("Order")).alias("Order")).select("PeriodType","LegacyPeriod","Order")
            )
            
            legacysddates_num1_df = legacysddates_num_df.alias("ld").join(ctePS_df.alias("ps"),
                    (col("ps.periodtype") == col("ld.PeriodType")) &
                    (col("ps.legacyperiod") == col("ld.legacyperiod")) & (col("ps.Order") != col("ld.ftpperiod")) ,"left_outer").select("ld.*",col("ps.Order").alias("ps_Order"),col("ps.legacyperiod").alias("ps_legacyperiod"),col("ps.periodtype").alias("ps_periodtype")) \
                    .withColumn("ftpperiod_new",when((col("ps_periodtype") == col("PeriodType")) &
                    (col("ps_legacyperiod") == col("legacyperiod")) & (col("ps_Order") != col("ftpperiod")), col("ps_Order") + 1).otherwise(col("ftpperiod"))).drop("ps_Order","ps_legacyperiod","ps_periodtype","ftpperiod")
            legacysddates_num1_df = legacysddates_num1_df.withColumnRenamed("ftpperiod_new","ftpperiod")       
            legacysddates_num1_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates_num")

        else:
            print("else not exists")
            legacysddates_num1_df = legacysddates_num_df
            legacysddates_num1_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates_num")
        
        legacysddates_num2_df = spark.read.parquet(adls_path+folder_path+"legacysddates_num",schema=legacysddates_schema)
        deleteplev2_df = tblegacymatchperiod_updated_df.filter(col("Period") == col("LegacyPeriod")).select("LegacyPeriod","PeriodType")

        prioritylevel2_final_del_df = prioritylevel2_final_df.alias("pl").join(
            deleteplev2_df.alias("tbm"),
            (col("tbm.PeriodType") == col("pl.PeriodType")) &
            (col("tbm.LegacyPeriod") == col("pl.LegacyPeriod")),
            "left_anti"
        )
        
        prioritylevel2_final_del_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"prioritylevel2_ext")
        prioritylevel2_final_del_df = spark.read.parquet(adls_path+folder_path+"prioritylevel2_ext", schema = prioritylevel2_ext_schema)

        tblegacymatchperiod_del_df = (
        tblegacymatchperiod_updated_df.alias("tbm")
        .join(
        prioritylevel2_final_del_df.alias("pl"),
        (col("tbm.PeriodType") == col("pl.PeriodType")) &
        (col("tbm.LegacyPeriod") == col("pl.LegacyPeriod")),
        "left_anti"
        )
        )

        priorityseq_final_del_df = (
        priorityseq_final_df.alias("ps")
        .join(
        legacysddates_num1_df.alias("ld"),
        (col("ps.PeriodType") == col("ld.PeriodType")) &
        (col("ps.LegacyPeriod") == col("ld.LegacyPeriod")) &
        (col("ps.order") != (col("ld.ftpperiod")- 1)),
        "left_anti"
        )
        )

        priorityseq_final_del_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"priorityseq")
        priorityseq_final_del_df = spark.read.parquet(adls_path+folder_path+"priorityseq", schema = priorityseq_schema)

        bspriority_tmp_df = (
                        legacysddates_num1_df.alias("ld").join(priorityseq_final_del_df.alias("ps"), (col("ld.PeriodType") == col("ps.PeriodType")) & (col("ld.LegacyPeriod") == col("ps.LegacyPeriod"))).select("ld.PeriodType","ld.legacyperiod","ld.currentvalue","ps.Period","ps.stdperiod") \
                        .union(
                            legacysddates_num1_df.alias("ld").join(prioritylevel2_final_del_df.alias("ps"), (col("ld.PeriodType") == col("ps.PeriodType")) & (col("ld.LegacyPeriod") == col("ps.LegacyPeriod"))).select("ld.PeriodType","ld.legacyperiod","ld.currentvalue","ps.Period","ps.stdperiod")))

        tblegacymatchperiod_del_grouped_df =  tblegacymatchperiod_del_df.withColumn("currentvalue",col("legacyperiod").substr(1,6)).select("PeriodType","legacyperiod","currentvalue","Period","stdperiod")


        bspriority_df = bspriority_tmp_df.union(tblegacymatchperiod_del_grouped_df) \
                        .groupBy("PeriodType", "legacyperiod", "currentvalue") \
                        .agg(F.max("Period").alias("Period"),
                            F.max("stdperiod").alias("stdperiod")).select("PeriodType","legacyperiod","currentvalue","Period","stdperiod")
                        
        bspriority_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"bspriority")
        bspriority_df = spark.read.parquet(adls_path+folder_path+"bspriority")

        legacysddates_updt_df = legacysddates_num1_df.alias("ld") \
        .join(
        bspriority_df.alias("ps"),
        (col("ld.PeriodType") == col("ps.PeriodType")) &
        (col("ld.LegacyPeriod") == col("ps.LegacyPeriod")),
        "left_outer").select("ld.*",col("ps.Period").alias("ps_Period"),col("ps.stdperiod").alias("ps_stdperiod"),col("ps.PeriodType").alias("ps_PeriodType"),col("ps.LegacyPeriod").alias("ps_LegacyPeriod")).withColumn("legacyperiod",when((col("PeriodType") == col("ps_PeriodType")) &
        (col("LegacyPeriod") == col("ps_LegacyPeriod")),col("ps_Period")).otherwise(col("legacyperiod"))) \
        .withColumn("currentvalue",when((col("PeriodType") == col("ps_PeriodType")) &
        (col("LegacyPeriod") == col("ps_LegacyPeriod")),col("ps_stdperiod")).otherwise(col("currentvalue"))).drop("ps_Period","ps_stdperiod","ps_PeriodType","ps_LegacyPeriod")


        legacysddates_updt_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates") 
        legacysddates_updt_df = spark.read.parquet(adls_path+folder_path+"legacysddates", schema = legacysddates_schema)

        ytddates_del_df = ytddates_df.alias("ys").join(legacysddates_updt_df.alias("ld"),(col("ys.periodtype")==col("ld.periodtype")) & (col("financialyear") == extendedcycleyear),"left_anti")

        # # #     #### ---delete from legacyperiod -----


        tblegacymatchperiod_del_df1 = tblegacymatchperiod_del_df.filter(col("Period").substr(1, 6) != col("LegacyPeriod").substr(1, 6)).alias("a").join(tblegacymatchperiod_del_df.alias("b"),col("a.LegacyPeriod") == col("b.Period"),"left_anti")

        tblegacymatchperiod_final_del_df = tblegacymatchperiod_del_df.alias("tb").join(tblegacymatchperiod_del_df1.alias("del"),col("tb.LegacyPeriod") ==col("del.LegacyPeriod"),"left_anti")
        tblegacymatchperiod_final_del_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tblegacymatchperiod")
        tblegacymatchperiod_final_del_df = spark.read.parquet(adls_path+folder_path+"tblegacymatchperiod",schema=tblegacymatchperiod_schema)

        print("count of comp",countofcomp)
        if countofcomp == 0 or countofcomp ==1:
            print("inside countodcomp 0 or 1")
            ytddates_df1 = tblegacymatchperiod_final_del_df.filter(col("Periodtype").like('%YTD') | col("Periodtype").like('%Year')).select("PeriodTYpe",col("legacyperiod").substr(1,6).alias("CurrentValue"),col("Period").alias("FTPValue")) \
            .union(priorityseq_final_del_df.alias("ps").join(legacysddates_updt_df.alias("ld"),col("ps.periodtype") == col("ld.periodtype")).filter((col("ld.Periodtype").like('%YTD') | col("ld.Periodtype").like('%Year')) & (col("financialyear") == extendedcycleyear)).select("ld.PeriodTYpe",col("ld.legacyperiod").substr(1,6).alias("CurrentValue"),col("Period").alias("FTPValue"))) \
            .union(prioritylevel2_final_del_df.alias("pl").join(legacysddates_updt_df.alias("ld"),col("pl.periodtype") == col("ld.periodtype")).filter((col("ld.Periodtype").like('%YTD') | col("ld.Periodtype").like('%Year')) & (col("financialyear") == extendedcycleyear)).select("ld.PeriodTYpe",col("pl.legacyperiod").substr(1,6).alias("CurrentValue"),col("Period").alias("FTPValue")))
            ytddates_df = ytddates_del_df.union(ytddates_df1).distinct()
            ytddates_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"ytddates") 
            

        if countofcomp > 1:
            print("count greater than 1")
            ytddates_df1 = (
            legacysddates_updt_df.select("periodtype","pcurrentvalue","legacyperiod")
            .filter(~(col("periodtype").isin("currentPeriod", "PreviousPeriod")) & (col("financialyear") == extendedcycleyear))
            .exceptAll(
            legacysddates_updt_df.filter((col("legacyperiod").substr(1, 6) != col("pcurrentvalue")) & (col("financialyear") == extendedcycleyear)).alias("a")
            .join(
                tblegacymatchperiod_final_del_df.alias("b"),    
                (col("a.legacyperiod") == col("b.Period")) &
                (col("a.periodtype") == col("b.PeriodType")),
                "left_anti"
            )
            .select("a.periodtype", "a.pcurrentvalue", "a.legacyperiod")
            )
            .union(
                priorityseq_final_del_df
                .select("PeriodType", col("legacyperiod").substr(1, 6).alias("legacyperiod"), "Period")
            )
            .union(
                prioritylevel2_final_del_df
                .select("PeriodType", col("legacyperiod").substr(1, 6).alias("legacyperiod"), "Period")
            )
            .union(
            tblegacymatchperiod_final_del_df
            .select("PeriodType", col("LegacyPeriod").substr(1, 6).alias("pcurrentvalue"), "Period")
            .exceptAll(
                legacysddates_updt_df.filter(col("financialyear") ==  extendedcycleyear)
                .select("PeriodType", "pcurrentvalue", "legacyperiod"))
                .select("PeriodType", col("pcurrentvalue"), "Period")
            
        )
            )
        print("ytddates table result final")
        ytddates_df1 = ytddates_df1.withColumnRenamed("pcurrentvalue","CurrentValue") \
        .withColumnRenamed("legacyperiod","FTPValue")
        ytddates_df = ytddates_del_df.union(ytddates_df1).distinct()
        ytddates_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"ytddates") 


In [0]:
legacysddates_df = spark.read.parquet(adls_path+folder_path+"legacysddates",schema=legacysddates_schema)
legacysddates_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates_final")

In [0]:
legacyadjustmentsdrilldown_src_uc_df = spark.read.table(catalog_schema_name+"."+"legacyadjustmentsdrilldown")
legacysddates_final_df = spark.read.parquet(adls_path+folder_path+"legacysddates_final",schema=legacysddates_schema)
for row in legacysddates_final_df.collect():
    ID = row.id
    LEGACYPERIOD = row.legacyperiod
    CatYear = row.financialyear
    result_category_df = categorydropdown_src_df.filter((col("categoryname") == CategorySDName) & (col("accountingstandardid") == AccountingStandardID) & (col("EntityId") == EntityID) & (col("Year") == CatYear)).select("categoryid")
    
    if result_category_df.count() > 0:
    # Get the first categoryid value from the DataFrame
        category_value = result_category_df.first()[0]
    else:
    # Handle the case where no matching rows were found
        category_value = 0
    legacysddates_df_tmp = legacysddates_final_df.filter(col("id") == row.id).withColumn("categoryid",lit(category_value))

    legacysddates_union_df = legacysddates_union_df.union(legacysddates_df_tmp)

    row_values = (
        legacysddates_union_df.filter(col("id") == row.id)
        .select("legacyperiod", "categoryid", "castingperiod")
        .first()
    )
    print (row_values)
    LCurrperiod, LCategoryID, CastingPeriod = row_values

    legacyadjustmentsdrilldown_df = get_legacy_adjustment_by_category_drilldown(legacyadjustments_dim_df,legacyadjustmentsdrilldown_union_df,AccountingStandardID,EntityID,LCurrperiod,LCategoryID)
    legacyadjustmentsdrilldown_union_df = legacyadjustmentsdrilldown_union_df.union(legacyadjustmentsdrilldown_df).distinct()

    new_castingsdadjustment_df = castingadjustment_dim_df.filter(
                (col("categoryid") == int(LCategoryID)) &
                (col("fk_entityid") == EntityID) &
                (col("fk_accountingstandardid") == AccountingStandardID) &
                (col("roundoff") == Roundoff) &
                (col("amountsin") == AmountsIn) &
                (col("periodvalue") == CastingPeriod) &
                (col("roundtypelevel") == RoundTypeLevel)).select("CastingAdjustmentsID","Fk_AccountingStandardID","Fk_ValidationID","PeriodValue","FinancialYear","Fk_EntityID","EntityName","JournalNumber","JournalDate","UserGLCode","UserGLDescription","StandardGLCode","StandardGLDescription","DebitCredit","Fk_CurrencyID","Amount","JournalType","Narration","Fk_BusinessUnitID","CategoryID","Roundoff","AmountsIn","ReportType","CreatedDate","CreatedBy","ModifiedDate","ModifiedBy","IsUserGLCode","RoundTypeLevel",*alldimcols)
    castingsdadjustment_df = castingsdadjustment_df.union(new_castingsdadjustment_df).distinct()


In [0]:
legacysddates_union_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacysddates_final")
legacyadjustmentsdrilldown_union_df.write.mode("overwrite").saveAsTable("dtdf_assu_faas_uc.frh.legacyadjustmentsdrilldown")
castingsdadjustment_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"castingsdadjustment")

In [0]:
castingsdadjustment_df = spark.read.parquet(adls_path+folder_path+"castingsdadjustment",schema=castingsdadjustment_schema)
legacysddates_final_df = spark.read.parquet(adls_path+folder_path+"legacysddates_final",schema=legacysddates_schema)

In [0]:
# delete duplicate entries for Legacy adj and casting adj
legacyadjustmentsdrilldown_df = spark.read.table(catalog_schema_name+"."+"legacyadjustmentsdrilldown")
window_spec2 = Window.partitionBy("LegacyAdjustmentID").orderBy("LegacyAdjustmentID")

legacyadjustmentsdrilldown_df = legacyadjustmentsdrilldown_df.withColumn("row_number", row_number().over(window_spec2))
legacyadjustmentsdrilldown_df = legacyadjustmentsdrilldown_df.filter(legacyadjustmentsdrilldown_df.row_number == 1).drop("row_number")


ladjjfinal_df = legacyadjustmentsdrilldown_df.withColumn("convertedperiod", F.lit('')).withColumn("periodtype", F.lit(''))

ladjjfinal_updated1_df =  ladjjfinal_df.alias("la").join(legacysddates_union_df.alias("ld"),
    col("la.categoryid") == col("ld.categoryid"),
    "left_outer"
).select("la.*",col("ld.periodtype").alias("ld_PeriodType"),col("ld.currentvalue").alias("ld_currentvalue"),col("ld.categoryid").alias("ld_categoryid")) \


ladjjfinal_updated_df = ladjjfinal_updated1_df.withColumn("convertedperiod",when(col("categoryid")==col("ld_categoryid"),col("ld_currentvalue")).otherwise(col("ld_currentvalue"))) \
.withColumn("PeriodType",when(col("categoryid")==col("ld_categoryid"),col("ld_PeriodType")).otherwise(col("la.periodtype")))\
.drop("ld_PeriodType","ld_currentvalue","ld_categoryid")

window_spec3 = Window.partitionBy("CastingAdjustmentsID").orderBy("CastingAdjustmentsID")

castingsdadjustment_df = castingsdadjustment_df.withColumn("row_number", row_number().over(window_spec3))
castingsdadjustment_df = castingsdadjustment_df.filter(castingsdadjustment_df.row_number == 1).drop("row_number")

# ---Date range data fetch------

legacydateslite_df = (
    legacysddates_final_df
    .select("LegacyPeriod", "PeriodType")
    .distinct()
)

In [0]:
tempcategory_df = categorydropdown_src_df.filter((col("categoryname") == CategorySDName) & (col("EntityId") == EntityID) & (col("AccountingStandardId") == AccountingStandardID)).select("categoryid")

allperiodftpscoa_df = spark.read.parquet(adls_path+folder_path+"allperiodftpscoa", schema= allperiodftpscoa_schema)
ytddates_df = spark.read.parquet(adls_path+folder_path+"ytddates", schema = ytddates_schema)

#  --standard Period data fetch--

tbf_df = (
trialbalance_dim_df.alias("tb")
.join(legacydateslite_df.alias("ld"),( col("tb.Period") == col("ld.legacyperiod")) & (col("EntityID_fk") == EntityID) & (col("AccountingStdID_fk") == AccountingStandardID))
.select(col("ld.PeriodType").alias("PeriodType"), F.substring("tb.Period", 1, 6).alias("Period"), "userglcode", "closingbalance", col("CurrencyID_fk").alias("currency"),*alldimcols)
)

saf_df = (
    standaloneadjustments_dim_df.alias("sd").join(legacydateslite_df.alias("ld"), col("sd.Period") == "ld.legacyperiod") \
    .join(tempcategory_df.alias("tc"), col("tc.categoryid") == col("sd.CategoryID_fk"))
    .filter((col("EntityID_fk") == EntityID) &
            (col("AccountingStdID_fk") == AccountingStandardID) &
            (col("IsUserGLCodeAdjustment")  == True))
    .select(col("ld.PeriodType").alias("PeriodType"), F.substring("sd.Period", 1, 6).alias("Period"), "userglcode", col("amount").alias("closingbalance"), col("CurrencyID_fk").alias("currency"),*alldimcols)
)

ljf_df = (
    ladjjfinal_updated_df
    .filter(ladjjfinal_updated_df["ismappedglcode"] == True)
    .select("PeriodType", F.substring("convertedperiod", 1, 6).alias("Period"), "userglcode",col("amount").alias("closingbalance"), col("CurrencyID_Fk").alias("currency"),*alldimcols)
)

# Union of the three DataFrames
unioned_standard_df = tbf_df.union(saf_df).union(ljf_df).distinct()

# Group by PeriodType, Period, userglcode, currency, and calculate the sum of closingbalance
tbfinal_df = (
    unioned_standard_df
    .groupBy("PeriodType", "Period", "userglcode", "currency",*alldimcols)
    .agg(F.sum("closingbalance").alias("closingbalance"))
    .select("PeriodType","Period","userglcode","currency","closingbalance",*alldimcols).distinct()
)
tbfinal_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tbfinal")
# # # -----YTD -------------


tbytd_df = (
    trialbalance_dim_df.alias("tb")
    .join(ytddates_df.alias("yt"), col("tb.Period") == col("yt.FTPValue"))
    .filter(( col("EntityID_fk") == EntityID) &
            ( col("AccountingStdID_fk") == AccountingStandardID))
    .groupBy("PeriodType", F.substring("Period", 1, 6).alias("Period"), "userglcode", col("CurrencyID_fk").alias("currency"),*alldimcols)
    .agg(F.sum("closingbalance").alias("closingbalance"))
    .select("PeriodType", "Period", "userglcode", "currency", "closingbalance",*alldimcols)
)

saytd_df = standaloneadjustments_dim_df.alias("sd") \
    .join(ytddates_df.alias("yt"), col("sd.Period") == col("yt.FTPValue")) \
    .join(tempcategory_df.alias("tc"), col("tc.categoryid") == col("sd.CategoryID_fk")) \
    .filter((col("EntityID_fk") == EntityID) &
            (col("AccountingStdID_fk") == AccountingStandardID) &
            (col("IsUserGLCodeAdjustment")  == True)) \
    .groupBy("PeriodType", F.substring("Period", 1, 6).alias("Period"), "userglcode", col("CurrencyID_fk").alias("currency"),*alldimcols) \
    .agg(F.sum("amount").alias("closingbalance")) \
    .select("PeriodType", "Period", "userglcode", "currency", "closingbalance",*alldimcols)


ljytd_df = (
    ladjjfinal_updated_df
    .filter(ladjjfinal_updated_df["ismappedglcode"] == True)
    .groupBy("PeriodType", F.substring("convertedperiod", 1, 6).alias("Period"), "userglcode",col("CurrencyID_Fk").alias("currency"),*alldimcols)
    .agg(F.sum("amount").alias("closingbalance"))
    .select("PeriodType", "Period", "userglcode", "currency", "closingbalance",*alldimcols)
)

# Union of the three DataFrames
unioned_ytd_df = tbytd_df.union(saytd_df).union(ljytd_df).distinct()

# Group by PeriodType, Period, userglcode, currency, and calculate the sum of closingbalance
tbfinalytd_df = (
    unioned_ytd_df
    .groupBy("PeriodType", "Period", "userglcode", "currency",*alldimcols)
    .agg(F.sum("closingbalance").alias("closingbalance")).distinct()
)
tbfinalytd_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tbfinalytd")
#     #----select bulk upload disclsore data  if category not there else categorywise data-------------

disclosures_df = standaloneadjustments_dim_df.alias("tg").join(legacydateslite_df.alias("ld"),col("tg.Period") == col("ld.LegacyPeriod")) \
    .join(allperiodftpscoa_df.alias("sc"),(col("tg.Member") == col("sc.Member")) & (col("tg.AccountSubType") == (col("sc.AccountSubType")))) \
    .filter((col("EntityID_fk") == EntityID) & (col("AccountingStdID_fk") == AccountingStandardID)).drop("sc.column27")\
.select(
    F.col("PeriodType"),
    F.substring(F.col("Period"), 1, 6).alias("Period"),
    F.col("EntityID_fk").alias("entityid"),
    F.col("sc.ultimateparent").alias("ultimateparent"),
    F.col("CategoryID_fk").alias("categoryid"),
    F.col("tg.member").alias("member"),
    F.col("accounttype"),
    F.col("amount"),
    F.col("CurrencyID_fk").alias("currencyid"),
    F.lit("").alias("column27"),
    F.col("tg.AccountSubType").alias("AccountSubType"),
    *alldimcols
).distinct()
disclosures_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"disclosures")

# # # ---YTD disclsoures---
disclosuresytd_df = standaloneadjustments_dim_df.alias("tg").join(allperiodftpscoa_df.alias("sc"),(col("tg.Member") == col("sc.Member")) & (col("tg.AccountSubType") == col("sc.AccountSubType"))) \
.join(ytddates_df.alias("yt"), col("tg.Period") == col("yt.FTPValue")).filter((col("EntityID_fk") == EntityID) & (col("AccountingStdID_fk") == AccountingStandardID)).drop("sc.column27") \
.groupBy(
    "PeriodType",
    "Period",
    "EntityID_fk",
    "sc.ultimateparent",
    "CategoryID_fk",
    "tg.member",
    "accounttype",
    "CurrencyID_fk",
    "tg.AccountSubType",
    *alldimcols
).agg(F.sum(col("amount")).alias("amount")).select(
    "PeriodType",
    F.substring("Period", 1, 6).alias("Period"),
    col("EntityID_fk").alias("entityid"),
    col("sc.ultimateparent").alias("ultimateparent"),
    col("CategoryID_fk").alias("categoryid"),
    col("tg.member").alias("member"),
    "accounttype",
    "amount",
    col("CurrencyID_fk").alias("currencyid"),
    F.lit("").alias("column27"),
    col("tg.AccountSubType").alias("AccountSubType"),
    *alldimcols
).distinct()

disclosuresytd_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"disclosuresytd")

In [0]:
tbfinal_df = spark.read.parquet(adls_path+folder_path+"tbfinal",schema=tbfinal_schema)
tbfinalytd_df = spark.read.parquet(adls_path+folder_path+"tbfinalytd",schema=tbfinalytd_schema)
disclosures_df = spark.read.parquet(adls_path+folder_path+"disclosures",schema=disclosures_schema)
disclosuresytd_df = spark.read.parquet(adls_path+folder_path+"disclosuresytd", schema=disclosuresytd_schema)

# -----update all ftp child member of input file to ftp---------

updatechildftp_tmp_df = disclosures_df.alias("d").join(allperiodftpscoa_df.alias("sc"), (col("d.member") == col("sc.member")) & (col("d.AccountSubType") == col("sc.AccountSubType"))).where(col("sc.column27") == "FTP").select("sc.member", "sc.path")

updatechildftp_df = (
    updatechildftp_tmp_df.alias("up").join(allperiodftpscoa_df.alias("sc"), F.expr("instr(sc.Path, up.Path) > 0"))
    .select("sc.member", "sc.Path", F.lit("FTP").alias("column27"))
    
)

disclosures_upt_df = disclosures_df.alias("d").join(updatechildftp_df.alias("sc"), col("d.member") == col("sc.member"), "left_outer").select("d.*",col("sc.column27").alias("sc_column27"),col("sc.member").alias("sc_member")) \
.withColumn("column27",when(col("sc_member") == col("d.member"),coalesce(col("sc_column27"), lit(""))).otherwise(col("d.column27"))).drop("sc_column27","sc_member")

# Left outer join and update #disclosuresytd DataFrame
disclosuresytd_upt_df = disclosuresytd_df.alias("d").join(updatechildftp_df.alias("sc"),col("d.member") == col("sc.member"), "left_outer").select("d.*",col("sc.column27").alias("sc_column27"),col("sc.member").alias("sc_member"))\
.withColumn("column27",when(col("sc_member") == col("d.member"),coalesce(col("sc_column27"),lit(""))).otherwise(col("d.column27"))).drop("sc_column27","sc_member")


In [0]:
# #     # ----drilldown-------
tbdd_df = trialbalance_dim_df.alias("tb").join(legacydateslite_df.alias("ld"),(col("tb.Period") == col("ld.LegacyPeriod")),"inner").filter((col("EntityID_fk") == EntityID) & (col("AccountingStdID_fk") == AccountingStandardID)).select(col("ld.PeriodType"),
    F.substring(col("Period"), 1, 6).alias("Period"),
    col("tb.userglcode"),
    col("tb.usergldescription"),
    col("closingbalance"),
    col("CurrencyID_fk").alias("currency"),
    lit("TB").alias("source"),
    *alldimcols)


sddd_df = standaloneadjustments_dim_df.alias("sd").join(legacydateslite_df.alias("ld"),col("sd.Period") == col("ld.LegacyPeriod")).join(tempcategory_df.alias("tc"), col("tc.categoryid") == col("sd.CategoryID_fk")).filter((col("EntityID_fk") == EntityID) & (col("AccountingStdID_fk") == AccountingStandardID) & (col("IsUserGLCodeAdjustment") == True)
    ).select(
    col("PeriodType"),
F.substring(col("Period"), 1, 6).alias("Period"),
col("sd.userglcode"),
F.lit(None).alias("UserGLDescription"),
col("amount").alias("closingbalance"),
col("CurrencyID_fk").alias("currency"),
lit("Adjustments").alias("source"),
*alldimcols)


ljdd_df = ladjjfinal_updated_df.filter(col("ismappedglcode") == True).select(
col("PeriodType"),
F.substring(col("convertedperiod"), 1, 6).alias("Period"),
col("userglcode"),
F.lit(None).alias("UserGLDescription"),
col("amount").alias("closingbalance"),
col("CurrencyID_Fk").alias("currency"),
lit("Adjustments").alias("source"),
*alldimcols)


dd_union_df = tbdd_df.union(sddd_df).union(ljdd_df).distinct()

tbfinaldd_df = dd_union_df.groupBy("PeriodType", "Period", "userglcode", "usergldescription", "currency", "source",*alldimcols).agg(F.sum(col("closingbalance")).alias("closingbalance")).select("PeriodType","Period", "userglcode", "usergldescription", "currency","closingbalance","source",*alldimcols).distinct()

tbfinaldd_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tbfinaldd")
#     #----YTD------------

tbddytd_df = trialbalance_dim_df.alias("tb").join(ytddates_df.alias("yt"), col("tb.Period") == col("yt.FTPValue")).filter((col("EntityID_fk") == EntityID) & (col("AccountingStdID_fk") == AccountingStandardID)).select(
    col("PeriodType"),
    F.substring(col("tb.Period"), 1, 6).alias("Period"),
    col("userglcode"),
    col("usergldescription"),
    col("closingbalance"),
    col("CurrencyID_fk").alias("currency"),
    lit("TB").alias("source"),
    *alldimcols
)


sdddytd_df = standaloneadjustments_dim_df.alias("sd").join(ytddates_df.alias("yt"), col("sd.Period") == col("yt.FTPValue")).join(tempcategory_df.alias("tc"), col("tc.categoryid") == col("sd.CategoryID_fk")).filter(
    (col("EntityID_fk") == EntityID) & (col("AccountingStdID_fk") == AccountingStandardID) & (col("IsUserGLCodeAdjustment") == True)
).select(
    col("PeriodType"),
    F.substring(col("sd.Period"), 1, 6).alias("Period"),
    col("userglcode"),
    F.lit(None).alias("UserGLDescription"),
    col("amount").alias("closingbalance"),
    col("CurrencyID_fk").alias("currency"),
    lit("Adjustments").alias("source"),
    *alldimcols
)


ljddytd_df = ladjjfinal_updated_df.filter(col("ismappedglcode") == True).select(
    col("PeriodType"),
    F.substring(col("convertedperiod"), 1, 6).alias("Period"),
    col("userglcode"),
    F.lit(None).alias("UserGLDescription"),
    col("amount").alias("closingbalance"),
    col("CurrencyID_Fk").alias("currency"),
    lit("Adjustments").alias("source"),
    *alldimcols
)

tbfinalddytd_df = tbddytd_df.union(sdddytd_df).union(ljddytd_df).groupBy("PeriodType", "Period", "userglcode", "usergldescription", "currency", "source",*alldimcols).agg(F.sum(col("closingbalance")).alias("closingbalance")).select("PeriodType", "Period", "userglcode", "usergldescription", "currency","closingbalance","source",*alldimcols).distinct()

tbfinalddytd_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"tbfinalddytd")

In [0]:
tbfinaldd_df  = spark.read.parquet(adls_path+folder_path+"tbfinaldd",schema=tbfinaldd_schema)
tbfinalddytd_df  = spark.read.parquet(adls_path+folder_path+"tbfinalddytd",schema=tbfinalddytd_schema)

# ----------------------standalone FTP & YTD data processing--------------------
SCOAMappingTable_df  = spark.read.parquet(adls_path+folder_path+"SCOAMappingTable",schema=SCOAMappingTable_schema)

tbfinal_updated_df = tbfinal_df.alias('mb').join(
    legacysddates_final_df.alias('ld'),
    (col('mb.period') == col('ld.currentvalue')) & (col('mb.PeriodType') == col('ld.periodtype')),
    'left_outer'
).select("mb.*",col("ld.pcurrentvalue").alias("ld_pcurrentvalue"),col("ld.currentvalue").alias("ld_currentvalue"),col("ld.periodtype").alias("ld_periodtype")) \
.withColumn(
    'Period',when((col('period') == col('ld_currentvalue')) & (col('PeriodType') == col('ld_periodtype')),col('ld_pcurrentvalue')).otherwise(col("period"))
).drop("ld_currentvalue","ld_periodtype","ld_pcurrentvalue")

tbfinalytd_updated_df = tbfinalytd_df.alias('mb').join(
    ytddates_df.alias('yt'),
    (col('mb.period') == col('yt.ftpvalue').substr(1, 6)) & (col('mb.PeriodType') == col('yt.periodtype')),
    'left_outer'
).select("mb.*",col("yt.currentvalue").alias("yt_currentvalue"),col("yt.ftpvalue").alias("yt_ftpvalue"),col("yt.periodtype").alias("yt_periodtype")) \
    .withColumn(
    'Period',when((col('period') == col('yt_ftpvalue').substr(1, 6)) & (col('PeriodType') == col('yt_periodtype')),col('yt_currentvalue')).otherwise(col("period"))) \
    .drop("yt_currentvalue","yt_periodtype","yt_ftpvalue")

tbfinaldd_updated_df = tbfinaldd_df.alias('mb').join(
    legacysddates_final_df.alias('ld'),
    (col('mb.period') == col('ld.currentvalue')) & (col('mb.PeriodType') == col('ld.periodtype')),
    'left_outer'
).select("mb.*",col("ld.pcurrentvalue").alias("ld_pcurrentvalue"),col('ld.currentvalue').alias("ld_currentvalue"),col('ld.periodtype').alias("ld_periodtype")) \
    .withColumn(
    'Period',when((col('period') == col('ld_currentvalue')) & (col('PeriodType') == col('ld_periodtype')),col('ld_pcurrentvalue')).otherwise(col("period"))) \
.drop("ld_pcurrentvalue","ld_currentvalue","ld_periodtype")

tbfinalddytd_updated_df = tbfinalddytd_df.alias('mb').join(
    ytddates_df.alias('yt'),
    (col('mb.period') == col('yt.ftpvalue').substr(1, 6)) & (col('mb.PeriodType') == col('yt.periodtype')),
    'left_outer'
).select("mb.*",col("yt.currentvalue").alias("yt_currentvalue"),col('yt.ftpvalue').alias("yt_ftpvalue"),col('yt.periodtype').alias("yt_periodtype")) \
.withColumn(
    'Period',when((col('period') == col('yt_ftpvalue').substr(1, 6)) & (col('PeriodType') == col('yt_periodtype')),col('yt_currentvalue')).otherwise(col("period"))) \
.drop("yt_currentvalue","yt_ftpvalue","yt_periodtype")

disclosures_updated_df = disclosures_upt_df.alias('mb').join(
   legacysddates_final_df.alias('ld'),
    (col('mb.period') == col('ld.currentvalue')) & (col('mb.PeriodType') == col('ld.periodtype')),
    'left_outer'
).select("mb.*",col("ld.pcurrentvalue").alias("ld_pcurrentvalue"),col('ld.currentvalue').alias("ld_currentvalue"),col('ld.periodtype').alias("ld_periodtype")) \
    .withColumn(
    'Period',when((col('period') == col('ld_currentvalue')) & (col('PeriodType') == col('ld_periodtype')),col('ld_pcurrentvalue')).otherwise(col("period"))) \
.drop("ld_pcurrentvalue","ld_currentvalue","ld_periodtype")

disclosuresytd_updated_df = disclosuresytd_upt_df.alias('mb').join(
    ytddates_df.alias('yt'),
    (col('mb.period') == col('yt.ftpvalue').substr(1, 6)) & (col('mb.PeriodType') == col('yt.periodtype')),
    'left_outer'
).select("mb.*",col("yt.currentvalue").alias("yt_currentvalue"),col('yt.ftpvalue').alias("yt_ftpvalue"),col('yt.periodtype').alias("yt_periodtype"))\
    .withColumn(
    'Period',when((col('period') == col('yt_ftpvalue').substr(1, 6)) & (col('PeriodType') == col('yt_periodtype')),col('yt_currentvalue')).otherwise(col("period"))) \
.drop("yt_currentvalue","yt_ftpvalue","yt_periodtype")

legacydatesperiodtypelite_df = legacysddates_final_df.filter(
    (col('periodtype').like('%YTD')) | (col('periodtype').like('%Year'))
).select(
    'pcurrentvalue',
    'periodtype'
).distinct()

legacydatesperiodtypelite_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacydatesperiodtypelite")

legacydatesnotperiodtypelite_df = legacysddates_final_df.select("pcurrentvalue", "PeriodType") \
    .distinct() \
    .filter(~col("PeriodType").like("%YTD")) \
    .filter(~col("PeriodType").like("%Year"))

legacydatesnotperiodtypelite_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"legacydatesnotperiodtypelite")

# ----------YTD-------------

mapped_sdt_by_td_tbfinalytd_df = tbfinalytd_updated_df.alias("tb").join(SCOAMappingTable_df.alias("mt"),
    col("tb.UserGLCode") == col("mt.UserGLCode"),
    "inner"
).join(legacydatesperiodtypelite_df.alias("ld"),(col("tb.Period") == col("ld.pcurrentvalue")) & (col("tb.PeriodType") == col("ld.PeriodType")),
    "inner"
).filter(col("UltimateParent") == "IS_GROUP").select("tb.PeriodType","tb.Period","tb.UserGLCode","tb.Currency","ClosingBalance","CategoryID",col("mt.eyglcode").alias("Member"),lit("Y").alias("ytdtype"),*alldimcols) 

mapped_sdt_by_td_tbfinal_df = tbfinal_updated_df.alias("tb").join(
    SCOAMappingTable_df.alias("mt"),
    col("tb.UserGLCode") == col("mt.UserGLCode"),
    "inner"
).join(
    legacydatesperiodtypelite_df.alias("ld"),
    (col("tb.Period") == col("ld.pcurrentvalue")) &
    (col("tb.PeriodType") == col("ld.PeriodType")),
    "inner"
).filter(col("UltimateParent") == "BS_GROUP").select("tb.PeriodType", "tb.Period", "tb.UserGLCode", "tb.Currency", "ClosingBalance", "CategoryID",col("mt.eyglcode").alias("Member"),lit("Y").alias("ytdtype"),*alldimcols)
 
ytd_tbfinal_tbfinalytd = mapped_sdt_by_td_tbfinalytd_df.union(mapped_sdt_by_td_tbfinal_df)

ytd_tbfinal_tbfinalytd_grp = ytd_tbfinal_tbfinalytd.groupBy(
    "PeriodType", "Period", "UserGLCode", "Currency", "CategoryID", "member",*alldimcols
).agg(
    F.sum("ClosingBalance").alias("ClosingBalance"),
).select(
    "PeriodType", "Period", "UserGLCode", "Currency", "ClosingBalance", "CategoryID", "Member", lit("Y").alias("ytdtype"),*alldimcols
)

mapped_sdt_by_td_disclosuresytd_df = disclosuresytd_updated_df.filter(
    (~(col("UltimateParent").isin("BS_GROUP", "IS_GROUP"))) &
    (col("Column27") == "FTP")) \
.groupBy(
    "PeriodType", "Period", "currencyid", "CategoryID", "Member", "UltimateParent", "Column27",*alldimcols
).agg(
    F.sum("Amount").alias("ClosingBalance"),
).select(
    "PeriodType", "Period", F.lit(None).alias("UserGLCode"), col("currencyid").alias("Currency"), "ClosingBalance","CategoryID", "Member", lit("Y").alias("ytdtype"),*alldimcols
)

mapped_sdt_by_td_disclosures_df = disclosures_updated_df.alias("disc").join(
    legacydatesperiodtypelite_df.alias("ld"),
    (col("disc.Period") == col("ld.pcurrentvalue")) &
    (col("disc.PeriodType") == col("ld.PeriodType")),
    "inner"
).filter(
    (~col("UltimateParent").isin("BS_GROUP", "IS_GROUP")) &
    (col("Column27") != "FTP")).groupBy(
    "disc.PeriodType", "Period", "Currencyid", "CategoryID", "Member", "UltimateParent", "Column27",*alldimcols
).agg(
    F.sum("Amount").alias("ClosingBalance")
).select(
    "PeriodType", "Period", F.lit(None).alias("UserGLCode"),col("Currencyid").alias("Currency"), "ClosingBalance", "CategoryID", "Member", lit("Y").alias("ytdtype"),*alldimcols
)

# Union all the DataFrames
mappedsdtbytd_df = ytd_tbfinal_tbfinalytd_grp.union(mapped_sdt_by_td_disclosuresytd_df).union(mapped_sdt_by_td_disclosures_df).distinct()
mappedsdtbytd_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"mappedsdtbytd")

# # # # --------------FTP--------

mapped_sdt_by_ftp_tbfinal_df = tbfinal_updated_df.alias("tb").join(
    SCOAMappingTable_df.alias("mt"),
    col("tb.UserGLCode") == col("mt.UserGLCode"),
    "inner"
).join(
    legacydatesnotperiodtypelite_df.alias("ld"),
    (col("tb.Period") == col("ld.pcurrentvalue")) &
    (col("tb.PeriodType") == col("ld.PeriodType")),
    "inner"
).select(
        "tb.PeriodType",
        col("tb.Period").alias("Period"),
        col("tb.UserGLCode").alias("UserGLCode"),
        col("tb.Currency"),
        col("ClosingBalance"),
        col("mt.categoryid").alias("categoryid"),
        col("mt.eyglcode").alias("member"),
        lit('N').alias("ytdtype"),
        *alldimcols
    )


mapped_sdt_by_ftp_disclosure_df =  disclosures_updated_df.alias("d").join(legacydatesnotperiodtypelite_df.alias("ld"),(col("d.Period") == col("ld.pcurrentvalue")) & (col("d.PeriodType") == col("ld.PeriodType")),
        "inner"
    ).select(
        col("d.PeriodType").alias("PeriodType"),
        col("Period"),
        F.lit(None).alias("UserGLCode"),
        col("currencyid").alias("currency"),
        col("Amount").alias("ClosingBalance"),
        col("CategoryID"),
        col("Member"),
        lit('N').alias("ytdtype"),
        *alldimcols
    )
legacysddates_final_mapped_df = legacysddates_final_df.filter(~((col("PeriodType").like('%YTD')) | (col("PeriodType").like('%Year')))).select("LegacyPeriod", "PeriodType").distinct()
mapped_sdt_by_ftp_adj_df = standaloneadjustments_dim_df.alias("sd").join(
        legacysddates_final_mapped_df.alias("ld"),
        col("sd.Period") == col("ld.LegacyPeriod"),
        "inner"
    ).join(
        tempcategory_df.alias("tc"),
        col("sd.CategoryID_fk") == col("tc.CategoryID"),
        "inner"
    ).filter(
        (col("EntityID_fk") == EntityID) &
        (col("AccountingStdID_fk") == AccountingStandardID) &
        (col("IsUserGLCodeAdjustment") == True)
    ).select(
        col("ld.PeriodType"),
        col("Period").substr(1, 6).alias("Period"),
        col("sd.UserGLCode"),
        col("CurrencyID_fk").alias("CurrencyID"),
        col("Amount").alias("ClosingBalance"),
        col("tc.CategoryID"),
        col("Member"),
        lit('N').alias("ytdtype"),
        *alldimcols
    )
mapped_sdt_by_ftp_ladj_df = ladjjfinal_updated_df.filter(
        ladjjfinal_updated_df.IsMappedGLCode == True
    ).select(
        "PeriodType",
        col("ConvertedPeriod").substr(1, 6).alias("Period"),
        F.lit(None).alias("UserGLCode"),
        col("CurrencyID_Fk").alias("CurrencyID"),
        col("Amount").alias("ClosingBalance"),
        col("CategoryID"),
        col("Member"),
        lit('N').alias("ytdtype"),
        *alldimcols
    )

mappedsdtbftp_df = mapped_sdt_by_ftp_tbfinal_df.union(mapped_sdt_by_ftp_disclosure_df).union(mapped_sdt_by_ftp_adj_df).union(mapped_sdt_by_ftp_ladj_df).distinct()

mappedsdtbftp_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"mappedsdtbftp")

In [0]:
# ----------------drilldown YTD & FTP data------------------------- 

  #    -----------------------YTD------------
legacysddates_final_mapped1_df = legacysddates_final_df.filter((col("PeriodType").like('%YTD')) | (col("PeriodType").like('%Year'))).select("pcurrentvalue","PeriodType")
mapped_tbfinalddytd_dd_df = tbfinalddytd_updated_df.alias("tb").join(
    SCOAMappingTable_df.alias("mt"),
    col("tb.UserGLCode") == col("mt.UserGLCode"),
    "inner"
).join(
    legacysddates_final_mapped1_df.alias("ld"), \
    (col("tb.Period") == col("ld.PCurrentValue")) &
    (col("tb.PeriodType") == col("ld.PeriodType")),
    "inner"
).filter(col("UltimateParent") == "IS_GROUP").groupBy(
    "tb.PeriodType", "tb.Period", "tb.UserGLCode", "tb.UserGLDescription", "mt.eyglcode", "tb.Source",*alldimcols
).agg(
    F.sum("ClosingBalance").alias("ClosingBalance")
).select("tb.PeriodType",
                 col("tb.Period").alias("Period"),
                 col("userglcode").alias("UserGLCode"),
                 col("UserGLDescription").alias("UserGLDescription"),
                 "closingbalance",
                 col("mt.eyglcode").alias("member"),
                 "tb.Source",
                 lit('Y').alias("ytdtype"),
                 *alldimcols).distinct()
mapped_tbfinalddytd_dd_df = tbfinaldd_updated_df.alias("tb").join(
    SCOAMappingTable_df.alias("mt"),
    col("tb.UserGLCode") == col("mt.UserGLCode"),
    "inner"
).join(
    legacysddates_final_mapped1_df.alias("ld"), \
    (col("tb.Period") == col("ld.PCurrentValue"))&
    (col("tb.PeriodType") == col("ld.PeriodType")),
    "inner"
).filter(col("UltimateParent") == "BS_GROUP").groupBy(
    "tb.PeriodType", "tb.Period", "tb.UserGLCode", "tb.UserGLDescription", "mt.eyglcode", "tb.Source",*alldimcols
).agg(
    F.sum("ClosingBalance").alias("ClosingBalance")
).select("tb.PeriodType",
                 col("tb.Period").alias("Period"),
                 col("tb.userglcode").alias("UserGLCode"),
                 col("tb.UserGLDescription").alias("UserGLDescription"),
                 "closingbalance",
                 col("mt.eyglcode").alias("member"),
                 "tb.Source",
                 lit('Y').alias("ytdtype"),
                 *alldimcols).distinct()

mapped_disclosuresytd_dd_df = disclosuresytd_updated_df.filter(~(col("UltimateParent").isin("BS_GROUP", "IS_GROUP")) &  (col("Column27") == 'FTP')
).groupBy("PeriodType", "Period", "Member", "ultimateparent", "Column27",*alldimcols
).agg(F.sum("Amount").alias("ClosingBalance")
).select("PeriodType",
                 "Period",
                 F.lit(None).alias("UserGLCode"),
                 F.lit(None).alias("UserGLDescription"),
                 "closingbalance",
                 "member",
                 lit("Disclosures").alias("Source"),
                 lit('Y').alias("ytdtype"),
                 *alldimcols).distinct()
mapped_disclosures_dd_df = disclosures_updated_df.alias("disc").join(
    legacysddates_final_mapped1_df.alias("ld"),
    (col("disc.Period") == col("ld.pcurrentvalue")) &
    (col("disc.PeriodType") == col("ld.PeriodType")),
    "inner"
).filter((~(col("UltimateParent").isin("BS_GROUP", "IS_GROUP"))) & (col("Column27") != 'FTP')
).groupBy(
    "disc.PeriodType", "Period", "Member", "ultimateparent", "Column27",*alldimcols
).agg(
    F.sum("Amount").alias("ClosingBalance")
).select("disc.PeriodType",
                 "Period",
                  F.lit(None).alias("UserGLCode"),
                 F.lit(None).alias("UserGLDescription"),
                 "closingbalance",
                 "member",
                lit("Disclosures").alias("Source"),
                 lit('Y').alias("ytdtype"),
                 *alldimcols).distinct()

mappedtbddytd_df =  mapped_tbfinalddytd_dd_df.union(mapped_tbfinalddytd_dd_df).union(mapped_disclosuresytd_dd_df).union(mapped_disclosures_dd_df).distinct()
mappedtbddytd_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"mappedtbddytd")
# ------------FTP-------

mapped_tbfinaldd_dd_ftp_df = tbfinaldd_updated_df.alias("tb").join(SCOAMappingTable_df.alias("mt"),col("tb.UserGLCode") == col("mt.UserGLCode"),"inner").join( \
    legacydatesnotperiodtypelite_df.alias("ld"),(col("tb.Period") == col("ld.pcurrentvalue")) & (col("tb.PeriodType") == col("ld.PeriodType")),"inner") \
    .select("tb.PeriodType",
                 col("tb.Period").alias("Period"),
                 col("tb.UserGLCode").alias("UserGLCode"),
                 col("tb.UserGLDescription").alias("UserGLDescription"),
                 "closingbalance",
                 col("mt.eyglcode").alias("member"),
                 "tb.Source",
                 lit("N").alias("ytdtype"),
                 *alldimcols).distinct()
mapped_disclosures_dd_ftp_df = disclosures_updated_df.alias("d").join(legacydatesnotperiodtypelite_df.alias("ld"),(col("d.Period") == col("ld.pcurrentvalue")) & (col("d.PeriodType") == col("ld.PeriodType")),
    "inner").select("d.PeriodType",
                    "Period",
                     F.lit(None).alias("UserGLCode"),
                     F.lit(None).alias("UserGLDescription"),
                    col("amount").alias("closingbalance"),
                    "member",
                    lit("Disclosures").alias("Source"),
                    lit("N").alias("ytdtype"),
                    *alldimcols).distinct()
legacysddates_final_mapped2_df = legacysddates_final_df.filter(~((col("PeriodType").like('%YTD')) | (col("PeriodType").like('%Year')))).select("LegacyPeriod", "PeriodType").distinct()
mapped_adj_dd_ftp_df = standaloneadjustments_dim_df.alias("sd").join(
        legacysddates_final_mapped2_df.alias("ld"),
        col("ld.LegacyPeriod") == col("sd.Period"),
        "inner"
    ).join(
        tempcategory_df.alias("tc"),
        col("sd.CategoryID_fk") == col("tc.CategoryID"),
        "inner"
    ).filter(
        (col("EntityID_fk") == EntityID) &
        (col("AccountingStdID_fk") == AccountingStandardID) &
        (col("IsUserGLCodeAdjustment") == True)
    ).select(
        "PeriodType",
        col("Period").substr(1, 6).alias("Period"),
        col("UserGLCode"),
        F.lit(None).alias("UserGLDescription"),
        col("Amount").alias("ClosingBalance"),
        col("Member"),
        lit('Adjustments').alias("Source"),
        lit('N').alias("ytdtype"),
        *alldimcols 
    ).distinct()
mapped_ldj_dd_ftp_df = ladjjfinal_updated_df.filter(
        col("IsMappedGLCode") == True
    ).select(
        "PeriodType",
        col("ConvertedPeriod").substr(1, 6).alias("Period"),
        F.lit(None).alias("UserGLCode"),
        F.lit(None).alias("UserGLDescription"),
        col("Amount").alias("ClosingBalance"),
        col("Member"),
        lit('Adjustments').alias("Source"),
        lit('N').alias("ytdtype"),
        *alldimcols
    ).distinct()
mappedtbddftp_df = mapped_tbfinaldd_dd_ftp_df.union(mapped_disclosures_dd_ftp_df).union(mapped_adj_dd_ftp_df).union(mapped_ldj_dd_ftp_df).distinct()

mappedtbddftp_df.coalesce(1).write.format("parquet").mode("overwrite").save(adls_path+folder_path+"mappedtbddftp")

In [0]:
mappedsdtbytd_df = spark.read.parquet(adls_path+folder_path+"mappedsdtbytd",schema=mappedsdtbytd_schema)
mappedsdtbftp_df = spark.read.parquet(adls_path+folder_path+"mappedsdtbftp",schema=mappedsdtbftp_schema)
mappedtbddytd_df = spark.read.parquet(adls_path+folder_path+"mappedtbddytd",schema=mappedtbddytd_schema)
mappedtbddftp_df = spark.read.parquet(adls_path+folder_path+"mappedtbddftp",schema=mappedtbddftp_schema)

if PeriodValueInYHQM == 0:
    mappedsdtbftp_df = mappedsdtbftp_df.withColumn("periodtype", expr("REPLACE(periodtype, '_YTD', '')"))

    mappedsdtbytd_df = mappedsdtbytd_df.withColumn("periodtype", expr("REPLACE(periodtype, '_YTD', '')"))

    mappedtbddftp_df = mappedtbddftp_df.withColumn("periodtype", expr("REPLACE(periodtype, '_YTD', '')"))

    mappedtbddytd_df = mappedtbddytd_df.withColumn("periodtype", expr("REPLACE(periodtype, '_YTD', '')"))

    legacysddates_final_df = legacysddates_final_df.withColumn("periodtype", expr("REPLACE(periodtype, '_YTD', '')"))


In [0]:
# ------------IS Period having prev period data--------
legacysddates_final_nodata_df = legacysddates_final_df.filter(legacysddates_final_df["pcurrentvalue"] != legacysddates_final_df["currentvalue"]).select( "periodtype" ,"legacyperiod" )
is_no_data_df = legacysddates_final_nodata_df.alias("lg") \
    .join(ytddates_df.alias("yt"), (col("yt.FTPValue") == col("lg.legacyperiod")) & (col("yt.PeriodType") == col("lg.periodtype")), "left_outer") \
    .filter(col("yt.PeriodType").isNull()) \
    .select("lg.periodtype","lg.legacyperiod")

mappedsdtbftp_del_tmp_df = mappedsdtbftp_df.alias("mf").join(SCOAMappingTable_df.alias("sc"),(col("mf.member") == col("sc.EYGLCode"))) \
    .join(is_no_data_df.alias("nd"), col("mf.PeriodType") == col("nd.periodtype"),"inner") \
    .filter(col("sc.UltimateParent") != 'IS_GROUP') \
    .select("mf.*")
mappedsdtbftp_deleted_df = mappedsdtbftp_df.exceptAll(mappedsdtbftp_del_tmp_df)

mappedtbddftp_del_tmp_df = mappedtbddftp_df.alias("mf").join(SCOAMappingTable_df.alias("sc"),(col("mf.member") == col("sc.EYGLCode"))) \
     .join(is_no_data_df.alias("nd"), col("mf.PeriodType") == col("nd.periodtype"),"inner") \
    .filter(col("sc.UltimateParent") != 'IS_GROUP') \
    .select("mf.*")
mappedtbddftp_deleted_df = mappedtbddftp_df.exceptAll(mappedtbddftp_del_tmp_df)


In [0]:
allperioddata_df1 = legacysddates_final_df.select(
    col("id").alias("id"),
    col("PeriodType"),
	F.lit(None).alias("Period"),
	F.lit(None).alias("UserGLCode"),
	F.lit(None).alias("currency"),
	F.lit(None).alias("closingbalance"),
	col("categoryid"),
	F.lit(None).alias("member"),
	F.lit(None).alias("ytdtype"),
	F.lit(None).alias("usergldescription"),
	F.lit(None).alias("Source"),
    col("Value"),
    col("pcurrentvalue"),
    col("legacyperiod"),
    col("financialyear"),
    col("openingperiodvalue"),
    col("castingperiod"),
    col("p"),
    col("ppp"),
    col("py"),
    col("pypp"),
    col("ppy"),
    col("p3y"),
    col("p4y"),
	F.lit(None).alias("CastingAdjustmentsID"),
	F.lit(None).alias("Fk_AccountingStandardID"),
 	F.lit(None).alias("Fk_ValidationID"),
	F.lit(None).alias("PeriodValue"),
	F.lit(None).alias("Fk_EntityID"),
	F.lit(None).alias("EntityName"),
	F.lit(None).alias("JournalNumber"),
	F.lit(None).alias("JournalDate"),
	F.lit(None).alias("StandardGLCode"),
	F.lit(None).alias("StandardGLDescription"),
	F.lit(None).alias("DebitCredit"),
	F.lit(None).alias("Fk_CurrencyID"),
	F.lit(None).alias("Amount"),
	F.lit(None).alias("JournalType"),
	F.lit(None).alias("Narration"),
	F.lit(None).alias("Fk_BusinessUnitID"),
	F.lit(None).alias("Roundoff"),
	F.lit(None).alias("AmountsIn"),
	F.lit(None).alias("ReportType"),
	F.lit(None).alias("CreatedDate"),
	F.lit(None).alias("CreatedBy"),
	F.lit(None).alias("ModifiedDate"),
	F.lit(None).alias("ModifiedBy"),
	F.lit(None).alias("IsUserGLCode"),
	F.lit(None).alias("RoundTypeLevel"),
	F.lit(1).alias("typeofdata"),
	*[F.lit(None).cast("integer").alias(col) for col in alldimcols]
)

allperioddata_df1 = allperioddata_df1.select("id","PeriodType","Period","userglcode","currency","closingbalance","categoryid","member","ytdtype","usergldescription","Source","Value","pcurrentvalue","legacyperiod","financialyear","openingperiodvalue","castingperiod","p","ppp","py","pypp","ppy","p3y","p4y","CastingAdjustmentsID","Fk_AccountingStandardID","Fk_ValidationID","PeriodValue","Fk_EntityID","EntityName","JournalNumber","JournalDate","StandardGLCode","StandardGLDescription","DebitCredit","Fk_CurrencyID","Amount","JournalType","Narration","Fk_BusinessUnitID","Roundoff","AmountsIn","ReportType","CreatedDate","CreatedBy","ModifiedDate","ModifiedBy","IsUserGLCode","RoundTypeLevel","typeofdata",*alldimcols)

allperioddata_df2 = castingsdadjustment_df.withColumn("IsUserGLCode", 
                                               when(col("isuserglcode") == True, 1)
                                               .when(col("isuserglcode") == False, 0)
                                               .otherwise(None)).select(
    col("castingadjustmentsid").alias("id"),
	F.lit(None).alias("PeriodType"),
	F.lit(None).alias("Period"),
	col("userglcode"),
	F.lit(None).alias("currency"),
	F.lit(None).alias("closingbalance"),
	col("categoryid"),
	F.lit(None).alias("member"),
	F.lit(None).alias("ytdtype"),
	col("usergldescription"),
	F.lit("Recast Adj.").alias("Source"),
	F.lit(None).alias("Value"),
	F.lit(None).alias("pcurrentvalue"),
	F.lit(None).alias("legacyperiod"),
	col("financialyear"),
	F.lit(None).alias("openingperiodvalue"),
	F.lit(None).alias("castingperiod"),
	F.lit(None).alias("p"),
	F.lit(None).alias("ppp"),
	F.lit(None).alias("py"),
	F.lit(None).alias("pypp"),
	F.lit(None).alias("ppy"),
	F.lit(None).alias("p3y"),
	F.lit(None).alias("p4y"),
	col("castingadjustmentsid"),
    col("fk_accountingstandardid"),
    col("fk_validationid"),
    col("periodvalue"),
	col("fk_entityid"),
	col("entityname"),
    col("journalnumber"),
    col("journaldate"),
    col("standardglcode"),
    col("standardgldescription"),
    col("debitcredit"),
    col("fk_currencyid"),
    col("amount"),
    col("journaltype"),
    col("narration"),
    col("fk_businessunitid"),
	col("roundoff"),
    col("amountsin"),
    col("reporttype"),
    col("createddate"),
    col("createdby"),
    col("modifieddate"),
    col("modifiedby"),
    col("isuserglcode"),
    col("roundtypelevel"),
	F.lit(2).alias("typeofdata"),
 	*alldimcols)

allperioddata_df2 = allperioddata_df2.select("id","PeriodType","Period","userglcode","currency","closingbalance","categoryid","member","ytdtype","usergldescription","Source","Value","pcurrentvalue","legacyperiod","financialyear","openingperiodvalue","castingperiod","p","ppp","py","pypp","ppy","p3y","p4y","CastingAdjustmentsID","Fk_AccountingStandardID","Fk_ValidationID","PeriodValue","Fk_EntityID","EntityName","JournalNumber","JournalDate","StandardGLCode","StandardGLDescription","DebitCredit","Fk_CurrencyID","Amount","JournalType","Narration","Fk_BusinessUnitID","Roundoff","AmountsIn","ReportType","CreatedDate","CreatedBy","ModifiedDate","ModifiedBy","IsUserGLCode","RoundTypeLevel","typeofdata",*alldimcols)

allperioddata_result_df = allperioddata_df.union(allperioddata_df1).union(allperioddata_df2)


In [0]:
#write output table for staging
if ((mappedsdtbftp_deleted_df.count() > 0) &  (mappedsdtbytd_df.count() > 0)):
    print("inside if")
    allperioddata_df3 = mappedsdtbftp_deleted_df.select(
        col("PeriodType"), col("Period"), col("userglcode"), col("currency"), col("closingbalance"),
        col("categoryid"), col("member"), col("ytdtype"),*alldimcols
    ).union(mappedsdtbytd_df.select(
        col("PeriodType"), col("Period"), col("userglcode"), col("currency"), col("closingbalance"),
        col("categoryid"), col("member"), col("ytdtype"),*alldimcols
    )).withColumn("typeofdata", lit("0")).distinct()

    allperioddata_df3 = allperioddata_df3.select(
    F.lit(None).alias("id"),
    col("PeriodType"),
    col("Period"),
    col("UserGLCode"),
    col("currency"),
    col("closingbalance"),
    col("categoryid"),
    col("member"),
    col("ytdtype"),
    F.lit(None).alias("usergldescription"),
    F.lit(None).alias("Source"),
    F.lit(None).alias("Value"),
    F.lit(None).alias("pcurrentvalue"),
    F.lit(None).alias("legacyperiod"),
    F.lit(None).alias("financialyear"),
    F.lit(None).alias("openingperiodvalue"),
    F.lit(None).alias("castingperiod"),
    F.lit(None).alias("p"),
    F.lit(None).alias("ppp"),
    F.lit(None).alias("py"),
    F.lit(None).alias("pypp"),
    F.lit(None).alias("ppy"),
    F.lit(None).alias("p3y"),
    F.lit(None).alias("p4y"),
    F.lit(None).alias("CastingAdjustmentsID"),
    F.lit(None).alias("Fk_AccountingStandardID"),
    F.lit(None).alias("Fk_ValidationID"),
    F.lit(None).alias("PeriodValue"),
    F.lit(None).alias("Fk_EntityID"),
    F.lit(None).alias("EntityName"),
    F.lit(None).alias("JournalNumber"),
    F.lit(None).alias("JournalDate"),
    F.lit(None).alias("StandardGLCode"),
    F.lit(None).alias("StandardGLDescription"),
    F.lit(None).alias("DebitCredit"),
    F.lit(None).alias("Fk_CurrencyID"),
    F.lit(None).alias("Amount"),
    F.lit(None).alias("JournalType"),
    F.lit(None).alias("Narration"),
    F.lit(None).alias("Fk_BusinessUnitID"),
    F.lit(None).alias("Roundoff"),
    F.lit(None).alias("AmountsIn"),
    F.lit(None).alias("ReportType"),
    F.lit(None).alias("CreatedDate"),
    F.lit(None).alias("CreatedBy"),
    F.lit(None).alias("ModifiedDate"),
    F.lit(None).alias("ModifiedBy"),
    F.lit(None).alias("IsUserGLCode"),
    F.lit(None).alias("RoundTypeLevel"),
    col("typeofdata"),
    *alldimcols
)

    allperioddata_result_final_df = allperioddata_result_df.union(allperioddata_df3)
    
    allperioddata_result_final_df.write.format("delta").mode("overwrite").saveAsTable(staging_tbl)
else:

    print("drilldown is 0 but mappeddf are empty")
    allperioddata_result_final_df = allperioddata_result_df
    allperioddata_result_final_df.write.format("delta").mode("overwrite").saveAsTable(staging_tbl)


#write output table for drilldown   
if ((mappedtbddftp_deleted_df.count() > 0) & (mappedtbddytd_df.count() > 0)):
    print("inside else")
    allperioddata_df4 = mappedtbddftp_deleted_df.select(
        col("PeriodType"), col("Period"), col("userglcode"), col("usergldescription"), col("closingbalance"),
        col("member"), col("source"), col("ytdtype"),*alldimcols
    ).union(mappedtbddytd_df.select(
        col("PeriodType"), col("Period"), col("userglcode"), col("usergldescription"), col("closingbalance"),
        col("member"), col("source"), col("ytdtype"),*alldimcols)
    ).withColumn("typeofdata", lit("0")).distinct()

    allperioddata_df4 = allperioddata_df4.select(
    F.lit(None).alias("id"),
    col("PeriodType"),
    col("Period"),
    col("UserGLCode"),
    F.lit(None).alias("currency"),
    col("closingbalance"),
    F.lit(None).alias("categoryid"),
    col("member"),
    col("ytdtype"),
    col("usergldescription"),
    col("Source"),
    F.lit(None).alias("Value"),
    F.lit(None).alias("pcurrentvalue"),
    F.lit(None).alias("legacyperiod"),
    F.lit(None).alias("financialyear"),
    F.lit(None).alias("openingperiodvalue"),
    F.lit(None).alias("castingperiod"),
    F.lit(None).alias("p"),
    F.lit(None).alias("ppp"),
    F.lit(None).alias("py"),
    F.lit(None).alias("pypp"),
    F.lit(None).alias("ppy"),
    F.lit(None).alias("p3y"),
    F.lit(None).alias("p4y"),
    F.lit(None).alias("CastingAdjustmentsID"),
    F.lit(None).alias("Fk_AccountingStandardID"),
    F.lit(None).alias("Fk_ValidationID"),
    F.lit(None).alias("PeriodValue"),
    F.lit(None).alias("Fk_EntityID"),
    F.lit(None).alias("EntityName"),
    F.lit(None).alias("JournalNumber"),
    F.lit(None).alias("JournalDate"),
    F.lit(None).alias("StandardGLCode"),
    F.lit(None).alias("StandardGLDescription"),
    F.lit(None).alias("DebitCredit"),
    F.lit(None).alias("Fk_CurrencyID"),
    F.lit(None).alias("Amount"),
    F.lit(None).alias("JournalType"),
    F.lit(None).alias("Narration"),
    F.lit(None).alias("Fk_BusinessUnitID"),
    F.lit(None).alias("Roundoff"),
    F.lit(None).alias("AmountsIn"),
    F.lit(None).alias("ReportType"),
    F.lit(None).alias("CreatedDate"),
    F.lit(None).alias("CreatedBy"),
    F.lit(None).alias("ModifiedDate"),
    F.lit(None).alias("ModifiedBy"),
    F.lit(None).alias("IsUserGLCode"),
    F.lit(None).alias("RoundTypeLevel"),
    col("typeofdata"),
    *alldimcols
)
    
    allperioddata_result_final_df = allperioddata_result_df.union(allperioddata_df4)
    allperioddata_result_final_df.write.format("delta").mode("overwrite").saveAsTable(drilldown_tbl)
else:
    print("drilldown is 1 but mappeddf are empty")
    allperioddata_result_final_df = allperioddata_result_df
    allperioddata_result_final_df.write.format("delta").mode("overwrite").saveAsTable(drilldown_tbl)